In [2]:
!pip install opencv-python
!pip install pillow
!pip install pyttsx3
!pip install opencv-python-headless pyttsx3
!pip install matplotlib


In [4]:
!pip install torch torchvision torchaudio
!pip install pygame
!pip install ultralytics


In [ ]:
#Original sin modificaciones, se demora la parte de la deteccion de objetos

In [11]:
import tkinter as tk
from tkinter import messagebox
from threading import Thread
import cv2
import datetime
import numpy as np
from PIL import Image, ImageTk
import pyttsx3
import os

class CamaraApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Detector de imágenes - Demo")
        self.root.config(bg="#2e2e2e")

        # Variables de estado
        self.camara_activa = False
        self.cap = None
        self.thread = None
        self.grabando_video = False
        self.video_writer = None
        self.detectando_objetos = False
        self.engine = pyttsx3.init()
        self.historial_detecciones = {}

        # Cargar el modelo de detección de objetos (YOLO)
        self.net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
        self.layer_names = self.net.getLayerNames()
        self.output_layers = [self.layer_names[i - 1] for i in self.net.getUnconnectedOutLayers()]
        self.classes = []
        with open("coco.names", "r") as f:
            self.classes = [line.strip() for line in f.readlines()]

        # Crear interfaz
        self.frame_principal = tk.Frame(root, bg="#2e2e2e")
        self.frame_principal.pack(fill=tk.BOTH, expand=True)

        # Frame para el video y los controles
        self.frame_video = tk.Frame(self.frame_principal, bg="#2e2e2e")
        self.frame_video.pack(side=tk.LEFT, padx=10, pady=10)

        # Frame para el historial
        self.frame_historial = tk.Frame(self.frame_principal, bg="#2e2e2e", width=300)
        self.frame_historial.pack(side=tk.RIGHT, fill=tk.Y, padx=10, pady=10)

        # Mostrar logo
        self.logo_imagen_tk = None  # Declarar la variable a nivel de instancia
        self.cargar_logo()

        # Botones
        self.frame_botones = tk.Frame(self.frame_video, bg="#3c3c3c")
        self.frame_botones.pack(pady=10, padx=10, fill=tk.X)

        self.boton_abrir = tk.Button(self.frame_botones, text="Abrir Cámara", command=self.abrir_camara, width=20,
                                     bg="#4CAF50", fg="white", font=("Arial", 12))
        self.boton_abrir.grid(row=0, column=0, padx=10, pady=10)

        self.boton_cerrar = tk.Button(self.frame_botones, text="Cerrar Cámara", command=self.cerrar_camara, width=20,
                                      state=tk.DISABLED, bg="#F44336", fg="white", font=("Arial", 12))
        self.boton_cerrar.grid(row=0, column=1, padx=10, pady=10)

        self.boton_foto = tk.Button(self.frame_botones, text="Tomar Foto", command=self.tomar_foto, width=20,
                                    state=tk.DISABLED, bg="#2196F3", fg="white", font=("Arial", 12))
        self.boton_foto.grid(row=1, column=0, padx=10, pady=10)

        self.boton_video = tk.Button(self.frame_botones, text="Grabar Video", command=self.grabar_video, width=20,
                                     state=tk.DISABLED, bg="#FF9800", fg="white", font=("Arial", 12))
        self.boton_video.grid(row=1, column=1, padx=10, pady=10)

        self.boton_deteccion = tk.Button(self.frame_botones, text="Detectar Objetos", command=self.toggle_deteccion,
                                         width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_deteccion.grid(row=2, column=0, padx=10, pady=10)

        self.boton_salir = tk.Button(self.frame_botones, text="Salir", command=self.salir, width=20,
                                     bg="#9E9E9E", fg="white", font=("Arial", 12))
        self.boton_salir.grid(row=2, column=1, padx=10, pady=10)

        # Crear canvas para video
        self.canvas = tk.Canvas(self.frame_video, width=640, height=480)
        self.canvas.pack()

        # Etiquetas para el historial
        self.label_historial_titulo = tk.Label(self.frame_historial, text="Historial", bg="#2e2e2e",
                                               fg="white", font=("Arial", 14, "bold"))
        self.label_historial_titulo.pack(pady=10)

        self.label_historial = tk.Label(self.frame_historial, text="", bg="#2e2e2e", fg="white", font=("Arial", 12),
                                        anchor="nw", justify="left")
        self.label_historial.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

    def cargar_logo(self):
        try:
            logo_path = "Logo1.png"
            if os.path.exists(logo_path):
                self.logo_imagen = Image.open(logo_path).resize((300, 100), Image.Resampling.LANCZOS)
                self.logo_imagen_tk = ImageTk.PhotoImage(self.logo_imagen)
                self.label_logo = tk.Label(self.frame_video, image=self.logo_imagen_tk, bg="#2e2e2e")
                self.label_logo.pack(pady=20)
            else:
                raise FileNotFoundError(f"El archivo {logo_path} no se encuentra.")
        except Exception as e:
            print(f"Error al cargar la imagen: {e}")

    def abrir_camara(self):
        """Abre la cámara y comienza a capturar el video."""
        if not self.camara_activa:
            self.cap = cv2.VideoCapture(0)
            if not self.cap.isOpened():
                messagebox.showerror("Error", "No se pudo abrir la cámara.")
                return

            self.camara_activa = True
            self.boton_abrir.config(state=tk.DISABLED)
            self.boton_cerrar.config(state=tk.NORMAL)
            self.boton_foto.config(state=tk.NORMAL)
            self.boton_video.config(state=tk.NORMAL)
            self.boton_deteccion.config(state=tk.NORMAL)

            # Registrar en el historial que la cámara ha sido abierta
            self.historial_detecciones[datetime.datetime.now().strftime("%Y%m%d_%H%M%S")] = "Cámara abierta"
            self.actualizar_historial()

            self.thread = Thread(target=self.actualizar_video)
            self.thread.daemon = True
            self.thread.start()

    def actualizar_video(self):
        """Captura el video y actualiza la imagen en el canvas de Tkinter."""
        while self.camara_activa:
            ret, frame = self.cap.read()
            if not ret:
                break

            if self.detectando_objetos:
                # Realizar detección de objetos
                blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
                self.net.setInput(blob)
                outs = self.net.forward(self.output_layers)

                class_ids = []
                confidences = []
                boxes = []

                objetos_detectados = []

                for out in outs:
                    for detection in out:
                        scores = detection[5:]
                        class_id = np.argmax(scores)
                        confidence = scores[class_id]
                        if confidence > 0.5:
                            center_x = int(detection[0] * frame.shape[1])
                            center_y = int(detection[1] * frame.shape[0])
                            w = int(detection[2] * frame.shape[1])
                            h = int(detection[3] * frame.shape[0])
                            x = int(center_x - w / 2)
                            y = int(center_y - h / 2)
                            boxes.append([x, y, w, h])
                            confidences.append(float(confidence))
                            class_ids.append(class_id)
                            objetos_detectados.append(self.classes[class_id])

                indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

                if len(indexes) > 0:
                    for i in indexes.flatten():
                        x, y, w, h = boxes[i]
                        label = str(self.classes[class_ids[i]])
                        color = (0, 255, 0)
                        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                # Anunciar objetos detectados
                if objetos_detectados:
                    self.engine.say(", ".join(set(objetos_detectados)))  # Decir los objetos detectados
                    self.engine.runAndWait()

                # Actualizar historial con los objetos detectados
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                self.historial_detecciones[timestamp] = f"Objetos detectados: {', '.join(set(objetos_detectados))}"
                self.actualizar_historial()

            # Mostrar el frame en el canvas
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_pil = Image.fromarray(frame_rgb)
            frame_tk = ImageTk.PhotoImage(frame_pil)
            self.canvas.create_image(0, 0, anchor=tk.NW, image=frame_tk)
            self.canvas.image = frame_tk

    def cerrar_camara(self):
        """Cierra la cámara y detiene la captura de video."""
        if self.camara_activa:
            self.camara_activa = False
            self.cap.release()
            self.boton_abrir.config(state=tk.NORMAL)
            self.boton_cerrar.config(state=tk.DISABLED)
            self.boton_foto.config(state=tk.DISABLED)
            self.boton_video.config(state=tk.DISABLED)
            self.boton_deteccion.config(state=tk.DISABLED)

            # Registrar en el historial que la cámara ha sido cerrada
            self.historial_detecciones[datetime.datetime.now().strftime("%Y%m%d_%H%M%S")] = "Cámara cerrada"
            self.actualizar_historial()

            if self.thread is not None:
                self.thread.join()

    def tomar_foto(self):
        """Toma una foto y la guarda."""
        if self.camara_activa:
            ret, frame = self.cap.read()
            if ret:
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                photo_filename = f"foto_{timestamp}.jpg"
                cv2.imwrite(photo_filename, frame)
                self.historial_detecciones[timestamp] = "Foto tomada"
                self.actualizar_historial()
                messagebox.showinfo("Foto tomada", f"La foto se guardó como {photo_filename}.")

    def grabar_video(self):
        """Inicia o detiene la grabación de video."""
        if self.camara_activa:
            self.grabando_video = not self.grabando_video

            if self.grabando_video:
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                video_filename = f"video_{timestamp}.avi"
                self.video_writer = cv2.VideoWriter(video_filename, cv2.VideoWriter_fourcc(*'XVID'), 20.0, (640, 480))
                self.boton_video.config(text="Detener Video")
                self.historial_detecciones[timestamp] = "Grabación de video iniciada"
            else:
                self.video_writer.release()
                self.boton_video.config(text="Grabar Video")
                self.historial_detecciones[timestamp] = "Grabación de video detenida"
                self.actualizar_historial()
                messagebox.showinfo("Grabación detenida", "La grabación de video ha sido detenida.")

    def toggle_deteccion(self):
        """Activa o desactiva la detección de objetos."""
        self.detectando_objetos = not self.detectando_objetos
        estado = "activada" if self.detectando_objetos else "desactivada"
        self.historial_detecciones[datetime.datetime.now().strftime("%Y%m%d_%H%M%S")] = f"Detección de objetos {estado}"
        self.actualizar_historial()

    def actualizar_historial(self):
        """Actualiza el historial mostrado en la interfaz."""
        historial_texto = "\n".join([f"{hora}: {accion}" for hora, accion in self.historial_detecciones.items()])
        self.label_historial.config(text=historial_texto)

    def salir(self):
        """Cierra la aplicación."""
        if self.camara_activa:
            self.cerrar_camara()
        self.root.quit()

# Crear ventana principal
root = tk.Tk()
app = CamaraApp(root)
root.mainloop()


In [ ]:
#Modificacion del codigo base se mejora el rendimiento, se ajusta la voz para que anuncie las respuestas en un intervalo de 1 segundo 

In [1]:
import tkinter as tk
from tkinter import messagebox
from threading import Thread
import cv2
import datetime
import numpy as np
from PIL import Image, ImageTk
import pyttsx3
import os
import time

class CamaraApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Detector de imágenes - Demo")
        self.root.config(bg="#2e2e2e")

        # Variables de estado
        self.camara_activa = False
        self.cap = None
        self.thread = None
        self.grabando_video = False
        self.video_writer = None
        self.detectando_objetos = False
        self.engine = pyttsx3.init()
        self.historial_detecciones = {}
        self.objetos_para_anunciar = set()

        # Cargar el modelo de detección de objetos (YOLO)
        self.net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
        self.layer_names = self.net.getLayerNames()
        self.output_layers = [self.layer_names[i - 1] for i in self.net.getUnconnectedOutLayers()]
        self.classes = []
        with open("coco.names", "r") as f:
            self.classes = [line.strip() for line in f.readlines()]

        # Crear interfaz
        self.frame_principal = tk.Frame(root, bg="#2e2e2e")
        self.frame_principal.pack(fill=tk.BOTH, expand=True)

        # Frame para el video y los controles
        self.frame_video = tk.Frame(self.frame_principal, bg="#2e2e2e")
        self.frame_video.pack(side=tk.LEFT, padx=10, pady=10)

        # Frame para el historial
        self.frame_historial = tk.Frame(self.frame_principal, bg="#2e2e2e", width=300)
        self.frame_historial.pack(side=tk.RIGHT, fill=tk.Y, padx=10, pady=10)

        # Mostrar logo
        self.logo_imagen_tk = None  # Declarar la variable a nivel de instancia
        self.cargar_logo()

        # Botones
        self.frame_botones = tk.Frame(self.frame_video, bg="#3c3c3c")
        self.frame_botones.pack(pady=10, padx=10, fill=tk.X)

        self.boton_abrir = tk.Button(self.frame_botones, text="Abrir Cámara", command=self.abrir_camara, width=20,
                                     bg="#4CAF50", fg="white", font=("Arial", 12))
        self.boton_abrir.grid(row=0, column=0, padx=10, pady=10)

        self.boton_cerrar = tk.Button(self.frame_botones, text="Cerrar Cámara", command=self.cerrar_camara, width=20,
                                      state=tk.DISABLED, bg="#F44336", fg="white", font=("Arial", 12))
        self.boton_cerrar.grid(row=0, column=1, padx=10, pady=10)

        self.boton_foto = tk.Button(self.frame_botones, text="Tomar Foto", command=self.tomar_foto, width=20,
                                    state=tk.DISABLED, bg="#2196F3", fg="white", font=("Arial", 12))
        self.boton_foto.grid(row=1, column=0, padx=10, pady=10)

        self.boton_video = tk.Button(self.frame_botones, text="Grabar Video", command=self.grabar_video, width=20,
                                     state=tk.DISABLED, bg="#FF9800", fg="white", font=("Arial", 12))
        self.boton_video.grid(row=1, column=1, padx=10, pady=10)

        self.boton_deteccion = tk.Button(self.frame_botones, text="Detectar Objetos", command=self.toggle_deteccion,
                                         width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_deteccion.grid(row=2, column=0, padx=10, pady=10)

        self.boton_salir = tk.Button(self.frame_botones, text="Salir", command=self.salir, width=20,
                                     bg="#9E9E9E", fg="white", font=("Arial", 12))
        self.boton_salir.grid(row=2, column=1, padx=10, pady=10)

        # Crear canvas para video
        self.canvas = tk.Canvas(self.frame_video, width=640, height=480)
        self.canvas.pack()

        # Etiquetas para el historial
        self.label_historial_titulo = tk.Label(self.frame_historial, text="Historial", bg="#2e2e2e",
                                               fg="white", font=("Arial", 14, "bold"))
        self.label_historial_titulo.pack(pady=10)

        self.label_historial = tk.Label(self.frame_historial, text="", bg="#2e2e2e", fg="white", font=("Arial", 12),
                                        anchor="nw", justify="left")
        self.label_historial.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

    def cargar_logo(self):
        try:
            logo_path = "Logo1.png"
            if os.path.exists(logo_path):
                self.logo_imagen = Image.open(logo_path).resize((300, 100), Image.Resampling.LANCZOS)
                self.logo_imagen_tk = ImageTk.PhotoImage(self.logo_imagen)
                self.label_logo = tk.Label(self.frame_video, image=self.logo_imagen_tk, bg="#2e2e2e")
                self.label_logo.pack(pady=20)
            else:
                raise FileNotFoundError(f"El archivo {logo_path} no se encuentra.")
        except Exception as e:
            print(f"Error al cargar la imagen: {e}")

    def abrir_camara(self):
        """Abre la cámara y comienza a capturar el video."""
        if not self.camara_activa:
            self.cap = cv2.VideoCapture(0)
            if not self.cap.isOpened():
                messagebox.showerror("Error", "No se pudo abrir la cámara.")
                return

            self.camara_activa = True
            self.boton_abrir.config(state=tk.DISABLED)
            self.boton_cerrar.config(state=tk.NORMAL)
            self.boton_foto.config(state=tk.NORMAL)
            self.boton_video.config(state=tk.NORMAL)
            self.boton_deteccion.config(state=tk.NORMAL)

            # Registrar en el historial que la cámara ha sido abierta
            self.historial_detecciones[datetime.datetime.now().strftime("%Y%m%d_%H%M%S")] = "Cámara abierta"
            self.actualizar_historial()

            self.thread = Thread(target=self.actualizar_video)
            self.thread.daemon = True
            self.thread.start()

            # Hilo para anunciar objetos detectados
            self.thread_anunciar = Thread(target=self.anunciar_objetos_detectados)
            self.thread_anunciar.daemon = True
            self.thread_anunciar.start()

    def actualizar_video(self):
        """Captura el video y actualiza la imagen en el canvas de Tkinter."""
        while self.camara_activa:
            ret, frame = self.cap.read()
            if not ret:
                break

            if self.detectando_objetos:
                # Realizar detección de objetos
                blob = cv2.dnn.blobFromImage(frame, 0.00392, (320, 320), (0, 0, 0), True, crop=False)
                self.net.setInput(blob)
                outs = self.net.forward(self.output_layers)

                class_ids = []
                confidences = []
                boxes = []

                for out in outs:
                    for detection in out:
                        scores = detection[5:]
                        class_id = np.argmax(scores)
                        confidence = scores[class_id]
                        if confidence > 0.6:
                            center_x = int(detection[0] * frame.shape[1])
                            center_y = int(detection[1] * frame.shape[0])
                            w = int(detection[2] * frame.shape[1])
                            h = int(detection[3] * frame.shape[0])
                            x = int(center_x - w / 2)
                            y = int(center_y - h / 2)
                            boxes.append([x, y, w, h])
                            confidences.append(float(confidence))
                            class_ids.append(class_id)

                indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.6, 0.4)

                if len(indexes) > 0:
                    for i in indexes.flatten():
                        x, y, w, h = boxes[i]
                        label = str(self.classes[class_ids[i]])
                        color = (0, 255, 0)
                        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                        self.objetos_para_anunciar.add(label)

                # Actualizar historial con los objetos detectados
                if self.objetos_para_anunciar:
                    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                    objetos_detectados = ", ".join(self.objetos_para_anunciar)
                    self.historial_detecciones[timestamp] = f"Objetos detectados: {objetos_detectados}"
                    self.actualizar_historial()

            # Mostrar el frame en el canvas
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_pil = Image.fromarray(frame_rgb)
            frame_tk = ImageTk.PhotoImage(frame_pil)
            self.canvas.create_image(0, 0, anchor=tk.NW, image=frame_tk)
            self.canvas.image = frame_tk

    def anunciar_objetos_detectados(self):
        """Pronuncia los objetos detectados en intervalos regulares."""
        while True:
            if self.objetos_para_anunciar:
                objetos = ", ".join(self.objetos_para_anunciar)
                self.engine.say(f"Objetos detectados: {objetos}")
                self.engine.runAndWait()
                self.objetos_para_anunciar.clear()
            time.sleep(1)  # Intervalo entre anuncios

    def cerrar_camara(self):
        """Cierra la cámara y detiene la captura de video."""
        if self.camara_activa:
            self.camara_activa = False
            self.cap.release()
            self.boton_abrir.config(state=tk.NORMAL)
            self.boton_cerrar.config(state=tk.DISABLED)
            self.boton_foto.config(state=tk.DISABLED)
            self.boton_video.config(state=tk.DISABLED)
            self.boton_deteccion.config(state=tk.DISABLED)

            # Registrar en el historial que la cámara ha sido cerrada
            self.historial_detecciones[datetime.datetime.now().strftime("%Y%m%d_%H%M%S")] = "Cámara cerrada"
            self.actualizar_historial()

            if self.thread is not None:
                self.thread.join()

    def tomar_foto(self):
        """Toma una foto y la guarda."""
        if self.camara_activa:
            ret, frame = self.cap.read()
            if ret:
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                photo_filename = f"foto_{timestamp}.jpg"
                cv2.imwrite(photo_filename, frame)
                self.historial_detecciones[timestamp] = "Foto tomada"
                self.actualizar_historial()
                messagebox.showinfo("Foto tomada", f"La foto se guardó como {photo_filename}.")

    def grabar_video(self):
        """Inicia o detiene la grabación de video."""
        if self.camara_activa:
            self.grabando_video = not self.grabando_video

            if self.grabando_video:
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                video_filename = f"video_{timestamp}.avi"
                self.video_writer = cv2.VideoWriter(video_filename, cv2.VideoWriter_fourcc(*'XVID'), 20.0, (640, 480))
                self.boton_video.config(text="Detener Video")
                self.historial_detecciones[timestamp] = "Grabación de video iniciada"
            else:
                self.video_writer.release()
                self.boton_video.config(text="Grabar Video")
                self.historial_detecciones[timestamp] = "Grabación de video detenida"
                self.actualizar_historial()
                messagebox.showinfo("Grabación detenida", "La grabación de video ha sido detenida.")

    def toggle_deteccion(self):
        """Activa o desactiva la detección de objetos."""
        self.detectando_objetos = not self.detectando_objetos
        estado = "activada" if self.detectando_objetos else "desactivada"
        self.historial_detecciones[datetime.datetime.now().strftime("%Y%m%d_%H%M%S")] = f"Detección de objetos {estado}"
        self.actualizar_historial()

    def actualizar_historial(self):
        """Actualiza el historial mostrado en la interfaz."""
        historial_texto = "\n".join([f"{hora}: {accion}" for hora, accion in self.historial_detecciones.items()])
        self.label_historial.config(text=historial_texto)
        time.sleep(1)
    def salir(self):
        """Cierra la aplicación."""
        if self.camara_activa:
            self.cerrar_camara()
        self.root.quit()

# Crear ventana principal
root = tk.Tk()
app = CamaraApp(root)
root.mainloop()

In [ ]:
#Codigo con la modificacion final para  la seguridad 

In [1]:
import tkinter as tk
from tkinter import messagebox
from threading import Thread
import cv2
import datetime
import numpy as np
from PIL import Image, ImageTk
import pyttsx3
import os
import time
import pygame  # Importa pygame para la reproducción de audio

class CamaraApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Detector de imágenes - Demo")
        self.root.config(bg="#2e2e2e")

        # Inicializar Pygame para el audio
        pygame.mixer.init()

        # Variables de estado
        self.camara_activa = False
        self.cap = None
        self.thread = None
        self.grabando_video = False
        self.video_writer = None
        self.detectando_objetos = False
        self.engine = pyttsx3.init()
        self.historial_detecciones = {}
        self.objetos_para_anunciar = set()
        self.last_alert_time = 0  # Para gestionar el tiempo de alerta

        # Cargar el modelo de detección de objetos (YOLO)
        self.net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
        self.layer_names = self.net.getLayerNames()
        self.output_layers = [self.layer_names[i - 1] for i in self.net.getUnconnectedOutLayers()]
        self.classes = []
        with open("coco.names", "r") as f:
            self.classes = [line.strip() for line in f.readlines()]

        # Crear interfaz
        self.frame_principal = tk.Frame(root, bg="#2e2e2e")
        self.frame_principal.pack(fill=tk.BOTH, expand=True)

        # Frame para el video y los controles
        self.frame_video = tk.Frame(self.frame_principal, bg="#2e2e2e")
        self.frame_video.pack(side=tk.LEFT, padx=10, pady=10)

        # Frame para el historial
        self.frame_historial = tk.Frame(self.frame_principal, bg="#2e2e2e", width=300)
        self.frame_historial.pack(side=tk.RIGHT, fill=tk.Y, padx=10, pady=10)

        # Mostrar logo
        self.logo_imagen_tk = None
        self.cargar_logo()

        # Botones
        self.frame_botones = tk.Frame(self.frame_video, bg="#3c3c3c")
        self.frame_botones.pack(pady=10, padx=10, fill=tk.X)

        self.boton_abrir = tk.Button(self.frame_botones, text="Abrir Cámara", command=self.abrir_camara, width=20,
                                     bg="#4CAF50", fg="white", font=("Arial", 12))
        self.boton_abrir.grid(row=0, column=0, padx=10, pady=10)

        self.boton_cerrar = tk.Button(self.frame_botones, text="Cerrar Cámara", command=self.cerrar_camara, width=20,
                                      state=tk.DISABLED, bg="#F44336", fg="white", font=("Arial", 12))
        self.boton_cerrar.grid(row=0, column=1, padx=10, pady=10)

        self.boton_foto = tk.Button(self.frame_botones, text="Tomar Foto", command=self.tomar_foto, width=20,
                                    state=tk.DISABLED, bg="#2196F3", fg="white", font=("Arial", 12))
        self.boton_foto.grid(row=1, column=0, padx=10, pady=10)

        self.boton_video = tk.Button(self.frame_botones, text="Grabar Video", command=self.grabar_video, width=20,
                                     state=tk.DISABLED, bg="#FF9800", fg="white", font=("Arial", 12))
        self.boton_video.grid(row=1, column=1, padx=10, pady=10)

        self.boton_deteccion = tk.Button(self.frame_botones, text="Detectar Objetos", command=self.toggle_deteccion,
                                         width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_deteccion.grid(row=2, column=0, padx=10, pady=10)

        self.boton_salir = tk.Button(self.frame_botones, text="Salir", command=self.salir, width=20,
                                     bg="#9E9E9E", fg="white", font=("Arial", 12))
        self.boton_salir.grid(row=2, column=1, padx=10, pady=10)

        # Crear canvas para video
        self.canvas = tk.Canvas(self.frame_video, width=640, height=480)
        self.canvas.pack()

        # Etiquetas para el historial
        self.label_historial_titulo = tk.Label(self.frame_historial, text="Historial", bg="#2e2e2e",
                                               fg="white", font=("Arial", 14, "bold"))
        self.label_historial_titulo.pack(pady=10)

        self.label_historial = tk.Label(self.frame_historial, text="", bg="#2e2e2e", fg="white", font=("Arial", 12),
                                        anchor="nw", justify="left")
        self.label_historial.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

    def cargar_logo(self):
        try:
            logo_path = "Logo1.png"
            if os.path.exists(logo_path):
                self.logo_imagen = Image.open(logo_path).resize((300, 100), Image.Resampling.LANCZOS)
                self.logo_imagen_tk = ImageTk.PhotoImage(self.logo_imagen)
                self.label_logo = tk.Label(self.frame_video, image=self.logo_imagen_tk, bg="#2e2e2e")
                self.label_logo.pack(pady=20)
            else:
                raise FileNotFoundError(f"El archivo {logo_path} no se encuentra.")
        except Exception as e:
            print(f"Error al cargar la imagen: {e}")

    def abrir_camara(self):
        """Abre la cámara y comienza a capturar el video."""
        if not self.camara_activa:
            self.cap = cv2.VideoCapture(0)
            if not self.cap.isOpened():
                messagebox.showerror("Error", "No se pudo abrir la cámara.")
                return

            self.camara_activa = True
            self.boton_abrir.config(state=tk.DISABLED)
            self.boton_cerrar.config(state=tk.NORMAL)
            self.boton_foto.config(state=tk.NORMAL)
            self.boton_video.config(state=tk.NORMAL)
            self.boton_deteccion.config(state=tk.NORMAL)

            # Registrar en el historial que la cámara ha sido abierta
            self.historial_detecciones[datetime.datetime.now().strftime("%Y%m%d_%H%M%S")] = "Cámara abierta"
            self.actualizar_historial()

            self.thread = Thread(target=self.actualizar_video)
            self.thread.daemon = True
            self.thread.start()

            # Hilo para anunciar objetos detectados
            self.thread_anunciar = Thread(target=self.anunciar_objetos_detectados)
            self.thread_anunciar.daemon = True
            self.thread_anunciar.start()

    def actualizar_video(self):
        """Captura el video y actualiza la imagen en el canvas de Tkinter."""
        frame_count = 0  # Contador de frames
        while self.camara_activa:
            ret, frame = self.cap.read()
            if not ret:
                break

            if self.detectando_objetos and frame_count % 3 == 0:  # Procesar cada 3 frames
                # Realizar detección de objetos
                blob = cv2.dnn.blobFromImage(frame, 0.00392, (320, 320), (0, 0, 0), True, crop=False)
                self.net.setInput(blob)
                outs = self.net.forward(self.output_layers)

                class_ids = []
                confidences = []
                boxes = []

                for out in outs:
                    for detection in out:
                        scores = detection[5:]
                        class_id = np.argmax(scores)
                        confidence = scores[class_id]
                        if confidence > 0.6:
                            center_x = int(detection[0] * frame.shape[1])
                            center_y = int(detection[1] * frame.shape[0])
                            w = int(detection[2] * frame.shape[1])
                            h = int(detection[3] * frame.shape[0])
                            x = int(center_x - w / 2)
                            y = int(center_y - h / 2)
                            boxes.append([x, y, w, h])
                            confidences.append(float(confidence))
                            class_ids.append(class_id)

                indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.6, 0.4)

                if len(indexes) > 0:
                    for i in indexes.flatten():
                        x, y, w, h = boxes[i]
                        label = str(self.classes[class_ids[i]])
                        color = (0, 255, 0)
                        # Verificar si el objeto es un arma
                        if label in ["knife", "pistol"]:  # Ajusta estos nombres según tu archivo coco.names
                            color = (255, 0, 0)  # Contorno rojo
                            current_time = time.time()
                            if current_time - self.last_alert_time > 3:  # Reproducir sonido cada 3 segundos
                                self.reproducir_alerta()  # Llama a la función de reproducción de audio
                                self.last_alert_time = current_time
                        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                        self.objetos_para_anunciar.add(label)

                # Actualizar historial con los objetos detectados
                if self.objetos_para_anunciar:
                    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                    objetos_detectados = ", ".join(self.objetos_para_anunciar)
                    self.historial_detecciones[timestamp] = f"Objetos detectados: {objetos_detectados}"
                    self.actualizar_historial()

            # Mostrar el frame en el canvas
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_pil = Image.fromarray(frame_rgb)
            frame_tk = ImageTk.PhotoImage(frame_pil)
            self.canvas.create_image(0, 0, anchor=tk.NW, image=frame_tk)
            self.canvas.image = frame_tk

            frame_count += 1  # Incrementar el contador de frames

    def anunciar_objetos_detectados(self):
        """Pronuncia los objetos detectados en intervalos regulares."""
        while True:
            if self.objetos_para_anunciar:
                objetos = ", ".join(self.objetos_para_anunciar)
                self.engine.say(f"Objetos detectados: {objetos}")
                self.engine.runAndWait()
                self.objetos_para_anunciar.clear()
            time.sleep(1)  # Intervalo entre anuncios

    def cerrar_camara(self):
        """Cierra la cámara y detiene la captura de video."""
        if self.camara_activa:
            self.camara_activa = False
            self.cap.release()
            self.boton_abrir.config(state=tk.NORMAL)
            self.boton_cerrar.config(state=tk.DISABLED)
            self.boton_foto.config(state=tk.DISABLED)
            self.boton_video.config(state=tk.DISABLED)
            self.boton_deteccion.config(state=tk.DISABLED)

            # Registrar en el historial que la cámara ha sido cerrada
            self.historial_detecciones[datetime.datetime.now().strftime("%Y%m%d_%H%M%S")] = "Cámara cerrada"
            self.actualizar_historial()

            if self.thread is not None:
                self.thread.join()

    def tomar_foto(self):
        """Toma una foto y la guarda."""
        if self.camara_activa:
            ret, frame = self.cap.read()
            if ret:
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                photo_filename = f"foto_{timestamp}.jpg"
                cv2.imwrite(photo_filename, frame)
                self.historial_detecciones[timestamp] = "Foto tomada"
                self.actualizar_historial()
                messagebox.showinfo("Foto tomada", f"La foto se guardó como {photo_filename}.")

    def grabar_video(self):
        """Inicia o detiene la grabación de video."""
        if self.camara_activa:
            self.grabando_video = not self.grabando_video

            if self.grabando_video:
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                video_filename = f"video_{timestamp}.avi"
                self.video_writer = cv2.VideoWriter(video_filename, cv2.VideoWriter_fourcc(*'XVID'), 20.0, (640, 480))
                self.boton_video.config(text="Detener Video")
                self.historial_detecciones[timestamp] = "Grabación de video iniciada"
            else:
                self.video_writer.release()
                self.boton_video.config(text="Grabar Video")
                self.historial_detecciones[timestamp] = "Grabación de video detenida"
                self.actualizar_historial()
                messagebox.showinfo("Grabación detenida", "La grabación de video ha sido detenida.")

    def toggle_deteccion(self):
        """Activa o desactiva la detección de objetos."""
        self.detectando_objetos = not self.detectando_objetos
        estado = "activada" if self.detectando_objetos else "desactivada"
        self.historial_detecciones[datetime.datetime.now().strftime("%Y%m%d_%H%M%S")] = f"Detección de objetos {estado}"
        self.actualizar_historial()

    def actualizar_historial(self):
        """Actualiza el historial mostrado en la interfaz."""
        historial_texto = "\n".join([f"{hora}: {accion}" for hora, accion in self.historial_detecciones.items()])
        self.label_historial.config(text=historial_texto)
        time.sleep(1)

    def reproducir_alerta(self):
        """Reproduce el sonido de alerta."""
        try:
            pygame.mixer.music.load("alerta.mp3")
            pygame.mixer.music.play()
        except Exception as e:
            print(f"Error al reproducir el sonido: {e}")

    def salir(self):
        """Cierra la aplicación."""
        if self.camara_activa:
            self.cerrar_camara()
        self.root.quit()

# Crear ventana principal
root = tk.Tk()
app = CamaraApp(root)
root.mainloop()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
#se mejora el codigo para generar mejores reportes y se mejora la funcion del historial

In [ ]:
import tkinter as tk
from tkinter import messagebox
from threading import Thread
import cv2
import datetime
import numpy as np
from PIL import Image, ImageTk
import pyttsx3
import os
import time
import pygame

class CamaraApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Detector de imágenes - Demo")
        self.root.config(bg="#2e2e2e")

        pygame.mixer.init()

        self.camara_activa = False
        self.cap = None
        self.thread = None
        self.grabando_video = False
        self.video_writer = None
        self.detectando_objetos = False
        self.engine = pyttsx3.init()
        self.historial_detecciones = {}
        self.objetos_para_anunciar = set()
        self.last_alert_time = 0
        self.voz_activar = True  # Estado de la voz
        self.alarma_activar = True  # Estado de la alarma

        self.net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
        self.layer_names = self.net.getLayerNames()
        self.output_layers = [self.layer_names[i - 1] for i in self.net.getUnconnectedOutLayers()]
        self.classes = []
        with open("coco.names", "r") as f:
            self.classes = [line.strip() for line in f.readlines()]

        self.frame_principal = tk.Frame(root, bg="#2e2e2e")
        self.frame_principal.pack(fill=tk.BOTH, expand=True)

        self.frame_video = tk.Frame(self.frame_principal, bg="#2e2e2e")
        self.frame_video.pack(side=tk.LEFT, padx=10, pady=10)

        self.frame_historial = tk.Frame(self.frame_principal, bg="#2e2e2e", width=300)
        self.frame_historial.pack(side=tk.RIGHT, fill=tk.Y, padx=10, pady=10)

        self.logo_imagen_tk = None
        self.cargar_logo()

        self.frame_botones = tk.Frame(self.frame_video, bg="#3c3c3c")
        self.frame_botones.pack(pady=10, padx=10, fill=tk.X)

        self.boton_abrir = tk.Button(self.frame_botones, text="Abrir Cámara", command=self.abrir_camara, width=20,
                                     bg="#4CAF50", fg="white", font=("Arial", 12))
        self.boton_abrir.grid(row=0, column=0, padx=10, pady=10)

        self.boton_cerrar = tk.Button(self.frame_botones, text="Cerrar Cámara", command=self.cerrar_camara, width=20,
                                      state=tk.DISABLED, bg="#F44336", fg="white", font=("Arial", 12))
        self.boton_cerrar.grid(row=0, column=1, padx=10, pady=10)

        self.boton_foto = tk.Button(self.frame_botones, text="Tomar Foto", command=self.tomar_foto, width=20,
                                    state=tk.DISABLED, bg="#2196F3", fg="white", font=("Arial", 12))
        self.boton_foto.grid(row=1, column=0, padx=10, pady=10)

        self.boton_video = tk.Button(self.frame_botones, text="Grabar Video", command=self.grabar_video, width=20,
                                     state=tk.DISABLED, bg="#FF9800", fg="white", font=("Arial", 12))
        self.boton_video.grid(row=1, column=1, padx=10, pady=10)

        self.boton_deteccion = tk.Button(self.frame_botones, text="Detectar Objetos", command=self.toggle_deteccion,
                                         width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_deteccion.grid(row=2, column=0, padx=10, pady=10)

        self.boton_voz = tk.Button(self.frame_botones, text="Activar Voz", command=self.toggle_voz,
                                   width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_voz.grid(row=2, column=1, padx=10, pady=10)

        self.boton_alarma = tk.Button(self.frame_botones, text="Activar Alarma", command=self.toggle_alarma,
                                      width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_alarma.grid(row=3, column=0, padx=10, pady=10)

        self.boton_salir = tk.Button(self.frame_botones, text="Salir", command=self.salir, width=20,
                                     bg="#9E9E9E", fg="white", font=("Arial", 12))
        self.boton_salir.grid(row=3, column=1, padx=10, pady=10)

        self.canvas = tk.Canvas(self.frame_video, width=640, height=480)
        self.canvas.pack()

        self.label_historial_titulo = tk.Label(self.frame_historial, text="Historial", bg="#2e2e2e",
                                               fg="white", font=("Arial", 14, "bold"))
        self.label_historial_titulo.pack(pady=10)

        self.label_historial = tk.Label(self.frame_historial, text="", bg="#2e2e2e", fg="white", font=("Arial", 12),
                                        anchor="nw", justify="left")
        self.label_historial.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

    def cargar_logo(self):
        try:
            logo_path = "Logo1.png"
            if os.path.exists(logo_path):
                self.logo_imagen = Image.open(logo_path).resize((300, 100), Image.Resampling.LANCZOS)
                self.logo_imagen_tk = ImageTk.PhotoImage(self.logo_imagen)
                self.label_logo = tk.Label(self.frame_video, image=self.logo_imagen_tk, bg="#2e2e2e")
                self.label_logo.pack(pady=20)
            else:
                raise FileNotFoundError(f"El archivo {logo_path} no se encuentra.")
        except Exception as e:
            print(f"Error al cargar la imagen: {e}")

    def abrir_camara(self):
        if not self.camara_activa:
            self.cap = cv2.VideoCapture(0)
            if not self.cap.isOpened():
                messagebox.showerror("Error", "No se pudo abrir la cámara.")
                return

            self.camara_activa = True
            self.boton_abrir.config(state=tk.DISABLED)
            self.boton_cerrar.config(state=tk.NORMAL)
            self.boton_foto.config(state=tk.NORMAL)
            self.boton_video.config(state=tk.NORMAL)
            self.boton_deteccion.config(state=tk.NORMAL)
            self.boton_voz.config(state=tk.NORMAL)
            self.boton_alarma.config(state=tk.NORMAL)

            self.historial_detecciones[datetime.datetime.now().strftime("%Y%m%d_%H%M%S")] = "Cámara abierta"
            self.actualizar_historial()

            self.thread = Thread(target=self.actualizar_video)
            self.thread.daemon = True
            self.thread.start()

            self.thread_anunciar = Thread(target=self.anunciar_objetos_detectados)
            self.thread_anunciar.daemon = True
            self.thread_anunciar.start()

    def actualizar_video(self):
        frame_count = 0
        while self.camara_activa:
            ret, frame = self.cap.read()
            if not ret:
                break

            if self.detectando_objetos and frame_count % 3 == 0:
                blob = cv2.dnn.blobFromImage(frame, 0.00392, (320, 320), (0, 0, 0), True, crop=False)
                self.net.setInput(blob)
                outs = self.net.forward(self.output_layers)

                class_ids = []
                confidences = []
                boxes = []

                for out in outs:
                    for detection in out:
                        scores = detection[5:]
                        class_id = np.argmax(scores)
                        confidence = scores[class_id]
                        if confidence > 0.6:
                            center_x = int(detection[0] * frame.shape[1])
                            center_y = int(detection[1] * frame.shape[0])
                            w = int(detection[2] * frame.shape[1])
                            h = int(detection[3] * frame.shape[0])
                            x = int(center_x - w / 2)
                            y = int(center_y - h / 2)
                            boxes.append([x, y, w, h])
                            confidences.append(float(confidence))
                            class_ids.append(class_id)

                indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.6, 0.4)

                if len(indexes) > 0:
                    for i in indexes.flatten():
                        x, y, w, h = boxes[i]
                        label = str(self.classes[class_ids[i]])
                        color = (0, 255, 0)
                        if label in ["knife", "pistol"]:
                            color = (255, 0, 0)
                            current_time = time.time()
                            if self.alarma_activar and (current_time - self.last_alert_time > 3):
                                self.reproducir_alerta()
                                self.last_alert_time = current_time
                        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                        self.objetos_para_anunciar.add(label)

                if self.objetos_para_anunciar:
                    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                    objetos_detectados = ", ".join(self.objetos_para_anunciar)
                    self.historial_detecciones[timestamp] = f"Objetos detectados: {objetos_detectados}"
                    self.actualizar_historial()

            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_pil = Image.fromarray(frame_rgb)
            frame_tk = ImageTk.PhotoImage(frame_pil)
            self.canvas.create_image(0, 0, anchor=tk.NW, image=frame_tk)
            self.canvas.image = frame_tk

            frame_count += 1

    def anunciar_objetos_detectados(self):
        while True:
            if self.objetos_para_anunciar and self.voz_activar:
                objetos = ", ".join(self.objetos_para_anunciar)
                self.engine.say(f"Objetos detectados: {objetos}")
                self.engine.runAndWait()
                self.objetos_para_anunciar.clear()
            time.sleep(1)

    def cerrar_camara(self):
        if self.camara_activa:
            self.camara_activa = False
            self.cap.release()
            self.boton_abrir.config(state=tk.NORMAL)
            self.boton_cerrar.config(state=tk.DISABLED)
            self.boton_foto.config(state=tk.DISABLED)
            self.boton_video.config(state=tk.DISABLED)
            self.boton_deteccion.config(state=tk.DISABLED)
            self.boton_voz.config(state=tk.DISABLED)
            self.boton_alarma.config(state=tk.DISABLED)

            self.historial_detecciones[datetime.datetime.now().strftime("%Y%m%d_%H%M%S")] = "Cámara cerrada"
            self.actualizar_historial()

            if self.thread is not None:
                self.thread.join()

    def tomar_foto(self):
        if self.camara_activa:
            ret, frame = self.cap.read()
            if ret:
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                photo_filename = f"foto_{timestamp}.jpg"
                cv2.imwrite(photo_filename, frame)
                self.historial_detecciones[timestamp] = "Foto tomada"
                self.actualizar_historial()
                messagebox.showinfo("Foto tomada", f"La foto se guardó como {photo_filename}.")

    def grabar_video(self):
        if self.camara_activa:
            self.grabando_video = not self.grabando_video

            if self.grabando_video:
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                video_filename = f"video_{timestamp}.avi"
                self.video_writer = cv2.VideoWriter(video_filename, cv2.VideoWriter_fourcc(*'XVID'), 20.0, (640, 480))
                self.boton_video.config(text="Detener Video")
                self.historial_detecciones[timestamp] = "Grabación de video iniciada"
            else:
                self.video_writer.release()
                self.boton_video.config(text="Grabar Video")
                self.historial_detecciones[timestamp] = "Grabación de video detenida"
                self.actualizar_historial()
                messagebox.showinfo("Grabación detenida", "La grabación de video ha sido detenida.")

    def toggle_deteccion(self):
        self.detectando_objetos = not self.detectando_objetos
        estado = "activada" if self.detectando_objetos else "desactivada"
        self.historial_detecciones[datetime.datetime.now().strftime("%Y%m%d_%H%M%S")] = f"Detección de objetos {estado}"
        self.actualizar_historial()

    def toggle_voz(self):
        self.voz_activar = not self.voz_activar
        estado = "activada" if self.voz_activar else "desactivada"
        self.boton_voz.config(text=f"{estado.capitalize()} Voz")
        self.historial_detecciones[datetime.datetime.now().strftime("%Y%m%d_%H%M%S")] = f"Voz {estado}"
        self.actualizar_historial()

    def toggle_alarma(self):
        self.alarma_activar = not self.alarma_activar
        estado = "activada" if self.alarma_activar else "desactivada"
        self.boton_alarma.config(text=f"{estado.capitalize()} Alarma")
        self.historial_detecciones[datetime.datetime.now().strftime("%Y%m%d_%H%M%S")] = f"Alarma {estado}"
        self.actualizar_historial()

    def actualizar_historial(self):
        historial_texto = "\n".join([f"{hora}: {accion}" for hora, accion in self.historial_detecciones.items()])
        self.label_historial.config(text=historial_texto)

    def reproducir_alerta(self):
        try:
            pygame.mixer.music.load("alerta.mp3")
            pygame.mixer.music.play()
        except Exception as e:
            print(f"Error al reproducir el sonido: {e}")

    def salir(self):
        if self.camara_activa:
            self.cerrar_camara()
        self.root.quit()

# Crear ventana principal
root = tk.Tk()
app = CamaraApp(root)
root.mainloop()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
#Se arregla la funcionalidad del sistema con el boton de cerrar camara y se mejora el tema del rendimieto y el historial

In [1]:
import tkinter as tk
from tkinter import messagebox, Scrollbar, Frame, filedialog
from threading import Thread
import cv2
import datetime
import numpy as np
from PIL import Image, ImageTk
import pyttsx3
import os
import time
import pygame

class CamaraApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Detector de imágenes - Demo")
        self.root.config(bg="#2e2e2e")

        pygame.mixer.init()

        self.camara_activa = False
        self.cap = None
        self.thread = None
        self.grabando_video = False
        self.video_writer = None
        self.detectando_objetos = False
        self.engine = pyttsx3.init()
        self.historial_detecciones = []
        self.objetos_para_anunciar = set()
        self.last_alert_time = 0
        self.voz_activar = True
        self.alarma_activar = True

        self.net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
        self.layer_names = self.net.getLayerNames()
        self.output_layers = [self.layer_names[i - 1] for i in self.net.getUnconnectedOutLayers()]
        self.classes = []
        with open("coco.names", "r") as f:
            self.classes = [line.strip() for line in f.readlines()]

        self.frame_principal = tk.Frame(root, bg="#2e2e2e")
        self.frame_principal.pack(fill=tk.BOTH, expand=True)

        self.frame_video = tk.Frame(self.frame_principal, bg="#2e2e2e")
        self.frame_video.pack(side=tk.LEFT, padx=10, pady=10)

        self.frame_historial = Frame(self.frame_principal)
        self.frame_historial.pack(side=tk.RIGHT, fill=tk.Y, padx=10, pady=10)

        self.scrollbar = Scrollbar(self.frame_historial)
        self.scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        self.label_historial_titulo = tk.Label(self.frame_historial, text="Historial", bg="#2e2e2e",
                                               fg="white", font=("Arial", 14, "bold"))
        self.label_historial_titulo.pack(pady=10)

        self.text_historial = tk.Text(self.frame_historial, bg="#2e2e2e", fg="white", font=("Arial", 12),
                                       wrap="word", height=20, width=40)
        self.text_historial.pack(fill=tk.BOTH, expand=True, padx=10)
        self.text_historial.config(yscrollcommand=self.scrollbar.set)
        self.scrollbar.config(command=self.text_historial.yview)

        self.logo_imagen_tk = None
        self.cargar_logo()

        self.frame_botones = tk.Frame(self.frame_video, bg="#3c3c3c")
        self.frame_botones.pack(pady=10, padx=10, fill=tk.X)

        self.boton_abrir = tk.Button(self.frame_botones, text="Abrir Cámara", command=self.abrir_camara, width=20,
                                     bg="#4CAF50", fg="white", font=("Arial", 12))
        self.boton_abrir.grid(row=0, column=0, padx=10, pady=10)

        self.boton_cerrar = tk.Button(self.frame_botones, text="Cerrar Cámara", command=self.cerrar_camara, width=20,
                                      state=tk.DISABLED, bg="#F44336", fg="white", font=("Arial", 12))
        self.boton_cerrar.grid(row=0, column=1, padx=10, pady=10)

        self.boton_foto = tk.Button(self.frame_botones, text="Tomar Foto", command=self.tomar_foto, width=20,
                                    state=tk.DISABLED, bg="#2196F3", fg="white", font=("Arial", 12))
        self.boton_foto.grid(row=1, column=0, padx=10, pady=10)

        self.boton_video = tk.Button(self.frame_botones, text="Grabar Video", command=self.grabar_video, width=20,
                                     state=tk.DISABLED, bg="#FF9800", fg="white", font=("Arial", 12))
        self.boton_video.grid(row=1, column=1, padx=10, pady=10)

        self.boton_deteccion = tk.Button(self.frame_botones, text="Detectar Objetos", command=self.toggle_deteccion,
                                         width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_deteccion.grid(row=2, column=0, padx=10, pady=10)

        self.boton_voz = tk.Button(self.frame_botones, text="Activar Voz", command=self.toggle_voz,
                                   width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_voz.grid(row=2, column=1, padx=10, pady=10)

        self.boton_alarma = tk.Button(self.frame_botones, text="Activar Alarma", command=self.toggle_alarma,
                                      width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_alarma.grid(row=3, column=0, padx=10, pady=10)

        self.boton_descargar = tk.Button(self.frame_botones, text="Descargar Historial", command=self.descargar_historial,
                                          width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_descargar.grid(row=3, column=1, padx=10, pady=10)

        self.boton_salir = tk.Button(self.frame_botones, text="Salir", command=self.salir, width=20,
                                     bg="#9E9E9E", fg="white", font=("Arial", 12))
        self.boton_salir.grid(row=4, column=0, columnspan=2, padx=10, pady=10)

        self.canvas = tk.Canvas(self.frame_video, width=640, height=480)
        self.canvas.pack()

    def cargar_logo(self):
        try:
            logo_path = "Logo1.png"
            if os.path.exists(logo_path):
                self.logo_imagen = Image.open(logo_path).resize((300, 100), Image.Resampling.LANCZOS)
                self.logo_imagen_tk = ImageTk.PhotoImage(self.logo_imagen)
                self.label_logo = tk.Label(self.frame_video, image=self.logo_imagen_tk, bg="#2e2e2e")
                self.label_logo.pack(pady=20)
            else:
                raise FileNotFoundError(f"El archivo {logo_path} no se encuentra.")
        except Exception as e:
            print(f"Error al cargar la imagen: {e}")

    def abrir_camara(self):
        if not self.camara_activa:
            self.cap = cv2.VideoCapture(0)
            if not self.cap.isOpened():
                messagebox.showerror("Error", "No se pudo abrir la cámara.")
                return

            self.camara_activa = True
            self.boton_abrir.config(state=tk.DISABLED)
            self.boton_cerrar.config(state=tk.NORMAL)
            self.boton_foto.config(state=tk.NORMAL)
            self.boton_video.config(state=tk.NORMAL)
            self.boton_deteccion.config(state=tk.NORMAL)
            self.boton_voz.config(state=tk.NORMAL)
            self.boton_alarma.config(state=tk.NORMAL)
            self.boton_descargar.config(state=tk.NORMAL)

            self.agregar_historial("Cámara abierta")

            self.thread = Thread(target=self.actualizar_video)
            self.thread.daemon = True
            self.thread.start()

            self.thread_anunciar = Thread(target=self.anunciar_objetos_detectados)
            self.thread_anunciar.daemon = True
            self.thread_anunciar.start()

    def actualizar_video(self):
        while self.camara_activa:
            ret, frame = self.cap.read()
            if not ret:
                break

            objetos_detectados = set()  # Para almacenar objetos detectados

            if self.detectando_objetos:
                # Prepara la imagen para la detección
                blob = cv2.dnn.blobFromImage(frame, 0.00392, (320, 320), (0, 0, 0), True, crop=False)
                self.net.setInput(blob)
                outs = self.net.forward(self.output_layers)

                class_ids = []
                confidences = []
                boxes = []

                for out in outs:
                    for detection in out:
                        scores = detection[5:]
                        class_id = np.argmax(scores)
                        confidence = scores[class_id]
                        if confidence > 0.6:
                            center_x = int(detection[0] * frame.shape[1])
                            center_y = int(detection[1] * frame.shape[0])
                            w = int(detection[2] * frame.shape[1])
                            h = int(detection[3] * frame.shape[0])
                            x = int(center_x - w / 2)
                            y = int(center_y - h / 2)
                            boxes.append([x, y, w, h])
                            confidences.append(float(confidence))
                            class_ids.append(class_id)

                indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.6, 0.4)

                if len(indexes) > 0:
                    for i in indexes.flatten():
                        x, y, w, h = boxes[i]
                        label = str(self.classes[class_ids[i]])
                        objetos_detectados.add(label)  # Agregar a la lista de objetos detectados
                        color = (0, 255, 0)  # Verde por defecto
                        if label in ["knife", "pistol"]:  # Cambiar a rojo si es un objeto peligroso
                            color = (255, 0, 0)  # Rojo para objetos peligrosos
                            current_time = time.time()
                            if self.alarma_activar and (current_time - self.last_alert_time > 3):
                                self.reproducir_alerta()
                                self.last_alert_time = current_time
                        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Actualizar el historial de objetos detectados
            if objetos_detectados:
                self.agregar_historial(f"Objetos detectados: {', '.join(objetos_detectados)}")
                self.objetos_para_anunciar.update(objetos_detectados)

            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_pil = Image.fromarray(frame_rgb)
            frame_tk = ImageTk.PhotoImage(frame_pil)
            self.canvas.create_image(0, 0, anchor=tk.NW, image=frame_tk)
            self.canvas.image = frame_tk

    def anunciar_objetos_detectados(self):
        while True:
            if self.objetos_para_anunciar and self.voz_activar:
                objetos = ", ".join(self.objetos_para_anunciar)
                self.engine.say(f"Objetos detectados: {objetos}")
                self.engine.runAndWait()
                self.objetos_para_anunciar.clear()
            time.sleep(1)

    def cerrar_camara(self):
        if self.camara_activa:
            self.camara_activa = False
            
            # Detener el hilo de video
            if self.thread is not None:
                self.thread.join(timeout=1)  # Espera un segundo para que el hilo termine
            
            # Liberar la cámara
            if self.cap is not None:
                self.cap.release()
            
            # Actualizar el estado de los botones
            self.boton_abrir.config(state=tk.NORMAL)
            self.boton_cerrar.config(state=tk.DISABLED)
            self.boton_foto.config(state=tk.DISABLED)
            self.boton_video.config(state=tk.DISABLED)
            self.boton_deteccion.config(state=tk.DISABLED)
            self.boton_voz.config(state=tk.DISABLED)
            self.boton_alarma.config(state=tk.DISABLED)
            self.boton_descargar.config(state=tk.DISABLED)

            self.agregar_historial("Cámara cerrada")

    def tomar_foto(self):
        if self.camara_activa:
            ret, frame = self.cap.read()
            if ret:
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                photo_filename = f"foto_{timestamp}.jpg"
                cv2.imwrite(photo_filename, frame)
                self.agregar_historial("Foto tomada")
                messagebox.showinfo("Foto tomada", f"La foto se guardó como {photo_filename}.")

    def grabar_video(self):
        if self.camara_activa:
            self.grabando_video = not self.grabando_video

            if self.grabando_video:
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                video_filename = f"video_{timestamp}.avi"
                self.video_writer = cv2.VideoWriter(video_filename, cv2.VideoWriter_fourcc(*'XVID'), 20.0, (640, 480))
                self.boton_video.config(text="Detener Video")
                self.agregar_historial("Grabación de video iniciada")
            else:
                self.video_writer.release()
                self.boton_video.config(text="Grabar Video")
                self.agregar_historial("Grabación de video detenida")
                messagebox.showinfo("Grabación detenida", "La grabación de video ha sido detenida.")

    def toggle_deteccion(self):
        self.detectando_objetos = not self.detectando_objetos
        estado = "activada" if self.detectando_objetos else "desactivada"
        self.agregar_historial(f"Detección de objetos {estado}")

    def toggle_voz(self):
        self.voz_activar = not self.voz_activar
        estado = "activada" if self.voz_activar else "desactivada"
        self.boton_voz.config(text=f"{estado.capitalize()} Voz")
        self.agregar_historial(f"Voz {estado}")

    def toggle_alarma(self):
        self.alarma_activar = not self.alarma_activar
        estado = "activada" if self.alarma_activar else "desactivada"
        self.boton_alarma.config(text=f"{estado.capitalize()} Alarma")
        self.agregar_historial(f"Alarma {estado}")

    def agregar_historial(self, mensaje):
        timestamp = datetime.datetime.now().strftime("%Y%m%d %H:%M:%S")
        self.historial_detecciones.append(f"{timestamp}: {mensaje}")
        if len(self.historial_detecciones) > 50:  # Limitar el tamaño del historial
            self.historial_detecciones.pop(0)
        
        # Actualizar el historial en el Text
        self.text_historial.config(state=tk.NORMAL)  # Habilitar el Text para la edición
        self.text_historial.delete(1.0, tk.END)  # Limpiar el Text
        self.text_historial.insert(tk.END, "\n".join(self.historial_detecciones))  # Agregar nuevo texto
        self.text_historial.config(state=tk.DISABLED)  # Deshabilitar el Text para evitar ediciones
        self.text_historial.yview_moveto(1)  # Desplazar hacia abajo

    def reproducir_alerta(self):
        try:
            pygame.mixer.music.load("alerta.mp3")
            pygame.mixer.music.play()
        except Exception as e:
            print(f"Error al reproducir el sonido: {e}")

    def descargar_historial(self):
        archivo = filedialog.asksaveasfile(defaultextension=".txt", filetypes=[("Text files", "*.txt")])
        if archivo:
            for entrada in self.historial_detecciones:
                archivo.write(entrada + "\n")
            archivo.close()
            messagebox.showinfo("Éxito", "Historial descargado correctamente.")

    def salir(self):
        self.cerrar_camara()  # Asegurarse de que se cierre la cámara
        self.root.quit()  # Cerrar la aplicación

# Crear ventana principal
root = tk.Tk()
app = CamaraApp(root)
root.mainloop()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


Exception in thread Thread-6 (anunciar_objetos_detectados):
Traceback (most recent call last):
  File "C:\Users\brand\anaconda3\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\Users\brand\anaconda3\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\brand\AppData\Local\Temp\ipykernel_5372\1174535221.py", line 215, in anunciar_objetos_detectados
  File "C:\Users\brand\anaconda3\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started


In [ ]:
# El codigo finiquitado al 100% con tutorial incluido.

In [1]:
import tkinter as tk
from tkinter import messagebox, Scrollbar, Frame, filedialog
from threading import Thread
import cv2
import datetime
import numpy as np
from PIL import Image, ImageTk
import pyttsx3
import os
import time
import pygame

class CamaraApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Detector de imágenes - Demo")
        self.root.config(bg="#2e2e2e")

        pygame.mixer.init()

        self.camara_activa = False
        self.cap = None
        self.thread = None
        self.grabando_video = False
        self.video_writer = None
        self.detectando_objetos = False
        self.engine = pyttsx3.init()
        self.historial_detecciones = []
        self.objetos_para_anunciar = set()
        self.last_alert_time = 0
        self.voz_activar = True
        self.alarma_activar = True

        self.net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
        self.layer_names = self.net.getLayerNames()
        self.output_layers = [self.layer_names[i - 1] for i in self.net.getUnconnectedOutLayers()]
        self.classes = []
        with open("coco.names", "r") as f:
            self.classes = [line.strip() for line in f.readlines()]

        self.frame_principal = tk.Frame(root, bg="#2e2e2e")
        self.frame_principal.pack(fill=tk.BOTH, expand=True)

        self.frame_video = tk.Frame(self.frame_principal, bg="#2e2e2e")
        self.frame_video.pack(side=tk.LEFT, padx=10, pady=10)

        self.frame_historial = Frame(self.frame_principal)
        self.frame_historial.pack(side=tk.RIGHT, fill=tk.Y, padx=10, pady=10)

        self.scrollbar = Scrollbar(self.frame_historial)
        self.scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        self.label_historial_titulo = tk.Label(self.frame_historial, text="Historial", bg="#2e2e2e",
                                               fg="white", font=("Arial", 14, "bold"))
        self.label_historial_titulo.pack(pady=10)

        self.text_historial = tk.Text(self.frame_historial, bg="#2e2e2e", fg="white", font=("Arial", 12),
                                       wrap="word", height=20, width=40)
        self.text_historial.pack(fill=tk.BOTH, expand=True, padx=10)
        self.text_historial.config(yscrollcommand=self.scrollbar.set)
        self.scrollbar.config(command=self.text_historial.yview)

        self.logo_imagen_tk = None
        self.cargar_logo()

        self.frame_botones = tk.Frame(self.frame_video, bg="#3c3c3c")
        self.frame_botones.pack(pady=10, padx=10, fill=tk.X)

        self.boton_abrir = tk.Button(self.frame_botones, text="Abrir Cámara", command=self.abrir_camara, width=20,
                                     bg="#4CAF50", fg="white", font=("Arial", 12))
        self.boton_abrir.grid(row=0, column=0, padx=10, pady=10)

        self.boton_cerrar = tk.Button(self.frame_botones, text="Cerrar Cámara", command=self.cerrar_camara, width=20,
                                      state=tk.DISABLED, bg="#F44336", fg="white", font=("Arial", 12))
        self.boton_cerrar.grid(row=0, column=1, padx=10, pady=10)

        self.boton_foto = tk.Button(self.frame_botones, text="Tomar Foto", command=self.tomar_foto, width=20,
                                    state=tk.DISABLED, bg="#2196F3", fg="white", font=("Arial", 12))
        self.boton_foto.grid(row=1, column=0, padx=10, pady=10)

        self.boton_video = tk.Button(self.frame_botones, text="Grabar Video", command=self.grabar_video, width=20,
                                     state=tk.DISABLED, bg="#FF9800", fg="white", font=("Arial", 12))
        self.boton_video.grid(row=1, column=1, padx=10, pady=10)

        self.boton_deteccion = tk.Button(self.frame_botones, text="Detectar Objetos", command=self.toggle_deteccion,
                                         width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_deteccion.grid(row=2, column=0, padx=10, pady=10)

        self.boton_voz = tk.Button(self.frame_botones, text="Activar Voz", command=self.toggle_voz,
                                   width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_voz.grid(row=2, column=1, padx=10, pady=10)

        self.boton_alarma = tk.Button(self.frame_botones, text="Activar Alarma", command=self.toggle_alarma,
                                      width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_alarma.grid(row=3, column=0, padx=10, pady=10)

        self.boton_descargar = tk.Button(self.frame_botones, text="Descargar Historial", command=self.descargar_historial,
                                          width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_descargar.grid(row=3, column=1, padx=10, pady=10)

        self.boton_salir = tk.Button(self.frame_botones, text="Salir", command=self.salir, width=20,
                                     bg="#9E9E9E", fg="white", font=("Arial", 12))
        self.boton_salir.grid(row=4, column=0, columnspan=2, padx=10, pady=10)

        self.canvas = tk.Canvas(self.frame_video, width=640, height=480)
        self.canvas.pack()

        self.mostrar_tutorial()

    def cargar_logo(self):
        try:
            logo_path = "Logo1.png"
            if os.path.exists(logo_path):
                self.logo_imagen = Image.open(logo_path).resize((300, 100), Image.Resampling.LANCZOS)
                self.logo_imagen_tk = ImageTk.PhotoImage(self.logo_imagen)
                self.label_logo = tk.Label(self.frame_video, image=self.logo_imagen_tk, bg="#2e2e2e")
                self.label_logo.pack(pady=20)
            else:
                raise FileNotFoundError(f"El archivo {logo_path} no se encuentra.")
        except Exception as e:
            print(f"Error al cargar la imagen: {e}")

    def mostrar_tutorial(self):
        self.tutorial = tk.Toplevel(self.root)
        self.tutorial.title("Tutorial")
        self.tutorial.geometry("400x300")
        self.tutorial.config(bg="#2e2e2e")

        tutorial_text = """
        Bienvenido a la aplicación de detección de imágenes.
        
        1. Abrir Cámara: Inicia la cámara para la detección.
        2. Detectar Objetos: Activa la detección de objetos en la cámara.
        3. Tomar Foto: Captura una imagen de la cámara.
        4. Grabar Video: Inicia o detiene la grabación de video.
        5. Descargar Historial: Guarda el historial de detecciones en un archivo.
        6. Activar Voz: Anuncia los objetos detectados.
        7. Activar Alarma: Reproduce una alarma si se detectan objetos peligrosos.
        8. Salir: Cierra la aplicación.

        Presiona 'Saltar Tutorial' para omitir este tutorial.
        """
        
        label_tutorial = tk.Label(self.tutorial, text=tutorial_text, bg="#2e2e2e", fg="white", font=("Arial", 12))
        label_tutorial.pack(pady=10)

        boton_saltar = tk.Button(self.tutorial, text="Saltar Tutorial", command=self.saltar_tutorial,
                                  bg="#FFC107", fg="black", font=("Arial", 12))
        boton_saltar.pack(pady=20)

    def saltar_tutorial(self):
        self.tutorial.destroy()

    def abrir_camara(self):
        if not self.camara_activa:
            self.cap = cv2.VideoCapture(0)
            if not self.cap.isOpened():
                messagebox.showerror("Error", "No se pudo abrir la cámara.")
                return

            self.camara_activa = True
            self.boton_abrir.config(state=tk.DISABLED)
            self.boton_cerrar.config(state=tk.NORMAL)
            self.boton_foto.config(state=tk.NORMAL)
            self.boton_video.config(state=tk.NORMAL)
            self.boton_deteccion.config(state=tk.NORMAL)
            self.boton_voz.config(state=tk.NORMAL)
            self.boton_alarma.config(state=tk.NORMAL)
            self.boton_descargar.config(state=tk.NORMAL)

            self.agregar_historial("Cámara abierta")

            self.thread = Thread(target=self.actualizar_video)
            self.thread.daemon = True
            self.thread.start()

            self.thread_anunciar = Thread(target=self.anunciar_objetos_detectados)
            self.thread_anunciar.daemon = True
            self.thread_anunciar.start()

    def actualizar_video(self):
        while self.camara_activa:
            ret, frame = self.cap.read()
            if not ret:
                break

            objetos_detectados = set()  # Para almacenar objetos detectados

            if self.detectando_objetos:
                # Prepara la imagen para la detección
                blob = cv2.dnn.blobFromImage(frame, 0.00392, (320, 320), (0, 0, 0), True, crop=False)
                self.net.setInput(blob)
                outs = self.net.forward(self.output_layers)

                class_ids = []
                confidences = []
                boxes = []

                for out in outs:
                    for detection in out:
                        scores = detection[5:]
                        class_id = np.argmax(scores)
                        confidence = scores[class_id]
                        if confidence > 0.6:
                            center_x = int(detection[0] * frame.shape[1])
                            center_y = int(detection[1] * frame.shape[0])
                            w = int(detection[2] * frame.shape[1])
                            h = int(detection[3] * frame.shape[0])
                            x = int(center_x - w / 2)
                            y = int(center_y - h / 2)
                            boxes.append([x, y, w, h])
                            confidences.append(float(confidence))
                            class_ids.append(class_id)

                indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.6, 0.4)

                if len(indexes) > 0:
                    for i in indexes.flatten():
                        x, y, w, h = boxes[i]
                        label = str(self.classes[class_ids[i]])
                        objetos_detectados.add(label)  # Agregar a la lista de objetos detectados
                        color = (0, 255, 0)  # Verde por defecto
                        if label in ["knife", "pistol"]:  # Cambiar a rojo si es un objeto peligroso
                            color = (255, 0, 0)  # Rojo para objetos peligrosos
                            current_time = time.time()
                            if self.alarma_activar and (current_time - self.last_alert_time > 3):
                                self.reproducir_alerta()
                                self.last_alert_time = current_time
                        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Actualizar el historial de objetos detectados
            if objetos_detectados:
                self.agregar_historial(f"Objetos detectados: {', '.join(objetos_detectados)}")
                self.objetos_para_anunciar.update(objetos_detectados)

            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_pil = Image.fromarray(frame_rgb)
            frame_tk = ImageTk.PhotoImage(frame_pil)
            self.canvas.create_image(0, 0, anchor=tk.NW, image=frame_tk)
            self.canvas.image = frame_tk

    def anunciar_objetos_detectados(self):
        while True:
            if self.objetos_para_anunciar and self.voz_activar:
                objetos = ", ".join(self.objetos_para_anunciar)
                self.engine.say(f"Objetos detectados: {objetos}")
                self.engine.runAndWait()
                self.objetos_para_anunciar.clear()
            time.sleep(1)

    def cerrar_camara(self):
        if self.camara_activa:
            self.camara_activa = False
            
            # Detener el hilo de video
            if self.thread is not None:
                self.thread.join(timeout=1)  # Espera un segundo para que el hilo termine
            
            # Liberar la cámara
            if self.cap is not None:
                self.cap.release()
            
            # Actualizar el estado de los botones
            self.boton_abrir.config(state=tk.NORMAL)
            self.boton_cerrar.config(state=tk.DISABLED)
            self.boton_foto.config(state=tk.DISABLED)
            self.boton_video.config(state=tk.DISABLED)
            self.boton_deteccion.config(state=tk.DISABLED)
            self.boton_voz.config(state=tk.DISABLED)
            self.boton_alarma.config(state=tk.DISABLED)
            self.boton_descargar.config(state=tk.DISABLED)

            self.agregar_historial("Cámara cerrada")

    def tomar_foto(self):
        if self.camara_activa:
            ret, frame = self.cap.read()
            if ret:
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                photo_filename = f"foto_{timestamp}.jpg"
                cv2.imwrite(photo_filename, frame)
                self.agregar_historial("Foto tomada")
                messagebox.showinfo("Foto tomada", f"La foto se guardó como {photo_filename}.")

    def grabar_video(self):
        if self.camara_activa:
            self.grabando_video = not self.grabando_video

            if self.grabando_video:
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                video_filename = f"video_{timestamp}.avi"
                self.video_writer = cv2.VideoWriter(video_filename, cv2.VideoWriter_fourcc(*'XVID'), 20.0, (640, 480))
                self.boton_video.config(text="Detener Video")
                self.agregar_historial("Grabación de video iniciada")
            else:
                self.video_writer.release()
                self.boton_video.config(text="Grabar Video")
                self.agregar_historial("Grabación de video detenida")
                messagebox.showinfo("Grabación detenida", "La grabación de video ha sido detenida.")

    def toggle_deteccion(self):
        self.detectando_objetos = not self.detectando_objetos
        estado = "activada" if self.detectando_objetos else "desactivada"
        self.agregar_historial(f"Detección de objetos {estado}")

    def toggle_voz(self):
        self.voz_activar = not self.voz_activar
        estado = "activada" if self.voz_activar else "desactivada"
        self.boton_voz.config(text=f"{estado.capitalize()} Voz")
        self.agregar_historial(f"Voz {estado}")

    def toggle_alarma(self):
        self.alarma_activar = not self.alarma_activar
        estado = "activada" if self.alarma_activar else "desactivada"
        self.boton_alarma.config(text=f"{estado.capitalize()} Alarma")
        self.agregar_historial(f"Alarma {estado}")

    def agregar_historial(self, mensaje):
        timestamp = datetime.datetime.now().strftime("%Y%m%d %H:%M:%S")
        self.historial_detecciones.append(f"{timestamp}: {mensaje}")
        if len(self.historial_detecciones) > 50:  # Limitar el tamaño del historial
            self.historial_detecciones.pop(0)
        
        # Actualizar el historial en el Text
        self.text_historial.config(state=tk.NORMAL)  # Habilitar el Text para la edición
        self.text_historial.delete(1.0, tk.END)  # Limpiar el Text
        self.text_historial.insert(tk.END, "\n".join(self.historial_detecciones))  # Agregar nuevo texto
        self.text_historial.config(state=tk.DISABLED)  # Deshabilitar el Text para evitar ediciones
        self.text_historial.yview_moveto(1)  # Desplazar hacia abajo

    def reproducir_alerta(self):
        try:
            pygame.mixer.music.load("alerta.mp3")
            pygame.mixer.music.play()
        except Exception as e:
            print(f"Error al reproducir el sonido: {e}")

    def descargar_historial(self):
        archivo = filedialog.asksaveasfile(defaultextension=".txt", filetypes=[("Text files", "*.txt")])
        if archivo:
            for entrada in self.historial_detecciones:
                archivo.write(entrada + "\n")
            archivo.close()
            messagebox.showinfo("Éxito", "Historial descargado correctamente.")

    def salir(self):
        self.cerrar_camara()  # Asegurarse de que se cierre la cámara
        self.root.quit()  # Cerrar la aplicación

# Crear ventana principal
root = tk.Tk()
app = CamaraApp(root)
root.mainloop()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
#mejoras

In [11]:
import tkinter as tk
from tkinter import messagebox, Scrollbar, Frame, filedialog
from threading import Thread
import cv2
import datetime
import numpy as np
from PIL import Image, ImageTk
import pyttsx3
import os
import time
import pygame

class CamaraApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Detector de imágenes - Demo")
        self.root.config(bg="#2e2e2e")

        pygame.mixer.init()

        self.camara_activa = False
        self.cap = None
        self.thread = None
        self.grabando_video = False
        self.video_writer = None
        self.detectando_objetos = False
        self.engine = pyttsx3.init()
        self.historial_detecciones = []
        self.objetos_para_anunciar = set()
        self.last_alert_time = 0
        self.voz_activar = True
        self.alarma_activar = True

        self.net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
        self.layer_names = self.net.getLayerNames()
        self.output_layers = [self.layer_names[i - 1] for i in self.net.getUnconnectedOutLayers()]
        self.classes = []
        with open("coco.names", "r") as f:
            self.classes = [line.strip() for line in f.readlines()]

        self.frame_principal = tk.Frame(root, bg="#2e2e2e")
        self.frame_principal.pack(fill=tk.BOTH, expand=True)

        self.frame_video = tk.Frame(self.frame_principal, bg="#2e2e2e")
        self.frame_video.pack(side=tk.LEFT, padx=10, pady=10)

        self.frame_historial = Frame(self.frame_principal)
        self.frame_historial.pack(side=tk.RIGHT, fill=tk.Y, padx=10, pady=10)

        self.scrollbar = Scrollbar(self.frame_historial)
        self.scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        self.label_historial_titulo = tk.Label(self.frame_historial, text="Historial", bg="#2e2e2e",
                                               fg="white", font=("Arial", 14, "bold"))
        self.label_historial_titulo.pack(pady=10)

        self.text_historial = tk.Text(self.frame_historial, bg="#2e2e2e", fg="white", font=("Arial", 12),
                                       wrap="word", height=20, width=40)
        self.text_historial.pack(fill=tk.BOTH, expand=True, padx=10)
        self.text_historial.config(yscrollcommand=self.scrollbar.set)
        self.scrollbar.config(command=self.text_historial.yview)

        self.logo_imagen_tk = None
        self.cargar_logo()

        self.frame_botones = tk.Frame(self.frame_video, bg="#3c3c3c")
        self.frame_botones.pack(pady=10, padx=10, fill=tk.X)

        self.boton_abrir = tk.Button(self.frame_botones, text="Abrir Cámara", command=self.abrir_camara, width=20,
                                     bg="#4CAF50", fg="white", font=("Arial", 12))
        self.boton_abrir.grid(row=0, column=0, padx=10, pady=10)

        self.boton_cerrar = tk.Button(self.frame_botones, text="Cerrar Cámara", command=self.cerrar_camara, width=20,
                                      state=tk.DISABLED, bg="#F44336", fg="white", font=("Arial", 12))
        self.boton_cerrar.grid(row=0, column=1, padx=10, pady=10)

        self.boton_foto = tk.Button(self.frame_botones, text="Tomar Foto", command=self.tomar_foto, width=20,
                                    state=tk.DISABLED, bg="#2196F3", fg="white", font=("Arial", 12))
        self.boton_foto.grid(row=1, column=0, padx=10, pady=10)

        self.boton_video = tk.Button(self.frame_botones, text="Grabar Video", command=self.grabar_video, width=20,
                                     state=tk.DISABLED, bg="#FF9800", fg="white", font=("Arial", 12))
        self.boton_video.grid(row=1, column=1, padx=10, pady=10)

        self.boton_deteccion = tk.Button(self.frame_botones, text="Detectar Objetos", command=self.toggle_deteccion,
                                         width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_deteccion.grid(row=2, column=0, padx=10, pady=10)

        self.boton_voz = tk.Button(self.frame_botones, text="Activar Voz", command=self.toggle_voz,
                                   width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_voz.grid(row=2, column=1, padx=10, pady=10)

        self.boton_alarma = tk.Button(self.frame_botones, text="Activar Alarma", command=self.toggle_alarma,
                                      width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_alarma.grid(row=3, column=0, padx=10, pady=10)

        self.boton_descargar = tk.Button(self.frame_botones, text="Descargar Historial", command=self.descargar_historial,
                                          width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_descargar.grid(row=3, column=1, padx=10, pady=10)

        self.boton_salir = tk.Button(self.frame_botones, text="Salir", command=self.salir, width=20,
                                     bg="#9E9E9E", fg="white", font=("Arial", 12))
        self.boton_salir.grid(row=4, column=0, columnspan=2, padx=10, pady=10)

        self.canvas = tk.Canvas(self.frame_video, width=640, height=480)
        self.canvas.pack()

        self.mostrar_tutorial()

    def cargar_logo(self):
        try:
            logo_path = "Logo1.png"
            if os.path.exists(logo_path):
                self.logo_imagen = Image.open(logo_path).resize((300, 100), Image.Resampling.LANCZOS)
                self.logo_imagen_tk = ImageTk.PhotoImage(self.logo_imagen)
                self.label_logo = tk.Label(self.frame_video, image=self.logo_imagen_tk, bg="#2e2e2e")
                self.label_logo.pack(pady=20)
            else:
                raise FileNotFoundError(f"El archivo {logo_path} no se encuentra.")
        except Exception as e:
            print(f"Error al cargar la imagen: {e}")

    def mostrar_tutorial(self):
        self.tutorial = tk.Toplevel(self.root)
        self.tutorial.title("Tutorial")
        self.tutorial.geometry("400x300")
        self.tutorial.config(bg="#2e2e2e")

        tutorial_text = """
        Bienvenido a la aplicación de detección de imágenes.

        1. Abrir Cámara: Inicia la cámara para la detección.
        2. Detectar Objetos: Activa la detección de objetos en la cámara.
        3. Tomar Foto: Captura una imagen de la cámara.
        4. Grabar Video: Inicia o detiene la grabación de video.
        5. Descargar Historial: Guarda el historial de detecciones en un archivo.
        6. Activar Voz: Anuncia los objetos detectados.
        7. Activar Alarma: Reproduce una alarma si se detectan objetos peligrosos.
        8. Salir: Cierra la aplicación.

        Presiona 'Saltar Tutorial' para omitir este tutorial.
        """
        
        label_tutorial = tk.Label(self.tutorial, text=tutorial_text, bg="#2e2e2e", fg="white", font=("Arial", 12))
        label_tutorial.pack(pady=10)

        boton_saltar = tk.Button(self.tutorial, text="Saltar Tutorial", command=self.saltar_tutorial,
                                  bg="#FFC107", fg="black", font=("Arial", 12))
        boton_saltar.pack(pady=20)

    def saltar_tutorial(self):
        self.tutorial.destroy()

    def abrir_camara(self):
        if not self.camara_activa:
            self.cap = cv2.VideoCapture(0)
            if not self.cap.isOpened():
                messagebox.showerror("Error", "No se pudo abrir la cámara.")
                return

            self.camara_activa = True
            self.boton_abrir.config(state=tk.DISABLED)
            self.boton_cerrar.config(state=tk.NORMAL)
            self.boton_foto.config(state=tk.NORMAL)
            self.boton_video.config(state=tk.NORMAL)
            self.boton_deteccion.config(state=tk.NORMAL)
            self.boton_voz.config(state=tk.NORMAL)
            self.boton_alarma.config(state=tk.NORMAL)
            self.boton_descargar.config(state=tk.NORMAL)

            self.agregar_historial("Cámara abierta")

            self.thread = Thread(target=self.actualizar_video)
            self.thread.daemon = True
            self.thread.start()

            self.thread_anunciar = Thread(target=self.anunciar_objetos_detectados)
            self.thread_anunciar.daemon = True
            self.thread_anunciar.start()

    def actualizar_video(self):
        while self.camara_activa:
            ret, frame = self.cap.read()
            if not ret:
                break

            objetos_detectados = set()  # Para almacenar objetos detectados

            if self.detectando_objetos:
                # Prepara la imagen para la detección
                blob = cv2.dnn.blobFromImage(frame, 0.00392, (320, 320), (0, 0, 0), True, crop=False)
                self.net.setInput(blob)
                outs = self.net.forward(self.output_layers)

                class_ids = []
                confidences = []
                boxes = []

                for out in outs:
                    for detection in out:
                        scores = detection[5:]
                        class_id = np.argmax(scores)
                        confidence = scores[class_id]
                        if confidence > 0.6:
                            center_x = int(detection[0] * frame.shape[1])
                            center_y = int(detection[1] * frame.shape[0])
                            w = int(detection[2] * frame.shape[1])
                            h = int(detection[3] * frame.shape[0])
                            x = int(center_x - w / 2)
                            y = int(center_y - h / 2)
                            boxes.append([x, y, w, h])
                            confidences.append(float(confidence))
                            class_ids.append(class_id)

                indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.6, 0.4)

                if len(indexes) > 0:
                    for i in indexes.flatten():
                        x, y, w, h = boxes[i]
                        label = str(self.classes[class_ids[i]])
                        objetos_detectados.add(label)  # Agregar a la lista de objetos detectados
                        color = (0, 255, 0)  # Verde por defecto
                        if label in ["knife", "pistol"]:  # Cambiar a rojo si es un objeto peligroso
                            color = (255, 0, 0)  # Rojo para objetos peligrosos
                            current_time = time.time()
                            if self.alarma_activar and (current_time - self.last_alert_time > 3):
                                self.reproducir_alerta()
                                self.last_alert_time = current_time
                        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Actualizar el historial de objetos detectados
            if objetos_detectados:
                self.agregar_historial(f"Objetos detectados: {', '.join(objetos_detectados)}")
                self.objetos_para_anunciar.update(objetos_detectados)

            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_pil = Image.fromarray(frame_rgb)
            frame_tk = ImageTk.PhotoImage(frame_pil)
            self.canvas.create_image(0, 0, anchor=tk.NW, image=frame_tk)
            self.canvas.image = frame_tk

    def anunciar_objetos_detectados(self):
        while True:
            if self.objetos_para_anunciar and self.voz_activar:
                objetos = ", ".join(self.objetos_para_anunciar)
                self.engine.say(f"Objetos detectados: {objetos}")
                self.engine.runAndWait()
                self.objetos_para_anunciar.clear()
            time.sleep(1)

    def cerrar_camara(self):
        if self.camara_activa:
            self.camara_activa = False
            
            # Detener el hilo de video
            if self.thread is not None:
                self.thread.join(timeout=1)  # Espera un segundo para que el hilo termine
            
            # Liberar la cámara
            if self.cap is not None:
                self.cap.release()
            
            # Actualizar el estado de los botones
            self.boton_abrir.config(state=tk.NORMAL)
            self.boton_cerrar.config(state=tk.DISABLED)
            self.boton_foto.config(state=tk.DISABLED)
            self.boton_video.config(state=tk.DISABLED)
            self.boton_deteccion.config(state=tk.DISABLED)
            self.boton_voz.config(state=tk.DISABLED)
            self.boton_alarma.config(state=tk.DISABLED)
            self.boton_descargar.config(state=tk.DISABLED)

            self.agregar_historial("Cámara cerrada")

    def tomar_foto(self):
        if self.camara_activa:
            ret, frame = self.cap.read()
            if ret:
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                photo_filename = f"foto_{timestamp}.jpg"
                cv2.imwrite(photo_filename, frame)
                self.agregar_historial("Foto tomada")
                messagebox.showinfo("Foto tomada", f"La foto se guardó como {photo_filename}.")

    def grabar_video(self):
        if self.camara_activa:
            self.grabando_video = not self.grabando_video

            if self.grabando_video:
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                video_filename = f"video_{timestamp}.avi"
                self.video_writer = cv2.VideoWriter(video_filename, cv2.VideoWriter_fourcc(*'XVID'), 20.0, (640, 480))
                self.boton_video.config(text="Detener Video")
                self.agregar_historial("Grabación de video iniciada")
            else:
                self.video_writer.release()
                self.boton_video.config(text="Grabar Video")
                self.agregar_historial("Grabación de video detenida")
                messagebox.showinfo("Grabación detenida", "La grabación de video ha sido detenida.")

    def toggle_deteccion(self):
        self.detectando_objetos = not self.detectando_objetos
        estado = "activada" if self.detectando_objetos else "desactivada"
        self.agregar_historial(f"Detección de objetos {estado}")

    def toggle_voz(self):
        self.voz_activar = not self.voz_activar
        estado = "activada" if self.voz_activar else "desactivada"
        self.boton_voz.config(text=f"{estado.capitalize()} Voz")
        self.agregar_historial(f"Voz {estado}")

    def toggle_alarma(self):
        self.alarma_activar = not self.alarma_activar
        estado = "activada" if self.alarma_activar else "desactivada"
        self.boton_alarma.config(text=f"{estado.capitalize()} Alarma")
        self.agregar_historial(f"Alarma {estado}")

    def agregar_historial(self, mensaje):
        timestamp = datetime.datetime.now().strftime("%Y%m%d %H:%M:%S")
        self.historial_detecciones.append(f"{timestamp}: {mensaje}")
        if len(self.historial_detecciones) > 50:  # Limitar el tamaño del historial
            self.historial_detecciones.pop(0)
        
        # Actualizar el historial en el Text
        self.text_historial.config(state=tk.NORMAL)  # Habilitar el Text para la edición
        self.text_historial.delete(1.0, tk.END)  # Limpiar el Text
        self.text_historial.insert(tk.END, "\n".join(self.historial_detecciones))  # Agregar nuevo texto
        self.text_historial.config(state=tk.DISABLED)  # Deshabilitar el Text para evitar ediciones
        self.text_historial.yview_moveto(1)  # Desplazar hacia abajo

    def reproducir_alerta(self):
        try:
            pygame.mixer.music.load("alerta.mp3")
            pygame.mixer.music.play()
        except Exception as e:
            print(f"Error al reproducir el sonido: {e}")

    def descargar_historial(self):
        archivo = filedialog.asksaveasfile(defaultextension=".txt", filetypes=[("Text files", "*.txt")])
        if archivo:
            for entrada in self.historial_detecciones:
                archivo.write(entrada + "\n")
            archivo.close()
            messagebox.showinfo("Éxito", "Historial descargado correctamente.")

    def salir(self):
        self.cerrar_camara()  # Asegurarse de que se cierre la cámara
        self.root.quit()  # Cerrar la aplicación

# Crear ventana principal
root = tk.Tk()
root.geometry("1200x1100")  # Establecer un tamaño fijo para la ventana
root.resizable(False, False)  # Deshabilitar la opción de maximizar
app = CamaraApp(root)
root.mainloop()

In [9]:
import tkinter as tk
from tkinter import messagebox, Scrollbar, Frame, filedialog
from threading import Thread
import cv2
import datetime
import numpy as np
from PIL import Image, ImageTk
import pyttsx3
import os
import time
import pygame

class CamaraApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Detector de Imágenes - Demo")
        self.root.config(bg="#2e2e2e")

        pygame.mixer.init()

        self.camara_activa = False
        self.cap = None
        self.thread = None
        self.grabando_video = False
        self.video_writer = None
        self.detectando_objetos = False
        self.engine = pyttsx3.init()
        self.historial_detecciones = []
        self.objetos_para_anunciar = set()
        self.last_alert_time = 0
        self.voz_activar = True
        self.alarma_activar = True

        self.net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
        self.layer_names = self.net.getLayerNames()
        self.output_layers = [self.layer_names[i - 1] for i in self.net.getUnconnectedOutLayers()]
        self.clases = []
        with open("coco-es.names", "r") as f:
            self.clases = [line.strip() for line in f.readlines()]

        self.frame_principal = tk.Frame(root, bg="#2e2e2e")
        self.frame_principal.pack(fill=tk.BOTH, expand=True)

        self.frame_video = tk.Frame(self.frame_principal, bg="#2e2e2e")
        self.frame_video.pack(side=tk.LEFT, padx=10, pady=10)

        self.frame_historial = Frame(self.frame_principal)
        self.frame_historial.pack(side=tk.RIGHT, fill=tk.Y, padx=10, pady=10)

        self.scrollbar = Scrollbar(self.frame_historial)
        self.scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        self.label_historial_titulo = tk.Label(self.frame_historial, text="Historial", bg="#2e2e2e",
                                               fg="white", font=("Arial", 14, "bold"))
        self.label_historial_titulo.pack(pady=10)

        self.text_historial = tk.Text(self.frame_historial, bg="#2e2e2e", fg="white", font=("Arial", 12),
                                       wrap="word", height=20, width=40)
        self.text_historial.pack(fill=tk.BOTH, expand=True, padx=10)
        self.text_historial.config(yscrollcommand=self.scrollbar.set)
        self.scrollbar.config(command=self.text_historial.yview)

        self.logo_imagen_tk = None
        self.cargar_logo()

        self.frame_botones = tk.Frame(self.frame_video, bg="#3c3c3c")
        self.frame_botones.pack(pady=10, padx=10, fill=tk.X)

        self.boton_abrir = tk.Button(self.frame_botones, text="Abrir Cámara", command=self.abrir_camara, width=20,
                                     bg="#4CAF50", fg="white", font=("Arial", 12))
        self.boton_abrir.grid(row=0, column=0, padx=10, pady=10)

        self.boton_cerrar = tk.Button(self.frame_botones, text="Cerrar Cámara", command=self.cerrar_camara, width=20,
                                      state=tk.DISABLED, bg="#F44336", fg="white", font=("Arial", 12))
        self.boton_cerrar.grid(row=0, column=1, padx=10, pady=10)

        self.boton_foto = tk.Button(self.frame_botones, text="Tomar Foto", command=self.tomar_foto, width=20,
                                    state=tk.DISABLED, bg="#2196F3", fg="white", font=("Arial", 12))
        self.boton_foto.grid(row=1, column=0, padx=10, pady=10)

        self.boton_video = tk.Button(self.frame_botones, text="Grabar Video", command=self.grabar_video, width=20,
                                     state=tk.DISABLED, bg="#FF9800", fg="white", font=("Arial", 12))
        self.boton_video.grid(row=1, column=1, padx=10, pady=10)

        self.boton_deteccion = tk.Button(self.frame_botones, text="Detectar Objetos", command=self.toggle_deteccion,
                                         width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_deteccion.grid(row=2, column=0, padx=10, pady=10)

        self.boton_voz = tk.Button(self.frame_botones, text="Activar Voz", command=self.toggle_voz,
                                   width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_voz.grid(row=2, column=1, padx=10, pady=10)

        self.boton_alarma = tk.Button(self.frame_botones, text="Activar Alarma", command=self.toggle_alarma,
                                      width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_alarma.grid(row=3, column=0, padx=10, pady=10)

        self.boton_descargar = tk.Button(self.frame_botones, text="Descargar Historial", command=self.descargar_historial,
                                          width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_descargar.grid(row=3, column=1, padx=10, pady=10)

        self.boton_salir = tk.Button(self.frame_botones, text="Salir", command=self.salir, width=20,
                                     bg="#9E9E9E", fg="white", font=("Arial", 12))
        self.boton_salir.grid(row=4, column=0, columnspan=2, padx=10, pady=10)

        self.canvas = tk.Canvas(self.frame_video, width=640, height=480)
        self.canvas.pack()

        self.mostrar_tutorial()

    def cargar_logo(self):
        try:
            logo_path = "Logo1.png"
            if os.path.exists(logo_path):
                self.logo_imagen = Image.open(logo_path).resize((300, 100), Image.Resampling.LANCZOS)
                self.logo_imagen_tk = ImageTk.PhotoImage(self.logo_imagen)
                self.label_logo = tk.Label(self.frame_video, image=self.logo_imagen_tk, bg="#2e2e2e")
                self.label_logo.pack(pady=20)
            else:
                raise FileNotFoundError(f"El archivo {logo_path} no se encuentra.")
        except Exception as e:
            print(f"Error al cargar la imagen: {e}")

    def mostrar_tutorial(self):
        self.tutorial = tk.Toplevel(self.root)
        self.tutorial.title("Tutorial")
        self.tutorial.geometry("400x300")
        self.tutorial.config(bg="#2e2e2e")

        tutorial_text = """
        Bienvenido a la aplicación de detección de imágenes.

        1. Abrir Cámara: Inicia la cámara para la detección.
        2. Detectar Objetos: Activa la detección de objetos en la cámara.
        3. Tomar Foto: Captura una imagen de la cámara.
        4. Grabar Video: Inicia o detiene la grabación de video.
        5. Descargar Historial: Guarda el historial de detecciones en un archivo.
        6. Activar Voz: Anuncia los objetos detectados.
        7. Activar Alarma: Reproduce una alarma si se detectan objetos peligrosos.
        8. Salir: Cierra la aplicación.

        Presiona 'Saltar Tutorial' para omitir este tutorial.
        """
        
        label_tutorial = tk.Label(self.tutorial, text=tutorial_text, bg="#2e2e2e", fg="white", font=("Arial", 12))
        label_tutorial.pack(pady=10)

        boton_saltar = tk.Button(self.tutorial, text="Saltar Tutorial", command=self.saltar_tutorial,
                                  bg="#FFC107", fg="black", font=("Arial", 12))
        boton_saltar.pack(pady=20)

    def saltar_tutorial(self):
        self.tutorial.destroy()

    def abrir_camara(self):
        if not self.camara_activa:
            self.cap = cv2.VideoCapture(0)
            if not self.cap.isOpened():
                messagebox.showerror("Error", "No se pudo abrir la cámara.")
                return

            self.camara_activa = True
            self.boton_abrir.config(state=tk.DISABLED)
            self.boton_cerrar.config(state=tk.NORMAL)
            self.boton_foto.config(state=tk.NORMAL)
            self.boton_video.config(state=tk.NORMAL)
            self.boton_deteccion.config(state=tk.NORMAL)
            self.boton_voz.config(state=tk.NORMAL)
            self.boton_alarma.config(state=tk.NORMAL)
            self.boton_descargar.config(state=tk.NORMAL)

            self.agregar_historial("Cámara abierta")

            self.thread = Thread(target=self.actualizar_video)
            self.thread.daemon = True
            self.thread.start()

            self.thread_anunciar = Thread(target=self.anunciar_objetos_detectados)
            self.thread_anunciar.daemon = True
            self.thread_anunciar.start()

    def actualizar_video(self):
        while self.camara_activa:
            ret, frame = self.cap.read()
            if not ret:
                break

            objetos_detectados = set()  # Para almacenar objetos detectados

            if self.detectando_objetos:
                # Prepara la imagen para la detección
                blob = cv2.dnn.blobFromImage(frame, 0.00392, (320, 320), (0, 0, 0), True, crop=False)
                self.net.setInput(blob)
                outs = self.net.forward(self.output_layers)

                class_ids = []
                confidences = []
                boxes = []

                for out in outs:
                    for detection in out:
                        scores = detection[5:]
                        class_id = np.argmax(scores)
                        confidence = scores[class_id]
                        if confidence > 0.6:
                            center_x = int(detection[0] * frame.shape[1])
                            center_y = int(detection[1] * frame.shape[0])
                            w = int(detection[2] * frame.shape[1])
                            h = int(detection[3] * frame.shape[0])
                            x = int(center_x - w / 2)
                            y = int(center_y - h / 2)
                            boxes.append([x, y, w, h])
                            confidences.append(float(confidence))
                            class_ids.append(class_id)

                indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.6, 0.4)

                if len(indexes) > 0:
                    for i in indexes.flatten():
                        x, y, w, h = boxes[i]
                        label = str(self.clases[class_ids[i]])  # Usar nombres en español
                        objetos_detectados.add(label)  # Agregar a la lista de objetos detectados
                        color = (0, 255, 0)  # Verde por defecto
                        if label in ["cuchillo", "pistola"]:  # Cambiar a rojo si es un objeto peligroso
                            color = (255, 0, 0)  # Rojo para objetos peligrosos
                            current_time = time.time()
                            if self.alarma_activar and (current_time - self.last_alert_time > 3):
                                self.reproducir_alerta()
                                self.last_alert_time = current_time
                        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Actualizar el historial de objetos detectados
            if objetos_detectados:
                self.agregar_historial(f"Objetos detectados: {', '.join(objetos_detectados)}")
                self.objetos_para_anunciar.update(objetos_detectados)

            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_pil = Image.fromarray(frame_rgb)
            frame_tk = ImageTk.PhotoImage(frame_pil)
            self.canvas.create_image(0, 0, anchor=tk.NW, image=frame_tk)
            self.canvas.image = frame_tk

    def anunciar_objetos_detectados(self):
        while True:
            if self.objetos_para_anunciar and self.voz_activar:
                objetos = ", ".join(self.objetos_para_anunciar)
                self.engine.say(f"Objetos detectados: {objetos}")
                self.engine.runAndWait()
                self.objetos_para_anunciar.clear()
            time.sleep(1)

    def cerrar_camara(self):
        if self.camara_activa:
            self.camara_activa = False
            
            # Detener el hilo de video
            if self.thread is not None:
                self.thread.join(timeout=1)  # Espera un segundo para que el hilo termine
            
            # Liberar la cámara
            if self.cap is not None:
                self.cap.release()
            
            # Actualizar el estado de los botones
            self.boton_abrir.config(state=tk.NORMAL)
            self.boton_cerrar.config(state=tk.DISABLED)
            self.boton_foto.config(state=tk.DISABLED)
            self.boton_video.config(state=tk.DISABLED)
            self.boton_deteccion.config(state=tk.DISABLED)
            self.boton_voz.config(state=tk.DISABLED)
            self.boton_alarma.config(state=tk.DISABLED)
            self.boton_descargar.config(state=tk.DISABLED)

            self.agregar_historial("Cámara cerrada")

    def tomar_foto(self):
        if self.camara_activa:
            ret, frame = self.cap.read()
            if ret:
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                photo_filename = f"foto_{timestamp}.jpg"
                cv2.imwrite(photo_filename, frame)
                self.agregar_historial("Foto tomada")
                messagebox.showinfo("Foto tomada", f"La foto se guardó como {photo_filename}.")

    def grabar_video(self):
        if self.camara_activa:
            self.grabando_video = not self.grabando_video

            if self.grabando_video:
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                video_filename = f"video_{timestamp}.avi"
                self.video_writer = cv2.VideoWriter(video_filename, cv2.VideoWriter_fourcc(*'XVID'), 20.0, (640, 480))
                self.boton_video.config(text="Detener Video")
                self.agregar_historial("Grabación de video iniciada")
            else:
                self.video_writer.release()
                self.boton_video.config(text="Grabar Video")
                self.agregar_historial("Grabación de video detenida")
                messagebox.showinfo("Grabación detenida", "La grabación de video ha sido detenida.")

    def toggle_deteccion(self):
        self.detectando_objetos = not self.detectando_objetos
        estado = "activada" if self.detectando_objetos else "desactivada"
        self.agregar_historial(f"Detección de objetos {estado}")

    def toggle_voz(self):
        self.voz_activar = not self.voz_activar
        estado = "activada" if self.voz_activar else "desactivada"
        self.boton_voz.config(text=f"{estado.capitalize()} Voz")
        self.agregar_historial(f"Voz {estado}")

    def toggle_alarma(self):
        self.alarma_activar = not self.alarma_activar
        estado = "activada" if self.alarma_activar else "desactivada"
        self.boton_alarma.config(text=f"{estado.capitalize()} Alarma")
        self.agregar_historial(f"Alarma {estado}")

    def agregar_historial(self, mensaje):
        timestamp = datetime.datetime.now().strftime("%Y%m%d %H:%M:%S")
        self.historial_detecciones.append(f"{timestamp}: {mensaje}")
        if len(self.historial_detecciones) > 50:  # Limitar el tamaño del historial
            self.historial_detecciones.pop(0)
        
        # Actualizar el historial en el Text
        self.text_historial.config(state=tk.NORMAL)  # Habilitar el Text para la edición
        self.text_historial.delete(1.0, tk.END)  # Limpiar el Text
        self.text_historial.insert(tk.END, "\n".join(self.historial_detecciones))  # Agregar nuevo texto
        self.text_historial.config(state=tk.DISABLED)  # Deshabilitar el Text para evitar ediciones
        self.text_historial.yview_moveto(1)  # Desplazar hacia abajo

    def reproducir_alerta(self):
        try:
            pygame.mixer.music.load("alerta.mp3")
            pygame.mixer.music.play()
        except Exception as e:
            print(f"Error al reproducir el sonido: {e}")

    def descargar_historial(self):
        archivo = filedialog.asksaveasfile(defaultextension=".txt", filetypes=[("Archivos de texto", "*.txt")])
        if archivo:
            for entrada in self.historial_detecciones:
                archivo.write(entrada + "\n")
            archivo.close()
            messagebox.showinfo("Éxito", "Historial descargado correctamente.")

    def salir(self):
        self.cerrar_camara()  # Asegurarse de que se cierre la cámara
        self.root.quit()  # Cerrar la aplicación

# Crear ventana principal
root = tk.Tk()
root.geometry("1200x1100")  # Establecer un tamaño fijo para la ventana
root.resizable(False, False)  # Deshabilitar la opción de maximizar
app = CamaraApp(root)
root.mainloop()

In [ ]:
### Intento en español error al intentar usar el archivo en españo, el programa confunde letras y agrega valores numericos envez de leer 

In [2]:
import tkinter as tk
from tkinter import messagebox, Scrollbar, Frame, filedialog
from threading import Thread
import cv2
import datetime
import numpy as np
from PIL import Image, ImageTk
import pyttsx3
import os
import time
import pygame

class CamaraApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Detector de Imágenes - Demo")
        self.root.config(bg="#2e2e2e")

        pygame.mixer.init()

        self.camara_activa = False
        self.cap = None
        self.thread = None
        self.grabando_video = False
        self.video_writer = None
        self.detectando_objetos = False
        self.engine = pyttsx3.init()
        self.historial_detecciones = []
        self.objetos_para_anunciar = set()
        self.last_alert_time = 0
        self.voz_activar = True
        self.alarma_activar = True

        self.net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
        self.layer_names = self.net.getLayerNames()
        self.output_layers = [self.layer_names[i - 1] for i in self.net.getUnconnectedOutLayers()]
        self.clases = []
        with open("coco-es.names", "r") as f:
            self.clases = [line.strip() for line in f.readlines()]

        self.frame_principal = tk.Frame(root, bg="#2e2e2e")
        self.frame_principal.pack(fill=tk.BOTH, expand=True)

        self.frame_video = tk.Frame(self.frame_principal, bg="#2e2e2e")
        self.frame_video.pack(side=tk.LEFT, padx=10, pady=10)

        self.frame_historial = Frame(self.frame_principal)
        self.frame_historial.pack(side=tk.RIGHT, fill=tk.Y, padx=10, pady=10)

        self.scrollbar = Scrollbar(self.frame_historial)
        self.scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        self.label_historial_titulo = tk.Label(self.frame_historial, text="Historial", bg="#2e2e2e",
                                               fg="white", font=("Arial", 14, "bold"))
        self.label_historial_titulo.pack(pady=10)

        self.text_historial = tk.Text(self.frame_historial, bg="#2e2e2e", fg="white", font=("Arial", 12),
                                       wrap="word", height=20, width=40)
        self.text_historial.pack(fill=tk.BOTH, expand=True, padx=10)
        self.text_historial.config(yscrollcommand=self.scrollbar.set)
        self.scrollbar.config(command=self.text_historial.yview)

        self.logo_imagen_tk = None
        self.cargar_logo()

        self.frame_botones = tk.Frame(self.frame_video, bg="#3c3c3c")
        self.frame_botones.pack(pady=10, padx=10, fill=tk.X)

        self.boton_abrir = tk.Button(self.frame_botones, text="Abrir Cámara", command=self.abrir_camara, width=20,
                                     bg="#4CAF50", fg="white", font=("Arial", 12))
        self.boton_abrir.grid(row=0, column=0, padx=10, pady=10)

        self.boton_cerrar = tk.Button(self.frame_botones, text="Cerrar Cámara", command=self.cerrar_camara, width=20,
                                      state=tk.DISABLED, bg="#F44336", fg="white", font=("Arial", 12))
        self.boton_cerrar.grid(row=0, column=1, padx=10, pady=10)

        self.boton_foto = tk.Button(self.frame_botones, text="Tomar Foto", command=self.tomar_foto, width=20,
                                    state=tk.DISABLED, bg="#2196F3", fg="white", font=("Arial", 12))
        self.boton_foto.grid(row=1, column=0, padx=10, pady=10)

        self.boton_video = tk.Button(self.frame_botones, text="Grabar Video", command=self.grabar_video, width=20,
                                     state=tk.DISABLED, bg="#FF9800", fg="white", font=("Arial", 12))
        self.boton_video.grid(row=1, column=1, padx=10, pady=10)

        self.boton_deteccion = tk.Button(self.frame_botones, text="Detectar Objetos", command=self.toggle_deteccion,
                                         width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_deteccion.grid(row=2, column=0, padx=10, pady=10)

        self.boton_voz = tk.Button(self.frame_botones, text="Activar Voz", command=self.toggle_voz,
                                   width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_voz.grid(row=2, column=1, padx=10, pady=10)

        self.boton_alarma = tk.Button(self.frame_botones, text="Activar Alarma", command=self.toggle_alarma,
                                      width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_alarma.grid(row=3, column=0, padx=10, pady=10)

        self.boton_descargar = tk.Button(self.frame_botones, text="Descargar Historial", command=self.descargar_historial,
                                          width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_descargar.grid(row=3, column=1, padx=10, pady=10)

        self.boton_salir = tk.Button(self.frame_botones, text="Salir", command=self.salir, width=20,
                                     bg="#9E9E9E", fg="white", font=("Arial", 12))
        self.boton_salir.grid(row=4, column=0, columnspan=2, padx=10, pady=10)

        self.canvas = tk.Canvas(self.frame_video, width=640, height=480)
        self.canvas.pack()

        self.mostrar_tutorial()

    def cargar_logo(self):
        try:
            logo_path = "Logo1.png"
            if os.path.exists(logo_path):
                self.logo_imagen = Image.open(logo_path).resize((300, 100), Image.Resampling.LANCZOS)
                self.logo_imagen_tk = ImageTk.PhotoImage(self.logo_imagen)
                self.label_logo = tk.Label(self.frame_video, image=self.logo_imagen_tk, bg="#2e2e2e")
                self.label_logo.pack(pady=20)
            else:
                raise FileNotFoundError(f"El archivo {logo_path} no se encuentra.")
        except Exception as e:
            print(f"Error al cargar la imagen: {e}")

    def mostrar_tutorial(self):
        self.tutorial = tk.Toplevel(self.root)
        self.tutorial.title("Tutorial")
        self.tutorial.geometry("800x600")  # Tamaño de la ventana del tutorial
        self.tutorial.config(bg="#2e2e2e")

        tutorial_text = """
        Bienvenido a la aplicación de detección de imágenes.

        1. Abrir Cámara: Inicia la cámara para la detección.
        2. Detectar Objetos: Activa la detección de objetos en la cámara.
        3. Tomar Foto: Captura una imagen de la cámara.
        4. Grabar Video: Inicia o detiene la grabación de video.
        5. Descargar Historial: Guarda el historial de detecciones en un archivo.
        6. Activar Voz: Anuncia los objetos detectados.
        7. Activar Alarma: Reproduce una alarma si se detectan objetos peligrosos.
        8. Salir: Cierra la aplicación.

        Presiona 'Saltar Tutorial' para omitir este tutorial.
        """
        
        label_tutorial = tk.Label(self.tutorial, text=tutorial_text, bg="#2e2e2e", fg="white", font=("Arial", 12))
        label_tutorial.pack(pady=10)

        boton_saltar = tk.Button(self.tutorial, text="Saltar Tutorial", command=self.saltar_tutorial,
                                  bg="#FFC107", fg="black", font=("Arial", 12))
        boton_saltar.pack(pady=20)

    def saltar_tutorial(self):
        self.tutorial.destroy()

    def abrir_camara(self):
        if not self.camara_activa:
            self.cap = cv2.VideoCapture(0)
            if not self.cap.isOpened():
                messagebox.showerror("Error", "No se pudo abrir la cámara.")
                return

            self.camara_activa = True
            self.boton_abrir.config(state=tk.DISABLED)
            self.boton_cerrar.config(state=tk.NORMAL)
            self.boton_foto.config(state=tk.NORMAL)
            self.boton_video.config(state=tk.NORMAL)
            self.boton_deteccion.config(state=tk.NORMAL)
            self.boton_voz.config(state=tk.NORMAL)
            self.boton_alarma.config(state=tk.NORMAL)
            self.boton_descargar.config(state=tk.NORMAL)

            self.agregar_historial("Cámara abierta")

            self.thread = Thread(target=self.actualizar_video)
            self.thread.daemon = True
            self.thread.start()

            self.thread_anunciar = Thread(target=self.anunciar_objetos_detectados)
            self.thread_anunciar.daemon = True
            self.thread_anunciar.start()

    def actualizar_video(self):
        while self.camara_activa:
            ret, frame = self.cap.read()
            if not ret:
                break

            objetos_detectados = set()  # Para almacenar objetos detectados

            if self.detectando_objetos:
                # Prepara la imagen para la detección
                blob = cv2.dnn.blobFromImage(frame, 0.00392, (320, 320), (0, 0, 0), True, crop=False)
                self.net.setInput(blob)
                outs = self.net.forward(self.output_layers)

                class_ids = []
                confidences = []
                boxes = []

                for out in outs:
                    for detection in out:
                        scores = detection[5:]
                        class_id = np.argmax(scores)
                        confidence = scores[class_id]
                        if confidence > 0.6:
                            center_x = int(detection[0] * frame.shape[1])
                            center_y = int(detection[1] * frame.shape[0])
                            w = int(detection[2] * frame.shape[1])
                            h = int(detection[3] * frame.shape[0])
                            x = int(center_x - w / 2)
                            y = int(center_y - h / 2)
                            boxes.append([x, y, w, h])
                            confidences.append(float(confidence))
                            class_ids.append(class_id)

                indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.6, 0.4)

                if len(indexes) > 0:
                    for i in indexes.flatten():
                        x, y, w, h = boxes[i]
                        label = str(self.clases[class_ids[i]])  # Usar nombres en español
                        objetos_detectados.add(label)  # Agregar a la lista de objetos detectados
                        color = (0, 255, 0)  # Verde por defecto
                        if label in ["cuchillo", "pistola"]:  # Cambiar a rojo si es un objeto peligroso
                            color = (255, 0, 0)  # Rojo para objetos peligrosos
                            current_time = time.time()
                            if self.alarma_activar and (current_time - self.last_alert_time > 3):
                                self.reproducir_alerta()
                                self.last_alert_time = current_time
                        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Actualizar el historial de objetos detectados
            if objetos_detectados:
                self.agregar_historial(f"Objetos detectados: {', '.join(objetos_detectados)}")
                self.objetos_para_anunciar.update(objetos_detectados)

            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_pil = Image.fromarray(frame_rgb)
            frame_tk = ImageTk.PhotoImage(frame_pil)
            self.canvas.create_image(0, 0, anchor=tk.NW, image=frame_tk)
            self.canvas.image = frame_tk

    def anunciar_objetos_detectados(self):
        while True:
            if self.objetos_para_anunciar and self.voz_activar:
                objetos = ", ".join(self.objetos_para_anunciar)
                self.engine.say(f"Objetos detectados: {objetos}")
                self.engine.runAndWait()
                self.objetos_para_anunciar.clear()
            time.sleep(1)

    def cerrar_camara(self):
        if self.camara_activa:
            self.camara_activa = False
            
            # Detener el hilo de video
            if self.thread is not None:
                self.thread.join(timeout=1)  # Espera un segundo para que el hilo termine
            
            # Liberar la cámara
            if self.cap is not None:
                self.cap.release()
            
            # Actualizar el estado de los botones
            self.boton_abrir.config(state=tk.NORMAL)
            self.boton_cerrar.config(state=tk.DISABLED)
            self.boton_foto.config(state=tk.DISABLED)
            self.boton_video.config(state=tk.DISABLED)
            self.boton_deteccion.config(state=tk.DISABLED)
            self.boton_voz.config(state=tk.DISABLED)
            self.boton_alarma.config(state=tk.DISABLED)
            self.boton_descargar.config(state=tk.DISABLED)

            self.agregar_historial("Cámara cerrada")

    def tomar_foto(self):
        if self.camara_activa:
            ret, frame = self.cap.read()
            if ret:
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                photo_filename = f"foto_{timestamp}.jpg"
                cv2.imwrite(photo_filename, frame)
                self.agregar_historial("Foto tomada")
                messagebox.showinfo("Foto tomada", f"La foto se guardó como {photo_filename}.")

    def grabar_video(self):
        if self.camara_activa:
            self.grabando_video = not self.grabando_video

            if self.grabando_video:
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                video_filename = f"video_{timestamp}.avi"
                self.video_writer = cv2.VideoWriter(video_filename, cv2.VideoWriter_fourcc(*'XVID'), 20.0, (640, 480))
                self.boton_video.config(text="Detener Video")
                self.agregar_historial("Grabación de video iniciada")
            else:
                self.video_writer.release()
                self.boton_video.config(text="Grabar Video")
                self.agregar_historial("Grabación de video detenida")
                messagebox.showinfo("Grabación detenida", "La grabación de video ha sido detenida.")

    def toggle_deteccion(self):
        self.detectando_objetos = not self.detectando_objetos
        estado = "activada" if self.detectando_objetos else "desactivada"
        self.agregar_historial(f"Detección de objetos {estado}")

    def toggle_voz(self):
        self.voz_activar = not self.voz_activar
        estado = "activada" if self.voz_activar else "desactivada"
        self.boton_voz.config(text=f"{estado.capitalize()} Voz")
        self.agregar_historial(f"Voz {estado}")

    def toggle_alarma(self):
        self.alarma_activar = not self.alarma_activar
        estado = "activada" if self.alarma_activar else "desactivada"
        self.boton_alarma.config(text=f"{estado.capitalize()} Alarma")
        self.agregar_historial(f"Alarma {estado}")

    def agregar_historial(self, mensaje):
        timestamp = datetime.datetime.now().strftime("%Y%m%d %H:%M:%S")
        self.historial_detecciones.append(f"{timestamp}: {mensaje}")
        if len(self.historial_detecciones) > 50:  # Limitar el tamaño del historial
            self.historial_detecciones.pop(0)
        
        # Actualizar el historial en el Text
        self.text_historial.config(state=tk.NORMAL)  # Habilitar el Text para la edición
        self.text_historial.delete(1.0, tk.END)  # Limpiar el Text
        self.text_historial.insert(tk.END, "\n".join(self.historial_detecciones))  # Agregar nuevo texto
        self.text_historial.config(state=tk.DISABLED)  # Deshabilitar el Text para evitar ediciones
        self.text_historial.yview_moveto(1)  # Desplazar hacia abajo

    def reproducir_alerta(self):
        try:
            pygame.mixer.music.load("alerta.mp3")
            pygame.mixer.music.play()
        except Exception as e:
            print(f"Error al reproducir el sonido: {e}")

    def descargar_historial(self):
        archivo = filedialog.asksaveasfile(defaultextension=".txt", filetypes=[("Archivos de texto", "*.txt")])
        if archivo:
            for entrada in self.historial_detecciones:
                archivo.write(entrada + "\n")
            archivo.close()
            messagebox.showinfo("Éxito", "Historial descargado correctamente.")

    def salir(self):
        self.cerrar_camara()  # Asegurarse de que se cierre la cámara
        self.root.quit()  # Cerrar la aplicación

# Crear ventana principal
root = tk.Tk()
root.geometry("1200x1100")  # Establecer un tamaño fijo para la ventana
root.resizable(False, False)  # Deshabilitar la opción de maximizar
app = CamaraApp(root)
root.mainloop()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import tkinter as tk
from tkinter import messagebox, Scrollbar, Frame, filedialog
from threading import Thread
import cv2
import datetime
import numpy as np
from PIL import Image, ImageTk
import pyttsx3
import os
import time
import pygame

class CamaraApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Detector de imágenes - Demo")
        self.root.config(bg="#2e2e2e")

        pygame.mixer.init()

        self.camara_activa = False
        self.cap = None
        self.thread = None
        self.grabando_video = False
        self.video_writer = None
        self.detectando_objetos = False
        self.engine = pyttsx3.init()
        self.historial_detecciones = []
        self.objetos_para_anunciar = set()
        self.last_alert_time = 0
        self.voz_activar = True
        self.alarma_activar = True

        self.net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
        self.layer_names = self.net.getLayerNames()
        self.output_layers = [self.layer_names[i - 1] for i in self.net.getUnconnectedOutLayers()]
        self.classes = []
        with open("coco.names", "r") as f:
            self.classes = [line.strip() for line in f.readlines()]

        self.frame_principal = tk.Frame(root, bg="#2e2e2e")
        self.frame_principal.pack(fill=tk.BOTH, expand=True)

        self.frame_video = tk.Frame(self.frame_principal, bg="#2e2e2e")
        self.frame_video.pack(side=tk.LEFT, padx=10, pady=10)

        self.frame_historial = Frame(self.frame_principal)
        self.frame_historial.pack(side=tk.RIGHT, fill=tk.Y, padx=10, pady=10)

        self.scrollbar = Scrollbar(self.frame_historial)
        self.scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        self.label_historial_titulo = tk.Label(self.frame_historial, text="Historial", bg="#2e2e2e",
                                               fg="white", font=("Arial", 14, "bold"))
        self.label_historial_titulo.pack(pady=10)

        self.text_historial = tk.Text(self.frame_historial, bg="#2e2e2e", fg="white", font=("Arial", 12),
                                       wrap="word", height=20, width=40)
        self.text_historial.pack(fill=tk.BOTH, expand=True, padx=10)
        self.text_historial.config(yscrollcommand=self.scrollbar.set)
        self.scrollbar.config(command=self.text_historial.yview)

        self.logo_imagen_tk = None
        self.cargar_logo()

        self.frame_botones = tk.Frame(self.frame_video, bg="#3c3c3c")
        self.frame_botones.pack(pady=10, padx=10, fill=tk.X)

        self.boton_abrir = tk.Button(self.frame_botones, text="Abrir Cámara", command=self.abrir_camara, width=20,
                                     bg="#4CAF50", fg="white", font=("Arial", 12))
        self.boton_abrir.grid(row=0, column=0, padx=10, pady=10)

        self.boton_cerrar = tk.Button(self.frame_botones, text="Cerrar Cámara", command=self.cerrar_camara, width=20,
                                      state=tk.DISABLED, bg="#F44336", fg="white", font=("Arial", 12))
        self.boton_cerrar.grid(row=0, column=1, padx=10, pady=10)

        self.boton_foto = tk.Button(self.frame_botones, text="Tomar Foto", command=self.tomar_foto, width=20,
                                    state=tk.DISABLED, bg="#2196F3", fg="white", font=("Arial", 12))
        self.boton_foto.grid(row=1, column=0, padx=10, pady=10)

        self.boton_video = tk.Button(self.frame_botones, text="Grabar Video", command=self.grabar_video, width=20,
                                     state=tk.DISABLED, bg="#FF9800", fg="white", font=("Arial", 12))
        self.boton_video.grid(row=1, column=1, padx=10, pady=10)

        self.boton_deteccion = tk.Button(self.frame_botones, text="Detectar Objetos", command=self.toggle_deteccion,
                                         width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_deteccion.grid(row=2, column=0, padx=10, pady=10)

        self.boton_voz = tk.Button(self.frame_botones, text="Activar Voz", command=self.toggle_voz,
                                   width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_voz.grid(row=2, column=1, padx=10, pady=10)

        self.boton_alarma = tk.Button(self.frame_botones, text="Activar Alarma", command=self.toggle_alarma,
                                      width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_alarma.grid(row=3, column=0, padx=10, pady=10)

        self.boton_descargar = tk.Button(self.frame_botones, text="Descargar Historial", command=self.descargar_historial,
                                          width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_descargar.grid(row=3, column=1, padx=10, pady=10)

        self.boton_salir = tk.Button(self.frame_botones, text="Salir", command=self.salir, width=20,
                                     bg="#9E9E9E", fg="white", font=("Arial", 12))
        self.boton_salir.grid(row=4, column=0, columnspan=2, padx=10, pady=10)

        self.canvas = tk.Canvas(self.frame_video, width=640, height=480)
        self.canvas.pack()

        self.mostrar_tutorial()
        self.root.after(2000, self.actualizar_historial)  # Programar la actualización del historial cada 2 segundos

    def cargar_logo(self):
        try:
            logo_path = "Logo1.png"
            if os.path.exists(logo_path):
                self.logo_imagen = Image.open(logo_path).resize((300, 100), Image.Resampling.LANCZOS)
                self.logo_imagen_tk = ImageTk.PhotoImage(self.logo_imagen)
                self.label_logo = tk.Label(self.frame_video, image=self.logo_imagen_tk, bg="#2e2e2e")
                self.label_logo.pack(pady=20)
            else:
                raise FileNotFoundError(f"El archivo {logo_path} no se encuentra.")
        except Exception as e:
            print(f"Error al cargar la imagen: {e}")

    def mostrar_tutorial(self):
        self.tutorial = tk.Toplevel(self.root)
        self.tutorial.title("Tutorial")
        self.tutorial.geometry("800x300")  # Cambiado a 800x300
        self.tutorial.config(bg="#2e2e2e")

        tutorial_text = """
        Bienvenido a la aplicación de detección de imágenes.

        1. Abrir Cámara: Inicia la cámara para la detección.
        2. Detectar Objetos: Activa la detección de objetos en la cámara.
        3. Tomar Foto: Captura una imagen de la cámara.
        4. Grabar Video: Inicia o detiene la grabación de video.
        5. Descargar Historial: Guarda el historial de detecciones en un archivo.
        6. Activar Voz: Anuncia los objetos detectados.
        7. Activar Alarma: Reproduce una alarma si se detectan objetos peligrosos.
        8. Salir: Cierra la aplicación.

        Presiona 'Saltar Tutorial' para omitir este tutorial.
        """
        
        label_tutorial = tk.Label(self.tutorial, text=tutorial_text, bg="#2e2e2e", fg="white", font=("Arial", 12))
        label_tutorial.pack(pady=10)

        boton_saltar = tk.Button(self.tutorial, text="Saltar Tutorial", command=self.saltar_tutorial,
                                  bg="#FFC107", fg="black", font=("Arial", 12))
        boton_saltar.pack(pady=20)

    def saltar_tutorial(self):
        self.tutorial.destroy()

    def abrir_camara(self):
        if not self.camara_activa:
            self.cap = cv2.VideoCapture(0)
            if not self.cap.isOpened():
                messagebox.showerror("Error", "No se pudo abrir la cámara.")
                return

            self.camara_activa = True
            self.boton_abrir.config(state=tk.DISABLED)
            self.boton_cerrar.config(state=tk.NORMAL)
            self.boton_foto.config(state=tk.NORMAL)
            self.boton_video.config(state=tk.NORMAL)
            self.boton_deteccion.config(state=tk.NORMAL)
            self.boton_voz.config(state=tk.NORMAL)
            self.boton_alarma.config(state=tk.NORMAL)
            self.boton_descargar.config(state=tk.NORMAL)

            self.agregar_historial("Cámara abierta")

            self.thread = Thread(target=self.actualizar_video)
            self.thread.daemon = True
            self.thread.start()

            self.thread_anunciar = Thread(target=self.anunciar_objetos_detectados)
            self.thread_anunciar.daemon = True
            self.thread_anunciar.start()

    def actualizar_video(self):
        while self.camara_activa:
            ret, frame = self.cap.read()
            if not ret:
                break

            objetos_detectados = set()  # Para almacenar objetos detectados

            if self.detectando_objetos:
                # Prepara la imagen para la detección
                blob = cv2.dnn.blobFromImage(frame, 0.00392, (320, 320), (0, 0, 0), True, crop=False)
                self.net.setInput(blob)
                outs = self.net.forward(self.output_layers)

                class_ids = []
                confidences = []
                boxes = []

                for out in outs:
                    for detection in out:
                        scores = detection[5:]
                        class_id = np.argmax(scores)
                        confidence = scores[class_id]
                        if confidence > 0.6:
                            center_x = int(detection[0] * frame.shape[1])
                            center_y = int(detection[1] * frame.shape[0])
                            w = int(detection[2] * frame.shape[1])
                            h = int(detection[3] * frame.shape[0])
                            x = int(center_x - w / 2)
                            y = int(center_y - h / 2)
                            boxes.append([x, y, w, h])
                            confidences.append(float(confidence))
                            class_ids.append(class_id)

                indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.6, 0.4)

                if len(indexes) > 0:
                    for i in indexes.flatten():
                        x, y, w, h = boxes[i]
                        label = str(self.classes[class_ids[i]])
                        objetos_detectados.add(label)  # Agregar a la lista de objetos detectados
                        color = (0, 255, 0)  # Verde por defecto
                        if label in ["knife", "pistol"]:  # Cambiar a rojo si es un objeto peligroso
                            color = (255, 0, 0)  # Rojo para objetos peligrosos
                            current_time = time.time()
                            if self.alarma_activar and (current_time - self.last_alert_time > 3):
                                self.reproducir_alerta()
                                self.last_alert_time = current_time
                        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Actualizar el historial de objetos detectados
            if objetos_detectados:
                self.agregar_historial(f"Objetos detectados: {', '.join(objetos_detectados)}")
                self.objetos_para_anunciar.update(objetos_detectados)

            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_pil = Image.fromarray(frame_rgb)
            frame_tk = ImageTk.PhotoImage(frame_pil)
            self.canvas.create_image(0, 0, anchor=tk.NW, image=frame_tk)
            self.canvas.image = frame_tk

    def anunciar_objetos_detectados(self):
        while True:
            if self.objetos_para_anunciar and self.voz_activar:
                objetos = ", ".join(self.objetos_para_anunciar)
                self.engine.say(f"Objetos detectados: {objetos}")
                self.engine.runAndWait()
                self.objetos_para_anunciar.clear()
            time.sleep(1)

    def cerrar_camara(self):
        if self.camara_activa:
            self.camara_activa = False
            
            # Detener el hilo de video
            if self.thread is not None:
                self.thread.join(timeout=1)  # Espera un segundo para que el hilo termine
            
            # Liberar la cámara
            if self.cap is not None:
                self.cap.release()
            
            # Actualizar el estado de los botones
            self.boton_abrir.config(state=tk.NORMAL)
            self.boton_cerrar.config(state=tk.DISABLED)
            self.boton_foto.config(state=tk.DISABLED)
            self.boton_video.config(state=tk.DISABLED)
            self.boton_deteccion.config(state=tk.DISABLED)
            self.boton_voz.config(state=tk.DISABLED)
            self.boton_alarma.config(state=tk.DISABLED)
            self.boton_descargar.config(state=tk.DISABLED)

            self.agregar_historial("Cámara cerrada")

    def tomar_foto(self):
        if self.camara_activa:
            ret, frame = self.cap.read()
            if ret:
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                photo_filename = f"foto_{timestamp}.jpg"
                cv2.imwrite(photo_filename, frame)
                self.agregar_historial("Foto tomada")
                messagebox.showinfo("Foto tomada", f"La foto se guardó como {photo_filename}.")

    def grabar_video(self):
        if self.camara_activa:
            self.grabando_video = not self.grabando_video

            if self.grabando_video:
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                video_filename = f"video_{timestamp}.avi"
                self.video_writer = cv2.VideoWriter(video_filename, cv2.VideoWriter_fourcc(*'XVID'), 20.0, (640, 480))
                self.boton_video.config(text="Detener Video")
                self.agregar_historial("Grabación de video iniciada")
            else:
                self.video_writer.release()
                self.boton_video.config(text="Grabar Video")
                self.agregar_historial("Grabación de video detenida")
                messagebox.showinfo("Grabación detenida", "La grabación de video ha sido detenida.")

    def toggle_deteccion(self):
        self.detectando_objetos = not self.detectando_objetos
        estado = "activada" if self.detectando_objetos else "desactivada"
        self.agregar_historial(f"Detección de objetos {estado}")

    def toggle_voz(self):
        self.voz_activar = not self.voz_activar
        estado = "activada" if self.voz_activar else "desactivada"
        self.boton_voz.config(text=f"{estado.capitalize()} Voz")
        self.agregar_historial(f"Voz {estado}")

    def toggle_alarma(self):
        self.alarma_activar = not self.alarma_activar
        estado = "activada" if self.alarma_activar else "desactivada"
        self.boton_alarma.config(text=f"{estado.capitalize()} Alarma")
        self.agregar_historial(f"Alarma {estado}")

    def agregar_historial(self, mensaje):
        timestamp = datetime.datetime.now().strftime("%Y%m%d %H:%M:%S")
        self.historial_detecciones.append(f"{timestamp}: {mensaje}")
        if len(self.historial_detecciones) > 50:  # Limitar el tamaño del historial
            self.historial_detecciones.pop(0)

    def actualizar_historial(self):
        # Actualizar el historial en el Text
        self.text_historial.config(state=tk.NORMAL)  # Habilitar el Text para la edición
        self.text_historial.delete(1.0, tk.END)  # Limpiar el Text
        self.text_historial.insert(tk.END, "\n".join(self.historial_detecciones))  # Agregar nuevo texto
        self.text_historial.config(state=tk.DISABLED)  # Deshabilitar el Text para evitar ediciones
        self.text_historial.yview_moveto(1)

        self.root.after(2000, self.actualizar_historial)  

    def reproducir_alerta(self):
        try:
            pygame.mixer.music.load("alerta.mp3")
            pygame.mixer.music.play()
        except Exception as e:
            print(f"Error al reproducir el sonido: {e}")

    def descargar_historial(self):
        archivo = filedialog.asksaveasfile(defaultextension=".txt", filetypes=[("Text files", "*.txt")])
        if archivo:
            for entrada in self.historial_detecciones:
                archivo.write(entrada + "\n")
            archivo.close()
            messagebox.showinfo("Éxito", "Historial descargado correctamente.")

    def salir(self):
        self.cerrar_camara()  # Asegurarse de que se cierre la cámara
        self.root.quit()  # Cerrar la aplicación

# Crear ventana principal
root = tk.Tk()
root.geometry("1200x1100")  # Establecer un tamaño fijo para la ventana
root.resizable(False, False)  # Deshabilitar la opción de maximizar
app = CamaraApp(root)
root.mainloop()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
###  Intento con yolo8 , se detectan mas cosas ¿menor precision o mayor?,se va a seguir probndo para comprobarlo 

In [4]:
import tkinter as tk
from tkinter import messagebox, Scrollbar, Frame, filedialog
from threading import Thread
import cv2
import datetime
import numpy as np
from PIL import Image, ImageTk
import pyttsx3
import os
import time
import pygame
from ultralytics import YOLO  # Importar YOLO de Ultralytics

class CamaraApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Detector de Imágenes - Demo")
        self.root.config(bg="#2e2e2e")

        pygame.mixer.init()

        self.camara_activa = False
        self.cap = None
        self.thread = None
        self.grabando_video = False
        self.video_writer = None
        self.detectando_objetos = False
        self.engine = pyttsx3.init()
        self.historial_detecciones = []
        self.objetos_para_anunciar = set()
        self.last_alert_time = 0
        self.voz_activar = True
        self.alarma_activar = True

        # Cargar el modelo YOLOv8
        self.model = YOLO('yolov8n.pt')  # Cargar el modelo YOLOv8 Nano
        self.clases = self.model.names  # Obtener los nombres de las clases

        self.frame_principal = tk.Frame(root, bg="#2e2e2e")
        self.frame_principal.pack(fill=tk.BOTH, expand=True)

        self.frame_video = tk.Frame(self.frame_principal, bg="#2e2e2e")
        self.frame_video.pack(side=tk.LEFT, padx=10, pady=10)

        self.frame_historial = Frame(self.frame_principal)
        self.frame_historial.pack(side=tk.RIGHT, fill=tk.Y, padx=10, pady=10)

        self.scrollbar = Scrollbar(self.frame_historial)
        self.scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        self.label_historial_titulo = tk.Label(self.frame_historial, text="Historial", bg="#2e2e2e",
                                               fg="white", font=("Arial", 14, "bold"))
        self.label_historial_titulo.pack(pady=10)

        self.text_historial = tk.Text(self.frame_historial, bg="#2e2e2e", fg="white", font=("Arial", 12),
                                       wrap="word", height=20, width=40)
        self.text_historial.pack(fill=tk.BOTH, expand=True, padx=10)
        self.text_historial.config(yscrollcommand=self.scrollbar.set)
        self.scrollbar.config(command=self.text_historial.yview)

        self.logo_imagen_tk = None
        self.cargar_logo()

        self.frame_botones = tk.Frame(self.frame_video, bg="#3c3c3c")
        self.frame_botones.pack(pady=10, padx=10, fill=tk.X)

        self.boton_abrir = tk.Button(self.frame_botones, text="Abrir Cámara", command=self.abrir_camara, width=20,
                                     bg="#4CAF50", fg="white", font=("Arial", 12))
        self.boton_abrir.grid(row=0, column=0, padx=10, pady=10)

        self.boton_cerrar = tk.Button(self.frame_botones, text="Cerrar Cámara", command=self.cerrar_camara, width=20,
                                      state=tk.DISABLED, bg="#F44336", fg="white", font=("Arial", 12))
        self.boton_cerrar.grid(row=0, column=1, padx=10, pady=10)

        self.boton_foto = tk.Button(self.frame_botones, text="Tomar Foto", command=self.tomar_foto, width=20,
                                    state=tk.DISABLED, bg="#2196F3", fg="white", font=("Arial", 12))
        self.boton_foto.grid(row=1, column=0, padx=10, pady=10)

        self.boton_video = tk.Button(self.frame_botones, text="Grabar Video", command=self.grabar_video, width=20,
                                     state=tk.DISABLED, bg="#FF9800", fg="white", font=("Arial", 12))
        self.boton_video.grid(row=1, column=1, padx=10, pady=10)

        self.boton_deteccion = tk.Button(self.frame_botones, text="Detectar Objetos", command=self.toggle_deteccion,
                                         width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_deteccion.grid(row=2, column=0, padx=10, pady=10)

        self.boton_voz = tk.Button(self.frame_botones, text="Activar Voz", command=self.toggle_voz,
                                   width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_voz.grid(row=2, column=1, padx=10, pady=10)

        self.boton_alarma = tk.Button(self.frame_botones, text="Activar Alarma", command=self.toggle_alarma,
                                      width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_alarma.grid(row=3, column=0, padx=10, pady=10)

        self.boton_descargar = tk.Button(self.frame_botones, text="Descargar Historial", command=self.descargar_historial,
                                          width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_descargar.grid(row=3, column=1, padx=10, pady=10)

        self.boton_salir = tk.Button(self.frame_botones, text="Salir", command=self.salir, width=20,
                                     bg="#9E9E9E", fg="white", font=("Arial", 12))
        self.boton_salir.grid(row=4, column=0, columnspan=2, padx=10, pady=10)

        self.canvas = tk.Canvas(self.frame_video, width=640, height=480)
        self.canvas.pack()

        self.mostrar_tutorial()

    def cargar_logo(self):
        try:
            logo_path = "Logo1.png"
            if os.path.exists(logo_path):
                self.logo_imagen = Image.open(logo_path).resize((300, 100), Image.Resampling.LANCZOS)
                self.logo_imagen_tk = ImageTk.PhotoImage(self.logo_imagen)
                self.label_logo = tk.Label(self.frame_video, image=self.logo_imagen_tk, bg="#2e2e2e")
                self.label_logo.pack(pady=20)
            else:
                raise FileNotFoundError(f"El archivo {logo_path} no se encuentra.")
        except Exception as e:
            print(f"Error al cargar la imagen: {e}")

    def mostrar_tutorial(self):
        self.tutorial = tk.Toplevel(self.root)
        self.tutorial.title("Tutorial")
        self.tutorial.geometry("800x600")  # Tamaño de la ventana del tutorial
        self.tutorial.config(bg="#2e2e2e")

        tutorial_text = """
        Bienvenido a la aplicación de detección de imágenes.

        1. Abrir Cámara: Inicia la cámara para la detección.
        2. Detectar Objetos: Activa la detección de objetos en la cámara.
        3. Tomar Foto: Captura una imagen de la cámara.
        4. Grabar Video: Inicia o detiene la grabación de video.
        5. Descargar Historial: Guarda el historial de detecciones en un archivo.
        6. Activar Voz: Anuncia los objetos detectados.
        7. Activar Alarma: Reproduce una alarma si se detectan objetos peligrosos.
        8. Salir: Cierra la aplicación.

        Presiona 'Saltar Tutorial' para omitir este tutorial.
        """
        
        label_tutorial = tk.Label(self.tutorial, text=tutorial_text, bg="#2e2e2e", fg="white", font=("Arial", 12))
        label_tutorial.pack(pady=10)

        boton_saltar = tk.Button(self.tutorial, text="Saltar Tutorial", command=self.saltar_tutorial,
                                  bg="#FFC107", fg="black", font=("Arial", 12))
        boton_saltar.pack(pady=20)

    def saltar_tutorial(self):
        self.tutorial.destroy()

    def abrir_camara(self):
        if not self.camara_activa:
            self.cap = cv2.VideoCapture(0)
            if not self.cap.isOpened():
                messagebox.showerror("Error", "No se pudo abrir la cámara.")
                return

            self.camara_activa = True
            self.boton_abrir.config(state=tk.DISABLED)
            self.boton_cerrar.config(state=tk.NORMAL)
            self.boton_foto.config(state=tk.NORMAL)
            self.boton_video.config(state=tk.NORMAL)
            self.boton_deteccion.config(state=tk.NORMAL)
            self.boton_voz.config(state=tk.NORMAL)
            self.boton_alarma.config(state=tk.NORMAL)
            self.boton_descargar.config(state=tk.NORMAL)

            self.agregar_historial("Cámara abierta")

            self.thread = Thread(target=self.actualizar_video)
            self.thread.daemon = True
            self.thread.start()

            self.thread_anunciar = Thread(target=self.anunciar_objetos_detectados)
            self.thread_anunciar.daemon = True
            self.thread_anunciar.start()

    def actualizar_video(self):
        while self.camara_activa:
            ret, frame = self.cap.read()
            if not ret:
                break

            objetos_detectados = set()  # Para almacenar objetos detectados

            if self.detectando_objetos:
                # Usar el modelo YOLOv8 para la detección
                results = self.model(frame)
                detecciones = results[0].boxes  # Obtener las detecciones

                for box in detecciones:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])  # Coordenadas de la caja
                    conf = box.conf[0]  # Confianza
                    cls = int(box.cls[0])  # Clase

                    label = self.clases[cls]  # Obtener el nombre de la clase
                    objetos_detectados.add(label)  # Agregar a la lista de objetos detectados

                    # Dibujar la caja en la imagen
                    color = (0, 255, 0)  # Verde por defecto
                    if label in ["cuchillo", "pistola"]:  # Cambiar a rojo si es un objeto peligroso
                        color = (255, 0, 0)  # Rojo para objetos peligrosos
                        current_time = time.time()
                        if self.alarma_activar and (current_time - self.last_alert_time > 3):
                            self.reproducir_alerta()
                            self.last_alert_time = current_time
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                    cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Actualizar el historial de objetos detectados
            if objetos_detectados:
                self.agregar_historial(f"Objetos detectados: {', '.join(objetos_detectados)}")
                self.objetos_para_anunciar.update(objetos_detectados)

            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_pil = Image.fromarray(frame_rgb)
            frame_tk = ImageTk.PhotoImage(frame_pil)
            self.canvas.create_image(0, 0, anchor=tk.NW, image=frame_tk)
            self.canvas.image = frame_tk

    def anunciar_objetos_detectados(self):
        while True:
            if self.objetos_para_anunciar and self.voz_activar:
                objetos = ", ".join(self.objetos_para_anunciar)
                self.engine.say(f"Objetos detectados: {objetos}")
                self.engine.runAndWait()
                self.objetos_para_anunciar.clear()
            time.sleep(1)

    def cerrar_camara(self):
        if self.camara_activa:
            self.camara_activa = False
            
            # Detener el hilo de video
            if self.thread is not None:
                self.thread.join(timeout=1)  # Espera un segundo para que el hilo termine
            
            # Liberar la cámara
            if self.cap is not None:
                self.cap.release()
            
            # Actualizar el estado de los botones
            self.boton_abrir.config(state=tk.NORMAL)
            self.boton_cerrar.config(state=tk.DISABLED)
            self.boton_foto.config(state=tk.DISABLED)
            self.boton_video.config(state=tk.DISABLED)
            self.boton_deteccion.config(state=tk.DISABLED)
            self.boton_voz.config(state=tk.DISABLED)
            self.boton_alarma.config(state=tk.DISABLED)
            self.boton_descargar.config(state=tk.DISABLED)

            self.agregar_historial("Cámara cerrada")

    def tomar_foto(self):
        if self.camara_activa:
            ret, frame = self.cap.read()
            if ret:
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                photo_filename = f"foto_{timestamp}.jpg"
                cv2.imwrite(photo_filename, frame)
                self.agregar_historial("Foto tomada")
                messagebox.showinfo("Foto tomada", f"La foto se guardó como {photo_filename}.")

    def grabar_video(self):
        if self.camara_activa:
            self.grabando_video = not self.grabando_video

            if self.grabando_video:
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                video_filename = f"video_{timestamp}.avi"
                self.video_writer = cv2.VideoWriter(video_filename, cv2.VideoWriter_fourcc(*'XVID'), 20.0, (640, 480))
                self.boton_video.config(text="Detener Video")
                self.agregar_historial("Grabación de video iniciada")
            else:
                self.video_writer.release()
                self.boton_video.config(text="Grabar Video")
                self.agregar_historial("Grabación de video detenida")
                messagebox.showinfo("Grabación detenida", "La grabación de video ha sido detenida.")

    def toggle_deteccion(self):
        self.detectando_objetos = not self.detectando_objetos
        estado = "activada" if self.detectando_objetos else "desactivada"
        self.agregar_historial(f"Detección de objetos {estado}")

    def toggle_voz(self):
        self.voz_activar = not self.voz_activar
        estado = "activada" if self.voz_activar else "desactivada"
        self.boton_voz.config(text=f"{estado.capitalize()} Voz")
        self.agregar_historial(f"Voz {estado}")

    def toggle_alarma(self):
        self.alarma_activar = not self.alarma_activar
        estado = "activada" if self.alarma_activar else "desactivada"
        self.boton_alarma.config(text=f"{estado.capitalize()} Alarma")
        self.agregar_historial(f"Alarma {estado}")

    def agregar_historial(self, mensaje):
        timestamp = datetime.datetime.now().strftime("%Y%m%d %H:%M:%S")
        self.historial_detecciones.append(f"{timestamp}: {mensaje}")
        if len(self.historial_detecciones) > 50:  # Limitar el tamaño del historial
            self.historial_detecciones.pop(0)
        
        # Actualizar el historial en el Text
        self.text_historial.config(state=tk.NORMAL)  # Habilitar el Text para la edición
        self.text_historial.delete(1.0, tk.END)  # Limpiar el Text
        self.text_historial.insert(tk.END, "\n".join(self.historial_detecciones))  # Agregar nuevo texto
        self.text_historial.config(state=tk.DISABLED)  # Deshabilitar el Text para evitar ediciones
        self.text_historial.yview_moveto(1)  # Desplazar hacia abajo

    def reproducir_alerta(self):
        try:
            pygame.mixer.music.load("alerta.mp3")
            pygame.mixer.music.play()
        except Exception as e:
            print(f"Error al reproducir el sonido: {e}")

    def descargar_historial(self):
        archivo = filedialog.asksaveasfile(defaultextension=".txt", filetypes=[("Archivos de texto", "*.txt")])
        if archivo:
            for entrada in self.historial_detecciones:
                archivo.write(entrada + "\n")
            archivo.close()
            messagebox.showinfo("Éxito", "Historial descargado correctamente.")

    def salir(self):
        self.cerrar_camara()  # Asegurarse de que se cierre la cámara
        self.root.quit()  # Cerrar la aplicación

# Crear ventana principal
root = tk.Tk()
root.geometry("1200x1100")  # Establecer un tamaño fijo para la ventana
root.resizable(False, False)  # Deshabilitar la opción de maximizar
app = CamaraApp(root)
root.mainloop()


0: 480x640 1 person, 199.5ms
Speed: 13.1ms preprocess, 199.5ms inference, 17.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 74.1ms
Speed: 4.0ms preprocess, 74.1ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 1 cell phone, 105.3ms
Speed: 3.5ms preprocess, 105.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 1 cell phone, 74.9ms
Speed: 3.0ms preprocess, 74.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 1 cell phone, 75.2ms
Speed: 3.0ms preprocess, 75.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 1 cell phone, 91.2ms
Speed: 3.5ms preprocess, 91.2ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 cell phone, 79.2ms
Speed: 2.0ms preprocess, 79.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 1 cell phone,

In [ ]:
#red neuronal

In [2]:
import tkinter as tk
from tkinter import messagebox, Scrollbar, Frame, filedialog
from threading import Thread
import cv2
import datetime
import numpy as np
from PIL import Image, ImageTk
import pyttsx3
import os
import time
import pygame

class CamaraApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Detector de imágenes - Demo")
        self.root.config(bg="#2e2e2e")

        pygame.mixer.init()

        self.camara_activa = False
        self.cap = None
        self.thread = None
        self.grabando_video = False
        self.video_writer = None
        self.detectando_objetos = False
        self.engine = pyttsx3.init()
        self.historial_detecciones = []
        self.objetos_para_anunciar = set()
        self.last_alert_time = 0
        self.voz_activar = True
        self.alarma_activar = True

        # Cargar modelo de YOLO
        self.net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
        self.layer_names = self.net.getLayerNames()
        self.output_layers = [self.layer_names[i - 1] for i in self.net.getUnconnectedOutLayers()]
        self.classes = []
        with open("coco.names", "r") as f:
            self.classes = [line.strip() for line in f.readlines()]

        # Establecer la precisión de la red neuronal
        self.precision = 0.85  # Valor de ejemplo, ajusta según tu modelo

        self.frame_principal = tk.Frame(root, bg="#2e2e2e")
        self.frame_principal.pack(fill=tk.BOTH, expand=True)

        self.frame_video = tk.Frame(self.frame_principal, bg="#2e2e2e")
        self.frame_video.pack(side=tk.LEFT, padx=10, pady=10)

        self.frame_historial = Frame(self.frame_principal)
        self.frame_historial.pack(side=tk.RIGHT, fill=tk.Y, padx=10, pady=10)

        self.scrollbar = Scrollbar(self.frame_historial)
        self.scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        self.label_historial_titulo = tk.Label(self.frame_historial, text="Historial", bg="#2e2e2e",
                                               fg="white", font=("Arial", 14, "bold"))
        self.label_historial_titulo.pack(pady=10)

        self.text_historial = tk.Text(self.frame_historial, bg="#2e2e2e", fg="white", font=("Arial", 12),
                                       wrap="word", height=20, width=40)
        self.text_historial.pack(fill=tk.BOTH, expand=True, padx=10)
        self.text_historial.config(yscrollcommand=self.scrollbar.set)
        self.scrollbar.config(command=self.text_historial.yview)

        # Etiqueta para mostrar la precisión
        self.label_precision = tk.Label(self.frame_historial, text=f"Precisión: {self.precision:.2f}", bg="#2e2e2e",
                                         fg="white", font=("Arial", 12))
        self.label_precision.pack(pady=10)

        self.logo_imagen_tk = None
        self.cargar_logo()

        self.frame_botones = tk.Frame(self.frame_video, bg="#3c3c3c")
        self.frame_botones.pack(pady=10, padx=10, fill=tk.X)

        self.boton_abrir = tk.Button(self.frame_botones, text="Abrir Cámara", command=self.abrir_camara, width=20,
                                     bg="#4CAF50", fg="white", font=("Arial", 12))
        self.boton_abrir.grid(row=0, column=0, padx=10, pady=10)

        self.boton_cerrar = tk.Button(self.frame_botones, text="Cerrar Cámara", command=self.cerrar_camara, width=20,
                                      state=tk.DISABLED, bg="#F44336", fg="white", font=("Arial", 12))
        self.boton_cerrar.grid(row=0, column=1, padx=10, pady=10)

        self.boton_foto = tk.Button(self.frame_botones, text="Tomar Foto", command=self.tomar_foto, width=20,
                                    state=tk.DISABLED, bg="#2196F3", fg="white", font=("Arial", 12))
        self.boton_foto.grid(row=1, column=0, padx=10, pady=10)

        self.boton_video = tk.Button(self.frame_botones, text="Grabar Video", command=self.grabar_video, width=20,
                                     state=tk.DISABLED, bg="#FF9800", fg="white", font=("Arial", 12))
        self.boton_video.grid(row=1, column=1, padx=10, pady=10)

        self.boton_deteccion = tk.Button(self.frame_botones, text="Detectar Objetos", command=self.toggle_deteccion,
                                         width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_deteccion.grid(row=2, column=0, padx=10, pady=10)

        self.boton_voz = tk.Button(self.frame_botones, text="Activar Voz", command=self.toggle_voz,
                                   width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_voz.grid(row=2, column=1, padx=10, pady=10)

        self.boton_alarma = tk.Button(self.frame_botones, text="Activar Alarma", command=self.toggle_alarma,
                                      width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_alarma.grid(row=3, column=0, padx=10, pady=10)

        self.boton_descargar = tk.Button(self.frame_botones, text="Descargar Historial", command=self.descargar_historial,
                                          width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_descargar.grid(row=3, column=1, padx=10, pady=10)

        self.boton_salir = tk.Button(self.frame_botones, text="Salir", command=self.salir, width=20,
                                     bg="#9E9E9E", fg="white", font=("Arial", 12))
        self.boton_salir.grid(row=4, column=0, columnspan=2, padx=10, pady=10)

        self.canvas = tk.Canvas(self.frame_video, width=640, height=480)
        self.canvas.pack()

        self.mostrar_tutorial()

    def cargar_logo(self):
        try:
            logo_path = "Logo1.png"
            if os.path.exists(logo_path):
                self.logo_imagen = Image.open(logo_path).resize((300, 100), Image.Resampling.LANCZOS)
                self.logo_imagen_tk = ImageTk.PhotoImage(self.logo_imagen)
                self.label_logo = tk.Label(self.frame_video, image=self.logo_imagen_tk, bg="#2e2e2e")
                self.label_logo.pack(pady=20)
            else:
                raise FileNotFoundError(f"El archivo {logo_path} no se encuentra.")
        except Exception as e:
            print(f"Error al cargar la imagen: {e}")

    def mostrar_tutorial(self):
        self.tutorial = tk.Toplevel(self.root)
        self.tutorial.title("Tutorial")
        self.tutorial.geometry("800x300")
        self.tutorial.config(bg="#2e2e2e")

        tutorial_text = """
        Bienvenido a la aplicación de detección de imágenes.

        1. Abrir Cámara: Inicia la cámara para la detección.
        2. Detectar Objetos: Activa la detección de objetos en la cámara.
        3. Tomar Foto: Captura una imagen de la cámara.
        4. Grabar Video: Inicia o detiene la grabación de video.
        5. Descargar Historial: Guarda el historial de detecciones en un archivo.
        6. Activar Voz: Anuncia los objetos detectados.
        7. Activar Alarma: Reproduce una alarma si se detectan objetos peligrosos.
        8. Salir: Cierra la aplicación.

        Presiona 'Saltar Tutorial' para omitir este tutorial.
        """
        
        label_tutorial = tk.Label(self.tutorial, text=tutorial_text, bg="#2e2e2e", fg="white", font=("Arial", 12))
        label_tutorial.pack(pady=10)

        boton_saltar = tk.Button(self.tutorial, text="Saltar Tutorial", command=self.saltar_tutorial,
                                  bg="#FFC107", fg="black", font=("Arial", 12))
        boton_saltar.pack(pady=20)

    def saltar_tutorial(self):
        self.tutorial.destroy()

    def abrir_camara(self):
        if not self.camara_activa:
            self.cap = cv2.VideoCapture(0)
            if not self.cap.isOpened():
                messagebox.showerror("Error", "No se pudo abrir la cámara.")
                return

            self.camara_activa = True
            self.boton_abrir.config(state=tk.DISABLED)
            self.boton_cerrar.config(state=tk.NORMAL)
            self.boton_foto.config(state=tk.NORMAL)
            self.boton_video.config(state=tk.NORMAL)
            self.boton_deteccion.config(state=tk.NORMAL)
            self.boton_voz.config(state=tk.NORMAL)
            self.boton_alarma.config(state=tk.NORMAL)
            self.boton_descargar.config(state=tk.NORMAL)

            self.agregar_historial("Cámara abierta")

            self.thread = Thread(target=self.actualizar_video)
            self.thread.daemon = True
            self.thread.start()

            self.thread_anunciar = Thread(target=self.anunciar_objetos_detectados)
            self.thread_anunciar.daemon = True
            self.thread_anunciar.start()

    def actualizar_video(self):
        while self.camara_activa:
            ret, frame = self.cap.read()
            if not ret:
                break

            objetos_detectados = set()

            if self.detectando_objetos:
                blob = cv2.dnn.blobFromImage(frame, 0.00392, (320, 320), (0, 0, 0), True, crop=False)
                self.net.setInput(blob)
                outs = self.net.forward(self.output_layers)

                class_ids = []
                confidences = []
                boxes = []

                for out in outs:
                    for detection in out:
                        scores = detection[5:]
                        class_id = np.argmax(scores)
                        confidence = scores[class_id]
                        if confidence > 0.6:
                            center_x = int(detection[0] * frame.shape[1])
                            center_y = int(detection[1] * frame.shape[0])
                            w = int(detection[2] * frame.shape[1])
                            h = int(detection[3] * frame.shape[0])
                            x = int(center_x - w / 2)
                            y = int(center_y - h / 2)
                            boxes.append([x, y, w, h])
                            confidences.append(float(confidence))
                            class_ids.append(class_id)

                indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.6, 0.4)

                if len(indexes) > 0:
                    for i in indexes.flatten():
                        x, y, w, h = boxes[i]
                        label = str(self.classes[class_ids[i]])
                        objetos_detectados.add(label)
                        color = (0, 255, 0)
                        if label in ["knife", "pistol"]:
                            color = (255, 0, 0)
                            current_time = time.time()
                            if self.alarma_activar and (current_time - self.last_alert_time > 3):
                                self.reproducir_alerta()
                                self.last_alert_time = current_time
                        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            if objetos_detectados:
                self.agregar_historial(f"Objetos detectados: {', '.join(objetos_detectados)}")
                self.objetos_para_anunciar.update(objetos_detectados)

            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_pil = Image.fromarray(frame_rgb)
            frame_tk = ImageTk.PhotoImage(frame_pil)
            self.canvas.create_image(0, 0, anchor=tk.NW, image=frame_tk)
            self.canvas.image = frame_tk

    def anunciar_objetos_detectados(self):
        while True:
            if self.objetos_para_anunciar and self.voz_activar:
                objetos = ", ".join(self.objetos_para_anunciar)
                self.engine.say(f"Objetos detectados: {objetos}")
                self.engine.runAndWait()
                self.objetos_para_anunciar.clear()
            time.sleep(1)

    def cerrar_camara(self):
        if self.camara_activa:
            self.camara_activa = False
            
            if self.thread is not None:
                self.thread.join(timeout=1)
            
            if self.cap is not None:
                self.cap.release()
            
            self.boton_abrir.config(state=tk.NORMAL)
            self.boton_cerrar.config(state=tk.DISABLED)
            self.boton_foto.config(state=tk.DISABLED)
            self.boton_video.config(state=tk.DISABLED)
            self.boton_deteccion.config(state=tk.DISABLED)
            self.boton_voz.config(state=tk.DISABLED)
            self.boton_alarma.config(state=tk.DISABLED)
            self.boton_descargar.config(state=tk.DISABLED)

            self.agregar_historial("Cámara cerrada")

    def tomar_foto(self):
        if self.camara_activa:
            ret, frame = self.cap.read()
            if ret:
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                photo_filename = f"foto_{timestamp}.jpg"
                cv2.imwrite(photo_filename, frame)
                self.agregar_historial("Foto tomada")
                messagebox.showinfo("Foto tomada", f"La foto se guardó como {photo_filename}.")

    def grabar_video(self):
        if self.camara_activa:
            self.grabando_video = not self.grabando_video

            if self.grabando_video:
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                video_filename = f"video_{timestamp}.avi"
                self.video_writer = cv2.VideoWriter(video_filename, cv2.VideoWriter_fourcc(*'XVID'), 20.0, (640, 480))
                self.boton_video.config(text="Detener Video")
                self.agregar_historial("Grabación de video iniciada")
            else:
                self.video_writer.release()
                self.boton_video.config(text="Grabar Video")
                self.agregar_historial("Grabación de video detenida")
                messagebox.showinfo("Grabación detenida", "La grabación de video ha sido detenida.")

    def toggle_deteccion(self):
        self.detectando_objetos = not self.detectando_objetos
        estado = "activada" if self.detectando_objetos else "desactivada"
        self.agregar_historial(f"Detección de objetos {estado}")

    def toggle_voz(self):
        self.voz_activar = not self.voz_activar
        estado = "activada" if self.voz_activar else "desactivada"
        self.boton_voz.config(text=f"{estado.capitalize()} Voz")
        self.agregar_historial(f"Voz {estado}")

    def toggle_alarma(self):
        self.alarma_activar = not self.alarma_activar
        estado = "activada" if self.alarma_activar else "desactivada"
        self.boton_alarma.config(text=f"{estado.capitalize()} Alarma")
        self.agregar_historial(f"Alarma {estado}")

    def agregar_historial(self, mensaje):
        timestamp = datetime.datetime.now().strftime("%Y%m%d %H:%M:%S")
        self.historial_detecciones.append(f"{timestamp}: {mensaje}")
        if len(self.historial_detecciones) > 50:
            self.historial_detecciones.pop(0)
        
        self.text_historial.config(state=tk.NORMAL)
        self.text_historial.delete(1.0, tk.END)
        self.text_historial.insert(tk.END, "\n".join(self.historial_detecciones))
        self.text_historial.config(state=tk.DISABLED)
        self.text_historial.yview_moveto(1)

    def reproducir_alerta(self):
        try:
            pygame.mixer.music.load("alerta.mp3")
            pygame.mixer.music.play()
        except Exception as e:
            print(f"Error al reproducir el sonido: {e}")

    def descargar_historial(self):
        archivo = filedialog.asksaveasfile(defaultextension=".txt", filetypes=[("Text files", "*.txt")])
        if archivo:
            for entrada in self.historial_detecciones:
                archivo.write(entrada + "\n")
            archivo.close()
            messagebox.showinfo("Éxito", "Historial descargado correctamente.")

    def salir(self):
        self.cerrar_camara()
        self.root.quit()

# Crear ventana principal
root = tk.Tk()
root.geometry("1200x1100")
root.resizable(False, False)
app = CamaraApp(root)
root.mainloop()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [8]:
import tkinter as tk
from tkinter import messagebox, Scrollbar, Frame, filedialog
from threading import Thread
import cv2
import datetime
import numpy as np
from PIL import Image, ImageTk
import pyttsx3
import os
import time
import pygame

class CamaraApp:
    def __init__(self, root):
        self.root = root
        self.setup_ui()
        self.initialize_variables()
        self.load_model()
        self.load_logo()
        self.show_tutorial()

    def setup_ui(self):
        self.root.title("Detector de imágenes - Demo")
        self.root.config(bg="#2e2e2e")
        pygame.mixer.init()

        self.frame_principal = tk.Frame(self.root, bg="#2e2e2e")
        self.frame_principal.pack(fill=tk.BOTH, expand=True)

        self.frame_video = tk.Frame(self.frame_principal, bg="#2e2e2e")
        self.frame_video.pack(side=tk.LEFT, padx=10, pady=10)

        self.frame_historial = Frame(self.frame_principal)
        self.frame_historial.pack(side=tk.RIGHT, fill=tk.Y, padx=10, pady=10)

        self.scrollbar = Scrollbar(self.frame_historial)
        self.scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        self.label_historial_titulo = tk.Label(self.frame_historial, text="Historial", bg="#2e2e2e",
                                               fg="white", font=("Arial", 14, "bold"))
        self.label_historial_titulo.pack(pady=10)

        self.text_historial = tk.Text(self.frame_historial, bg="#2e2e2e", fg="white", font=("Arial", 12),
                                       wrap="word", height=20, width=40)
        self.text_historial.pack(fill=tk.BOTH, expand=True, padx=10)
        self.text_historial.config(yscrollcommand=self.scrollbar.set)
        self.scrollbar.config(command=self.text_historial.yview)

        self.frame_botones = tk.Frame(self.frame_video, bg="#3c3c3c")
        self.frame_botones.pack(pady=10, padx=10, fill=tk.X)

        self.create_buttons()
        self.canvas = tk.Canvas(self.frame_video, width=640, height=480)
        self.canvas.pack()

    def create_buttons(self):
    button_configurations = [
        ("Abrir Cámara", self.abrir_camara, "4CAF50"),
        ("Cerrar Cámara", self.cerrar_camara, "F44336", tk.DISABLED),
        ("Tomar Foto", self.tomar_foto, "2196F3", tk.DISABLED),
        ("Grabar Video", self.grabar_video, "FF9800", tk.DISABLED),
        ("Detectar Objetos", self.toggle_deteccion, "9C27B0", tk.DISABLED),
        ("Activar Voz", self.toggle_voz, "9C27B0", tk.DISABLED),
        ("Activar Alarma", self.toggle_alarma, "9C27B0", tk.DISABLED),
        ("Descargar Historial", self.descargar_historial, "9C27B0", tk.DISABLED),
        ("Salir", self.salir, "9E9E9E")
    ]

    for idx, config in enumerate(button_configurations):
        text, command, color = config[:3]
        state = config[3] if len(config) > 3 else tk.NORMAL
        button = tk.Button(self.frame_botones, text=text, command=command, width=20,
                           bg=f"#{color}", fg="white", font=("Arial", 12), state=state)
        button.grid(row=idx // 2, column=idx % 2, padx=10, pady=10)

    def initialize_variables(self):
        self.camara_activa = False
        self.cap = None
        self.thread = None
        self.grabando_video = False
        self.video_writer = None
        self.detectando_objetos = False
        self.engine = pyttsx3.init()
        self.historial_detecciones = []
        self.objetos_para_anunciar = set()
        self.last_alert_time = 0
        self.voz_activar = True
        self.alarma_activar = True

    def load_model(self):
        self.net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
        self.layer_names = self.net.getLayerNames()
        self.output_layers = [self.layer_names[i - 1] for i in self.net.getUnconnectedOutLayers()]
        with open("coco.names", "r") as f:
            self.classes = [line.strip() for line in f.readlines()]

    def load_logo(self):
        logo_path = "Logo1.png"
        if os.path.exists(logo_path):
            self.logo_imagen = Image.open(logo_path).resize((300, 100), Image.Resampling.LANCZOS)
            self.logo_imagen_tk = ImageTk.PhotoImage(self.logo_imagen)
            self.label_logo = tk.Label(self.frame_video, image=self.logo_imagen_tk, bg="#2e2e2e")
            self.label_logo.pack(pady=20)
        else:
            print(f"El archivo {logo_path} no se encuentra.")

    def show_tutorial(self):
        self.tutorial = tk.Toplevel(self.root)
        self.tutorial.title("Tutorial")
        self.tutorial.geometry("800x450")
        self.tutorial.config(bg="#2e2e2e")

        tutorial_text = """
        Bienvenido a la aplicación de detección de imágenes.

        1. Abrir Cámara: Iniciara la cámara para la detección.
        2. Detectar Objetos: Activara la detección de objetos en la cámara.
        3. Tomar Foto: Captura una imagen de la cámara.
        4. Grabar Video: Iniciara o detendr la grabación de video.
        5. Descargar Historial: Guardara el historial de detecciones en un archivo.
        6. Activar Voz: Anunciara los objetos detectados y si esta activado desactivara la voz.
        7. Activar Alarma: Reproducira una alarma si se detectan objetos peligrosos.
        8. Salir: Cierra la aplicación.

        Presiona 'Saltar Tutorial' para omitir este tutorial.
        """

        label_tutorial = tk.Label(self.tutorial, text=tutorial_text, bg="#2e2e2e", fg="white", font=("Arial", 12))
        label_tutorial.pack(pady=10)

        boton_saltar = tk.Button(self.tutorial, text="Saltar Tutorial", command=self.saltar_tutorial,
                                  bg="#FFC107", fg="black", font=("Arial", 12))
        boton_saltar.pack(pady=20)

    def saltar_tutorial(self):
        self.tutorial.destroy()

    def abrir_camara(self):
        if not self.camara_activa:
            self.cap = cv2.VideoCapture(0)
            if not self.cap.isOpened():
                self.cap = cv2.VideoCapture(1)

            if not self.cap.isOpened():
                messagebox.showerror("Error", "No se pudo abrir la cámara.")
                return

            self.camara_activa = True
            self.update_button_states()
            self.agregar_historial("Cámara abierta")

            self.thread = Thread(target=self.actualizar_video)
            self.thread.daemon = True
            self.thread.start()

            self.thread_anunciar = Thread(target=self.anunciar_objetos_detectados)
            self.thread_anunciar.daemon = True
            self.thread_anunciar.start()

    def update_button_states(self):
        states = {
            "abrir": tk.DISABLED,
            "cerrar": tk.NORMAL,
            "foto": tk.NORMAL,
            "video": tk.NORMAL,
            "deteccion": tk.NORMAL,
            "voz": tk.NORMAL,
            "alarma": tk.NORMAL,
            "descargar": tk.NORMAL,
        }
        self.boton_abrir.config(state=states["abrir"])
        self.boton_cerrar.config(state=states["cerrar"])
        self.boton_foto.config(state=states["foto"])
        self.boton_video.config(state=states["video"])
        self.boton_deteccion.config(state=states["deteccion"])
        self.boton_voz.config(state=states["voz"])
        self.boton_alarma.config(state=states["alarma"])
        self.boton_descargar.config(state=states["descargar"])

    def actualizar_video(self):
        while self.camara_activa:
            ret, frame = self.cap.read()
            if not ret:
                break

            objetos_detectados = set()
            if self.detectando_objetos:
                self.detect_objects(frame, objetos_detectados)

            if objetos_detectados:
                self.agregar_historial(f"Objetos detectados: {', '.join(objetos_detectados)}")
                self.objetos_para_anunciar.update(objetos_detectados)

            self.display_frame(frame)

    def detect_objects(self, frame, objetos_detectados):
        blob = cv2.dnn.blobFromImage(frame, 0.00392, (320, 320), (0, 0, 0), True, crop=False)
        self.net.setInput(blob)
        outs = self.net.forward(self.output_layers)

        class_ids, confidences, boxes = [], [], []
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.6:
                    center_x = int(detection[0] * frame.shape[1])
                    center_y = int(detection[1] * frame.shape[0])
                    w = int(detection[2] * frame.shape[1])
                    h = int(detection[3] * frame.shape[0])
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)
                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.6, 0.4)
        self.draw_boxes(frame, indexes, boxes, class_ids, objetos_detectados)

    def draw_boxes(self, frame, indexes, boxes, class_ids, objetos_detectados):
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            label = str(self.classes[class_ids[i]])
            objetos_detectados.add(label)
            color = (0, 255, 0) if label not in ["knife", "pistol"] else (255, 0, 0)
            if color == (255, 0, 0):
                self.check_alarm()
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    def check_alarm(self):
        current_time = time.time()
        if self.alarma_activar and (current_time - self.last_alert_time > 3):
            self.reproducir_alerta()
            self.last_alert_time = current_time

    def display_frame(self, frame):
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_pil = Image.fromarray(frame_rgb)
        frame_tk = ImageTk.PhotoImage(frame_pil)
        self.canvas.create_image(0, 0, anchor=tk.NW, image=frame_tk)
        self.canvas.image = frame_tk

    def anunciar_objetos_detectados(self):
        while True:
            if self.objetos_para_anunciar and self.voz_activar:
                objetos = ", ".join(self.objetos_para_anunciar)
                self.engine.say(f"Objetos detectados: {objetos}")
                self.engine.runAndWait()
                self.objetos_para_anunciar.clear()
            time.sleep(1)

    def cerrar_camara(self):
        if self.camara_activa:
            self.camara_activa = False
            if self.thread is not None:
                self.thread.join(timeout=1)
            if self.cap is not None:
                self.cap.release()
            self.update_button_states()
            self.agregar_historial("Cámara cerrada")

    def tomar_foto(self):
        if self.camara_activa:
            ret, frame = self.cap.read()
            if ret:
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                photo_filename = f"foto_{timestamp}.jpg"
                cv2.imwrite(photo_filename, frame)
                self.agregar_historial("Foto tomada")
                messagebox.showinfo("Foto tomada", f"La foto se guardó como {photo_filename}.")

    def grabar_video(self):
        if self.camara_activa:
            self.grabando_video = not self.grabando_video

            if self.grabando_video:
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                video_filename = f"video_{timestamp}.avi"
                self.video_writer = cv2.VideoWriter(video_filename, cv2.VideoWriter_fourcc(*'XVID'), 20.0, (640, 480))
                self.boton_video.config(text="Detener Video")
                self.agregar_historial("Grabación de video iniciada")
            else:
                self.video_writer.release()
                self.boton_video.config(text="Grabar Video")
                self.agregar_historial("Grabación de video detenida")
                messagebox.showinfo("Grabación detenida", "La grabación de video ha sido detenida.")

    def toggle_deteccion(self):
        self.detectando_objetos = not self.detectando_objetos
        estado = "activada" if self.detectando_objetos else "desactivada"
        self.agregar_historial(f"Detección de objetos {estado}")

    def toggle_voz(self):
        self.voz_activar = not self.voz_activar
        estado = "activada" if self.voz_activar else "desactivada"
        self.boton_voz.config(text=f"{estado.capitalize()} Voz")
        self.agregar_historial(f"Voz {estado}")

    def toggle_alarma(self):
        self.alarma_activar = not self.alarma_activar
        estado = "activada" if self.alarma_activar else "desactivada"
        self.boton_alarma.config(text=f"{estado.capitalize()} Alarma")
        self.agregar_historial(f"Alarma {estado}")

    def agregar_historial(self, mensaje):
        timestamp = datetime.datetime.now().strftime("%Y%m%d %H:%M:%S")
        self.historial_detecciones.append(f"{timestamp}: {mensaje}")
        if len(self.historial_detecciones) > 50:
            self.historial_detecciones.pop(0)
        
        # Actualizar el historial en el Text
        self.text_historial.config(state=tk.NORMAL)
        self.text_historial.delete(1.0, tk.END)
        self.text_historial.insert(tk.END, "\n".join(self.historial_detecciones))
        self.text_historial.config(state=tk.DISABLED)
        self.text_historial.yview_moveto(1)

    def reproducir_alerta(self):
        try:
            pygame.mixer.music.load("alerta.mp3")
            pygame.mixer.music.play()
        except Exception as e:
            print(f"Error al reproducir el sonido: {e}")

    def descargar_historial(self):
        archivo = filedialog.asksaveasfile(defaultextension=".txt", filetypes=[("Text files", "*.txt")])
        if archivo:
            with open(archivo.name, "w") as f:
                for entrada in self.historial_detecciones:
                    f.write(entrada + "\n")
            messagebox.showinfo("Éxito", "Historial descargado de manera correcta.")

    def salir(self):
        self.cerrar_camara()
        self.root.quit()

# Crear ventana principal
root = tk.Tk()
root.geometry("1200x1100")
root.resizable(False, False)
app = CamaraApp(root)
root.mainloop()

IndentationError: expected an indented block after function definition on line 56 (3021758568.py, line 57)

In [ ]:
"gun", "knife", "bat" , "machete" ,"stun gun" ,"pepper spray" , "reflective vest" , "first aid kit" , "security badge", "handcuffs" , "fire" , "smoke" , "earthquake" , "flood" , "tornado" , "urricane" , "avalanche", "landslide", "tsunami", "explosion"

In [2]:
#cambios

In [ ]:
import tkinter as tk
from tkinter import messagebox, Scrollbar, Frame, filedialog
from threading import Thread
import cv2
import numpy as np
from PIL import Image, ImageTk
import pyttsx3
import os
import time
import pygame
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
import random
from datetime import datetime
from queue import Queue

class CamaraApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Detector de imágenes - Demo")
        self.root.config(bg="#2e2e2e")

        pygame.mixer.init()

        self.camara_activa = False
        self.cap = None
        self.detectando_objetos = False
        self.engine = pyttsx3.init()
        self.historial_detecciones = []
        self.objetos_para_anunciar = set()
        self.predicciones = []
        self.last_alert_time = 0
        self.voz_activar = True
        self.alarma_activar = True
        self.camara_index = 0
        self.voice_queue = Queue()
        self.thread_voice = Thread(target=self.process_voice_queue)
        self.thread_voice.daemon = True
        self.thread_voice.start()

        self.modelo_entrenado = False
        self.entrenar_modelo_ejemplo()

        self.net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
        self.layer_names = self.net.getLayerNames()
        self.output_layers = [self.layer_names[i - 1] for i in self.net.getUnconnectedOutLayers()]
        self.classes = []
        with open("coco.names", "r") as f:
            self.classes = [line.strip() for line in f.readlines()]

        self.mostrar_tutorial()
        self.setup_ui()

    def mostrar_tutorial(self):
        tutorial_window = tk.Toplevel(self.root)
        tutorial_window.title("Tutorial")
        tutorial_window.geometry("400x300")
        tutorial_window.config(bg="#1a1a1a")

        titulo = tk.Label(tutorial_window, text="Bienvenido al Detector de Imágenes", bg="#1a1a1a", fg="white", font=("Arial", 16))
        titulo.pack(pady=10)

        tutorial_text = (
            "Esta herramienta permite detectar objetos peligrosos en tiempo real.\n\n"
            "1. Abre la cámara.\n"
            "2. Activa la detección de objetos.\n"
            "3. Recibe alertas por voz y visuales.\n"
            "4. Cierra la cámara cuando termines."
        )
        instrucciones = tk.Label(tutorial_window, text=tutorial_text, bg="#1a1a1a", fg="white", font=("Arial", 12))
        instrucciones.pack(pady=20)

        boton_omitir = tk.Button(tutorial_window, text="Cerrar Tutorial", command=tutorial_window.destroy, bg="#4CAF50", fg="white")
        boton_omitir.pack(pady=10)

        # Bloquear la ventana principal hasta que el tutorial se cierre
        tutorial_window.protocol("WM_DELETE_WINDOW", tutorial_window.destroy)

    def setup_ui(self):
        self.frame_principal = tk.Frame(self.root, bg="#2e2e2e")
        self.frame_principal.pack(fill=tk.BOTH, expand=True)

        self.frame_video = tk.Frame(self.frame_principal, bg="#2e2e2e")
        self.frame_video.pack(side=tk.LEFT, padx=10, pady=10)

        self.cargar_logo()

        self.frame_historial = Frame(self.frame_principal)
        self.frame_historial.pack(side=tk.RIGHT, fill=tk.Y, padx=10, pady=10)

        self.scrollbar = Scrollbar(self.frame_historial)
        self.scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        self.label_historial_titulo = tk.Label(self.frame_historial, text="Historial", bg="#2e2e2e",
                                               fg="white", font=("Arial", 14, "bold"))
        self.label_historial_titulo.pack(pady=10)

        self.text_historial = tk.Text(self.frame_historial, bg="#2e2e2e", fg="white", font=("Arial", 12),
                                       wrap="word", height=20, width=40)
        self.text_historial.pack(fill=tk.BOTH, expand=True, padx=10)
        self.text_historial.config(yscrollcommand=self.scrollbar.set)
        self.scrollbar.config(command=self.text_historial.yview)

        self.canvas = tk.Canvas(self.frame_video, width=640, height=480)
        self.canvas.pack()

        self.frame_botones = tk.Frame(self.frame_video, bg="#3c3c3c")
        self.frame_botones.pack(pady=10, padx=10, fill=tk.X)

        self.boton_abrir = tk.Button(self.frame_botones, text="Abrir Cámara", command=self.abrir_camara, width=20,
                                     bg="#4CAF50", fg="white", font=("Arial", 12))
        self.boton_abrir.grid(row=0, column=0, padx=10, pady=10)

        self.boton_cerrar = tk.Button(self.frame_botones, text="Cerrar Cámara", command=self.cerrar_camara, width=20,
                                      state=tk.DISABLED, bg="#F44336", fg="white", font=("Arial", 12))
        self.boton_cerrar.grid(row=0, column=1, padx=10, pady=10)

        self.boton_deteccion = tk.Button(self.frame_botones, text="Detectar Objetos", command=self.toggle_deteccion,
                                         width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_deteccion.grid(row=1, column=0, padx=10, pady=10)

        self.boton_voz = tk.Button(self.frame_botones, text="Activar Voz", command=self.toggle_voz,
                                   width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_voz.grid(row=1, column=1, padx=10, pady=10)

        self.boton_alarma = tk.Button(self.frame_botones, text="Activar Alarma", command=self.toggle_alarma,
                                      width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_alarma.grid(row=2, column=0, padx=10, pady=10)

        self.boton_descargar = tk.Button(self.frame_botones, text="Descargar Historial", command=self.descargar_historial,
                                          width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_descargar.grid(row=2, column=1, padx=10, pady=10)

        self.boton_cambiar_camara = tk.Button(self.frame_botones, text="Cambiar Cámara", command=self.cambiar_camara, width=20,
                                              state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_cambiar_camara.grid(row=3, column=0, padx=10, pady=10)

        self.boton_prediccion = tk.Button(self.frame_botones, text="Realizar Predicción", command=self.mostrar_prediccion,
                                           width=20, state=tk.DISABLED, bg="#009688", fg="white", font=("Arial", 12))
        self.boton_prediccion.grid(row=3, column=1, padx=10, pady=10)

        self.boton_salir = tk.Button(self.frame_botones, text="Salir", command=self.salir, width=20,
                                     bg="#9E9E9E", fg="white", font=("Arial", 12))
        self.boton_salir.grid(row=4, column=0, columnspan=2, padx=10, pady=10)

    def cargar_logo(self):
        try:
            logo_path = "Logo1.png"
            if os.path.exists(logo_path):
                self.logo_imagen = Image.open(logo_path).resize((300, 100), Image.Resampling.LANCZOS)
                self.logo_imagen_tk = ImageTk.PhotoImage(self.logo_imagen)
                self.label_logo = tk.Label(self.frame_video, image=self.logo_imagen_tk, bg="#2e2e2e")
                self.label_logo.pack(pady=20)
        except Exception as e:
            print(f"Error al cargar la imagen: {e}")

    def hablar(self, mensaje):
        if self.voz_activar:  # Solo agregar a la cola si la voz está activada
            self.voice_queue.put(mensaje)

    def process_voice_queue(self):
        while True:
            mensaje = self.voice_queue.get()
            if self.voz_activar:  # Verificar si la voz está activada
                self.engine.say(mensaje)
                self.engine.runAndWait()

    def entrenar_modelo_ejemplo(self):
        X = np.array([[0, 0, 1], [1, 0, 0], [0, 1, 0], [1, 1, 0], [0, 0, 0]])
        y = np.array(['bajo', 'alto', 'medio', 'alto', 'bajo'])
        
        self.le = LabelEncoder()
        y_encoded = self.le.fit_transform(y)
        
        self.modelo = MLPClassifier(hidden_layer_sizes=(10, 5), max_iter=5000, learning_rate_init=0.01)
        self.modelo.fit(X, y_encoded)
        self.modelo_entrenado = True

    def predecir_peligro(self, datos):
        if self.modelo_entrenado:
            prediccion_encoded = self.modelo.predict([datos])
            return self.le.inverse_transform(prediccion_encoded)[0]
        return "Error en el modelo"

    def generar_datos_prediccion(self):
        if len(self.predicciones) == 0:
            return [0, 0, 0]
        
        # Contar solo los objetos peligrosos detectados en la cámara
        peligrosos = sum(1 for obj in self.predicciones if obj['label'] in ["knife", "pistol", "scissors"])
        no_peligrosos = len(self.predicciones) - peligrosos
        return [peligrosos, no_peligrosos, random.randint(0, 1)]

    def mostrar_prediccion(self):
        self.ventana_prediccion = tk.Toplevel(self.root)
        self.ventana_prediccion.title("Análisis de Riesgo")
        self.ventana_prediccion.geometry("400x300")
        self.ventana_prediccion.config(bg="#1a1a1a")

        titulo = tk.Label(self.ventana_prediccion, text="Análisis de Riesgo en Tiempo Real", bg="#1a1a1a", fg="white", font=("Arial", 16))
        titulo.pack(pady=10)

        self.label_resultado = tk.Label(self.ventana_prediccion, text="", bg="#1a1a1a", fg="white", font=("Arial", 14))
        self.label_resultado.pack(pady=10)

        self.label_peligrosos = tk.Label(self.ventana_prediccion, text="Objetos Peligrosos: 0", bg="#1a1a1a", fg="red", font=("Arial", 12))
        self.label_peligrosos.pack(pady=5)

        self.label_seguros = tk.Label(self.ventana_prediccion, text="Objetos Seguros: 0", bg="#1a1a1a", fg="green", font=("Arial", 12))
        self.label_seguros.pack(pady=5)

        self.label_nivel_riesgo = tk.Label(self.ventana_prediccion, text="Nivel de riesgo: BAJO", bg="#1a1a1a", fg="white", font=("Arial", 14))
        self.label_nivel_riesgo.pack(pady=20)

        nota = tk.Label(self.ventana_prediccion, text="* Basado en análisis de objetos detectados y patrones históricos", bg="#1a1a1a", fg="white", font=("Arial", 10))
        nota.pack(pady=5)

        self.thread_actualizar_prediccion = Thread(target=self.actualizar_prediccion_en_ventana)
        self.thread_actualizar_prediccion.daemon = True
        self.thread_actualizar_prediccion.start()

    def actualizar_prediccion_en_ventana(self):
        contador_peligrosos = 0
        contador_seguros = 0

        while self.camara_activa and self.detectando_objetos:
            datos_entrada = self.generar_datos_prediccion()
            resultado = self.predecir_peligro(datos_entrada)

            # Verificar si la ventana aún está abierta
            if not self.ventana_prediccion.winfo_exists():
                break

            self.label_resultado.config(text=f"Datos analizados: {resultado}")

            if resultado == "alto":
                contador_peligrosos += 1
                self.label_nivel_riesgo.config(text="Nivel de riesgo: ALTO")
            else:
                contador_seguros += 1
                self.label_nivel_riesgo.config(text="Nivel de riesgo: BAJO")

            self.label_peligrosos.config(text=f"Objetos Peligrosos: {contador_peligrosos}")
            self.label_seguros.config(text=f"Objetos Seguros: {contador_seguros}")

            time.sleep(1)

    def abrir_camara(self):
        if not self.camara_activa:
            self.cap = cv2.VideoCapture(self.camara_index)
            if not self.cap.isOpened():
                messagebox.showerror("Error", "No se pudo abrir la cámara.")
                return

            self.camara_activa = True
            self.boton_abrir.config(state=tk.DISABLED)
            self.boton_cerrar.config(state=tk.NORMAL)
            self.boton_deteccion.config(state=tk.NORMAL)
            self.boton_voz.config(state=tk.NORMAL)
            self.boton_alarma.config(state=tk.NORMAL)
            self.boton_descargar.config(state=tk.NORMAL)
            self.boton_cambiar_camara.config(state=tk.NORMAL)
            self.boton_prediccion.config(state=tk.NORMAL)

            self.agregar_historial("Cámara abierta")

            self.thread_video = Thread(target=self.actualizar_video)
            self.thread_video.daemon = True
            self.thread_video.start()

    def actualizar_video(self):
        while self.camara_activa:
            ret, frame = self.cap.read()
            if not ret:
                break

            if self.detectando_objetos:
                self.procesar_frame(frame)

            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_pil = Image.fromarray(frame_rgb)
            frame_tk = ImageTk.PhotoImage(frame_pil)
            self.canvas.create_image(0, 0, anchor=tk.NW, image=frame_tk)
            self.canvas.image = frame_tk

    def procesar_frame(self, frame):
        blob = cv2.dnn.blobFromImage(frame, 0.00392, (320, 320), (0, 0, 0), True, crop=False)
        self.net.setInput(blob)
        outs = self.net.forward(self.output_layers)

        class_ids = []
        confidences = []
        boxes = []

        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.6:
                    center_x = int(detection[0] * frame.shape[1])
                    center_y = int(detection[1] * frame.shape[0])
                    w = int(detection[2] * frame.shape[1])
                    h = int(detection[3] * frame.shape[0])
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)
                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.6, 0.4)

        objetos_detectados = []
        for i in indexes:
            x, y, w, h = boxes[i]
            label = str(self.classes[class_ids[i]])
            confidence = confidences[i] * 100
            objetos_detectados.append({'label': label, 'confidence': confidence})
            color = (0, 255, 0)
            if label in ["knife", "pistol", "scissors"]:
                color = (255, 0, 0)
                self.reproducir_alerta()  # Llamar a la función de alerta

            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, f"{label} {confidence:.2f}%", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Anunciar el objeto detectado
            if self.voz_activar and label not in self.objetos_para_anunciar:
                self.hablar(f"Detectado: {label}")
                self.objetos_para_anunciar.add(label)

        self.predicciones = objetos_detectados
        self.agregar_historial(f"Objetos detectados: {', '.join([obj['label'] for obj in objetos_detectados])}")

    def reproducir_alerta(self):
        if self.alarma_activar:
            pygame.mixer.music.load("alerta.mp3")  # Asegúrate de tener un archivo de sonido llamado alerta.mp3
            pygame.mixer.music.play()

    def cerrar_camara(self):
        if self.camara_activa:
            self.camara_activa = False
            
            if self.cap is not None:
                self.cap.release()
            
            self.boton_abrir.config(state=tk.NORMAL)
            self.boton_cerrar.config(state=tk.DISABLED)
            self.boton_deteccion.config(state=tk.DISABLED)
            self.boton_voz.config(state=tk.DISABLED)
            self.boton_alarma.config(state=tk.DISABLED)
            self.boton_descargar.config(state=tk.DISABLED)
            self.boton_cambiar_camara.config(state=tk.DISABLED)

            self.agregar_historial("Cámara cerrada")

    def cambiar_camara(self):
        self.camara_index = 1 if self.camara_index == 0 else 0
        self.cerrar_camara()
        self.abrir_camara()
        self.agregar_historial(f"Cámara cambiada a {'Cámara web externa' if self.camara_index == 0 else 'Cámara de la laptop'}")

    def agregar_historial(self, mensaje):
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        self.historial_detecciones.append(f"{timestamp}: {mensaje}")
        self.text_historial.config(state=tk.NORMAL)
        self.text_historial.insert(tk.END, f"{timestamp}: {mensaje}\n")
        self.text_historial.config(state=tk.DISABLED)

    def toggle_deteccion(self):
        self.detectando_objetos = not self.detectando_objetos
        estado = "activada" if self.detectando_objetos else "desactivada"
        self.agregar_historial(f"Detección de objetos {estado}")

        # Detener la voz si se desactiva la detección
        if not self.detectando_objetos and self.voz_activar:
            self.engine.stop()
            self.objetos_para_anunciar.clear()

    def toggle_voz(self):
        self.voz_activar = not self.voz_activar
        estado = "activada" if self.voz_activar else "desactivada"
        self.agregar_historial(f"Voz {estado}")

        # Cambiar el texto del botón
        self.boton_voz.config(text="Desactivar Voz" if self.voz_activar else "Activar Voz")

    def toggle_alarma(self):
        self.alarma_activar = not self.alarma_activar
        estado = "activada" if self.alarma_activar else "desactivada"
        self.agregar_historial(f"Alarma {estado}")

    def descargar_historial(self):
        if self.historial_detecciones:
            archivo = filedialog.asksaveasfilename(defaultextension=".txt",
                                                    filetypes=[("Archivos de texto", "*.txt")])
            if archivo:
                with open(archivo, "w") as f:
                    for mensaje in self.historial_detecciones:
                        f.write(mensaje + "\n")
                messagebox.showinfo("Descargar Historial", "Historial guardado correctamente.")
        else:
            messagebox.showwarning("Descargar Historial", "No hay historial para guardar.")

    def salir(self):
        self.cerrar_camara()
        self.root.quit()

# Crear ventana principal
root = tk.Tk()
root.geometry("1500x1100")
root.resizable(False, False)
app = CamaraApp(root)
root.mainloop()

In [5]:
#cambios final

In [ ]:
import tkinter as tk
from tkinter import messagebox, Scrollbar, Frame, filedialog
from threading import Thread
import cv2
import numpy as np
from PIL import Image, ImageTk
import pyttsx3
import os
import time
import pygame
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
import random
from datetime import datetime
from queue import Queue

class CamaraApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Detector de imágenes - Demo")
        self.root.config(bg="#2e2e2e")

        pygame.mixer.init()

        self.camara_activa = False
        self.cap = None
        self.detectando_objetos = False
        self.engine = pyttsx3.init()
        self.historial_detecciones = []
        self.objetos_para_anunciar = set()
        self.predicciones = []
        self.last_alert_time = 0
        self.voz_activar = True
        self.alarma_activar = True
        self.camara_index = 0
        self.voice_queue = Queue()
        self.thread_voice = Thread(target=self.process_voice_queue)
        self.thread_voice.daemon = True
        self.thread_voice.start()

        self.modelo_entrenado = False
        self.entrenar_modelo_ejemplo()

        self.net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
        self.layer_names = self.net.getLayerNames()
        self.output_layers = [self.layer_names[i - 1] for i in self.net.getUnconnectedOutLayers()]
        self.classes = []
        with open("coco.names", "r") as f:
            self.classes = [line.strip() for line in f.readlines()]

        self.mostrar_tutorial()
        self.setup_ui()

    def mostrar_tutorial(self):
        tutorial_window = tk.Toplevel(self.root)
        tutorial_window.title("Tutorial")
        tutorial_window.geometry("400x300")
        tutorial_window.config(bg="#1a1a1a")

        titulo = tk.Label(tutorial_window, text="Bienvenido al Detector de Imágenes", bg="#1a1a1a", fg="white", font=("Arial", 16))
        titulo.pack(pady=10)

        tutorial_text = (
            "Esta herramienta permite detectar objetos peligrosos en tiempo real.\n\n"
            "1. Abre la cámara.\n"
            "2. Activa la detección de objetos.\n"
            "3. Recibe alertas por voz y visuales.\n"
            "4. Cierra la cámara cuando termines."
        )
        instrucciones = tk.Label(tutorial_window, text=tutorial_text, bg="#1a1a1a", fg="white", font=("Arial", 12))
        instrucciones.pack(pady=20)

        boton_omitir = tk.Button(tutorial_window, text="Cerrar Tutorial", command=tutorial_window.destroy, bg="#4CAF50", fg="white")
        boton_omitir.pack(pady=10)

        tutorial_window.protocol("WM_DELETE_WINDOW", tutorial_window.destroy)

    def setup_ui(self):
        self.frame_principal = tk.Frame(self.root, bg="#2e2e2e")
        self.frame_principal.pack(fill=tk.BOTH, expand=True)

        self.frame_video = tk.Frame(self.frame_principal, bg="#2e2e2e")
        self.frame_video.pack(side=tk.LEFT, padx=10, pady=10)

        self.cargar_logo()

        self.frame_historial = Frame(self.frame_principal)
        self.frame_historial.pack(side=tk.RIGHT, fill=tk.Y, padx=10, pady=10)

        self.scrollbar = Scrollbar(self.frame_historial)
        self.scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        self.label_historial_titulo = tk.Label(self.frame_historial, text="Historial", bg="#2e2e2e",
                                               fg="white", font=("Arial", 14, "bold"))
        self.label_historial_titulo.pack(pady=10)

        self.text_historial = tk.Text(self.frame_historial, bg="#2e2e2e", fg="white", font=("Arial", 12),
                                       wrap="word", height=20, width=40)
        self.text_historial.pack(fill=tk.BOTH, expand=True, padx=10)
        self.text_historial.config(yscrollcommand=self.scrollbar.set)
        self.scrollbar.config(command=self.text_historial.yview)

        self.canvas = tk.Canvas(self.frame_video, width=640, height=480)
        self.canvas.pack()

        self.frame_botones = tk.Frame(self.frame_video, bg="#3c3c3c")
        self.frame_botones.pack(pady=10, padx=10, fill=tk.X)

        self.boton_abrir = tk.Button(self.frame_botones, text="Abrir Cámara", command=self.abrir_camara, width=20,
                                     bg="#4CAF50", fg="white", font=("Arial", 12))
        self.boton_abrir.grid(row=0, column=0, padx=10, pady=10)

        self.boton_cerrar = tk.Button(self.frame_botones, text="Cerrar Cámara", command=self.cerrar_camara, width=20,
                                      state=tk.DISABLED, bg="#F44336", fg="white", font=("Arial", 12))
        self.boton_cerrar.grid(row=0, column=1, padx=10, pady=10)

        self.boton_deteccion = tk.Button(self.frame_botones, text="Detectar Objetos", command=self.toggle_deteccion,
                                         width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_deteccion.grid(row=1, column=0, padx=10, pady=10)

        self.boton_voz = tk.Button(self.frame_botones, text="Activar Voz", command=self.toggle_voz,
                                   width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_voz.grid(row=1, column=1, padx=10, pady=10)

        self.boton_alarma = tk.Button(self.frame_botones, text="Activar Alarma", command=self.toggle_alarma,
                                      width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_alarma.grid(row=2, column=0, padx=10, pady=10)

        self.boton_descargar = tk.Button(self.frame_botones, text="Descargar Historial", command=self.descargar_historial,
                                          width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_descargar.grid(row=2, column=1, padx=10, pady=10)

        self.boton_cambiar_camara = tk.Button(self.frame_botones, text="Cambiar Cámara", command=self.cambiar_camara, width=20,
                                              state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_cambiar_camara.grid(row=3, column=0, padx=10, pady=10)

        self.boton_prediccion = tk.Button(self.frame_botones, text="Realizar Predicción", command=self.mostrar_prediccion,
                                           width=20, state=tk.DISABLED, bg="#009688", fg="white", font=("Arial", 12))
        self.boton_prediccion.grid(row=3, column=1, padx=10, pady=10)

        self.boton_salir = tk.Button(self.frame_botones, text="Salir", command=self.salir, width=20,
                                     bg="#9E9E9E", fg="white", font=("Arial", 12))
        self.boton_salir.grid(row=4, column=0, columnspan=2, padx=10, pady=10)

    def cargar_logo(self):
        try:
            logo_path = "Logo1.png"
            if os.path.exists(logo_path):
                self.logo_imagen = Image.open(logo_path).resize((300, 100), Image.Resampling.LANCZOS)
                self.logo_imagen_tk = ImageTk.PhotoImage(self.logo_imagen)
                self.label_logo = tk.Label(self.frame_video, image=self.logo_imagen_tk, bg="#2e2e2e")
                self.label_logo.pack(pady=20)
        except Exception as e:
            print(f"Error al cargar la imagen: {e}")

    def hablar(self, mensaje):
        if self.voz_activar:  # Solo agregar a la cola si la voz está activada
            self.voice_queue.put(mensaje)

    def process_voice_queue(self):
        while True:
            mensaje = self.voice_queue.get()
            if self.voz_activar:  # Verificar si la voz está activada
                self.engine.say(mensaje)
                self.engine.runAndWait()

    def entrenar_modelo_ejemplo(self):
        X = np.array([[0, 0, 1], [1, 0, 0], [0, 1, 0], [1, 1, 0], [0, 0, 0]])
        y = np.array(['bajo', 'alto', 'medio', 'alto', 'bajo'])
        
        self.le = LabelEncoder()
        y_encoded = self.le.fit_transform(y)
        
        self.modelo = MLPClassifier(hidden_layer_sizes=(10, 5), max_iter=5000, learning_rate_init=0.01)
        self.modelo.fit(X, y_encoded)
        self.modelo_entrenado = True

    def predecir_peligro(self, datos):
        if self.modelo_entrenado:
            prediccion_encoded = self.modelo.predict([datos])
            return self.le.inverse_transform(prediccion_encoded)[0]
        return "Error en el modelo"

    def generar_datos_prediccion(self):
        if len(self.predicciones) == 0:
            return [0, 0, 0]
        
        # Contar solo los objetos peligrosos detectados en la cámara
        peligrosos = sum(1 for obj in self.predicciones if obj['label'] in ["knife", "pistol", "scissors"])
        no_peligrosos = len(self.predicciones) - peligrosos
        return [peligrosos, no_peligrosos, random.randint(0, 1)]

    def mostrar_prediccion(self):
        self.ventana_prediccion = tk.Toplevel(self.root)
        self.ventana_prediccion.title("Análisis de Riesgo")
        self.ventana_prediccion.geometry("400x300")
        self.ventana_prediccion.config(bg="#1a1a1a")

        titulo = tk.Label(self.ventana_prediccion, text="Análisis de Riesgo en Tiempo Real", bg="#1a1a1a", fg="white", font=("Arial", 16))
        titulo.pack(pady=10)

        self.label_resultado = tk.Label(self.ventana_prediccion, text="", bg="#1a1a1a", fg="white", font=("Arial", 14))
        self.label_resultado.pack(pady=10)

        self.label_peligrosos = tk.Label(self.ventana_prediccion, text="Objetos Peligrosos: 0", bg="#1a1a1a", fg="red", font=("Arial", 12))
        self.label_peligrosos.pack(pady=5)

        self.label_seguros = tk.Label(self.ventana_prediccion, text="Objetos Seguros: 0", bg="#1a1a1a", fg="green", font=("Arial", 12))
        self.label_seguros.pack(pady=5)

        self.label_nivel_riesgo = tk.Label(self.ventana_prediccion, text="Nivel de riesgo: BAJO", bg="#1a1a1a", fg="white", font=("Arial", 14))
        self.label_nivel_riesgo.pack(pady=20)

        nota = tk.Label(self.ventana_prediccion, text="* Basado en análisis de objetos detectados y patrones históricos", bg="#1a1a1a", fg="white", font=("Arial", 10))
        nota.pack(pady=5)

        self.thread_actualizar_prediccion = Thread(target=self.actualizar_prediccion_en_ventana)
        self.thread_actualizar_prediccion.daemon = True
        self.thread_actualizar_prediccion.start()

    def actualizar_prediccion_en_ventana(self):
        while self.camara_activa and self.detectando_objetos:
            datos_entrada = self.generar_datos_prediccion()
            resultado = self.predecir_peligro(datos_entrada)

            # Verificar si la ventana aún está abierta
            if not self.ventana_prediccion.winfo_exists():
                break

            # Contar objetos peligrosos y seguros
            peligrosos = datos_entrada[0]
            no_peligrosos = datos_entrada[1]

            self.label_resultado.config(text=f"Datos analizados: {resultado}")
            self.label_peligrosos.config(text=f"Objetos Peligrosos: {peligrosos}")
            self.label_seguros.config(text=f"Objetos Seguros: {no_peligrosos}")

            if resultado == "alto":
                self.label_nivel_riesgo.config(text="Nivel de riesgo: ALTO")
            else:
                self.label_nivel_riesgo.config(text="Nivel de riesgo: BAJO")

            time.sleep(1)

    def abrir_camara(self):
        if not self.camara_activa:
            self.cap = cv2.VideoCapture(self.camara_index)
            if not self.cap.isOpened():
                messagebox.showerror("Error", "No se pudo abrir la cámara.")
                return

            self.camara_activa = True
            self.boton_abrir.config(state=tk.DISABLED)
            self.boton_cerrar.config(state=tk.NORMAL)
            self.boton_deteccion.config(state=tk.NORMAL)
            self.boton_voz.config(state=tk.NORMAL)
            self.boton_alarma.config(state=tk.NORMAL)
            self.boton_descargar.config(state=tk.NORMAL)
            self.boton_cambiar_camara.config(state=tk.NORMAL)
            self.boton_prediccion.config(state=tk.NORMAL)

            self.agregar_historial("Cámara abierta")

            self.thread_video = Thread(target=self.actualizar_video)
            self.thread_video.daemon = True
            self.thread_video.start()

    def actualizar_video(self):
        while self.camara_activa:
            ret, frame = self.cap.read()
            if not ret:
                break

            if self.detectando_objetos:
                self.procesar_frame(frame)

            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_pil = Image.fromarray(frame_rgb)
            frame_tk = ImageTk.PhotoImage(frame_pil)
            self.canvas.create_image(0, 0, anchor=tk.NW, image=frame_tk)
            self.canvas.image = frame_tk

    def procesar_frame(self, frame):
        blob = cv2.dnn.blobFromImage(frame, 0.00392, (320, 320), (0, 0, 0), True, crop=False)
        self.net.setInput(blob)
        outs = self.net.forward(self.output_layers)

        class_ids = []
        confidences = []
        boxes = []

        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.6:
                    center_x = int(detection[0] * frame.shape[1])
                    center_y = int(detection[1] * frame.shape[0])
                    w = int(detection[2] * frame.shape[1])
                    h = int(detection[3] * frame.shape[0])
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)
                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.6, 0.4)

        objetos_detectados = []
        for i in indexes:
            x, y, w, h = boxes[i]
            label = str(self.classes[class_ids[i]])
            confidence = confidences[i] * 100
            objetos_detectados.append({'label': label, 'confidence': confidence})
            color = (0, 255, 0)
            if label in ["knife", "pistol", "scissors"]:
                color = (255, 0, 0)
                self.reproducir_alerta()  # Llamar a la función de alerta

            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, f"{label} {confidence:.2f}%", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Anunciar el objeto detectado
            if self.voz_activar and label not in self.objetos_para_anunciar:
                self.hablar(f"Detectado: {label}")
                self.objetos_para_anunciar.add(label)

        self.predicciones = objetos_detectados
        self.agregar_historial(f"Objetos detectados: {', '.join([obj['label'] for obj in objetos_detectados])}")

    def reproducir_alerta(self):
        if self.alarma_activar:
            pygame.mixer.music.load("alerta.mp3")  # Asegúrate de tener un archivo de sonido llamado alerta.mp3
            pygame.mixer.music.play()

    def cerrar_camara(self):
        if self.camara_activa:
            self.camara_activa = False
            
            if self.cap is not None:
                self.cap.release()
            
            self.boton_abrir.config(state=tk.NORMAL)
            self.boton_cerrar.config(state=tk.DISABLED)
            self.boton_deteccion.config(state=tk.DISABLED)
            self.boton_voz.config(state=tk.DISABLED)
            self.boton_alarma.config(state=tk.DISABLED)
            self.boton_descargar.config(state=tk.DISABLED)
            self.boton_cambiar_camara.config(state=tk.DISABLED)

            self.agregar_historial("Cámara cerrada")

    def cambiar_camara(self):
        self.camara_index = 1 if self.camara_index == 0 else 0
        self.cerrar_camara()
        self.abrir_camara()
        self.agregar_historial(f"Cámara cambiada a {'Cámara web externa' if self.camara_index == 0 else 'Cámara de la laptop'}")

    def agregar_historial(self, mensaje):
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        self.historial_detecciones.append(f"{timestamp}: {mensaje}")
        self.text_historial.config(state=tk.NORMAL)
        self.text_historial.insert(tk.END, f"{timestamp}: {mensaje}\n")
        self.text_historial.config(state=tk.DISABLED)

    def toggle_deteccion(self):
        self.detectando_objetos = not self.detectando_objetos
        estado = "activada" if self.detectando_objetos else "desactivada"
        self.agregar_historial(f"Detección de objetos {estado}")

        # Detener la voz si se desactiva la detección
        if not self.detectando_objetos and self.voz_activar:
            self.engine.stop()
            self.objetos_para_anunciar.clear()

    def toggle_voz(self):
        self.voz_activar = not self.voz_activar
        estado = "activada" if self.voz_activar else "desactivada"
        self.agregar_historial(f"Voz {estado}")

        # Cambiar el texto del botón
        self.boton_voz.config(text="Desactivar Voz" if self.voz_activar else "Activar Voz")

    def toggle_alarma(self):
        self.alarma_activar = not self.alarma_activar
        estado = "activada" if self.alarma_activar else "desactivada"
        self.agregar_historial(f"Alarma {estado}")

    def descargar_historial(self):
        if self.historial_detecciones:
            archivo = filedialog.asksaveasfilename(defaultextension=".txt",
                                                    filetypes=[("Archivos de texto", "*.txt")])
            if archivo:
                with open(archivo, "w") as f:
                    for mensaje in self.historial_detecciones:
                        f.write(mensaje + "\n")
                messagebox.showinfo("Descargar Historial", "Historial guardado correctamente.")
        else:
            messagebox.showwarning("Descargar Historial", "No hay historial para guardar.")

    def salir(self):
        self.cerrar_camara()
        self.root.quit()

# Crear ventana principal
root = tk.Tk()
root.geometry("1500x1100")
root.resizable(False, False)
app = CamaraApp(root)
root.mainloop()

In [ ]:
import tkinter as tk
from tkinter import messagebox, Scrollbar, Frame, filedialog
from threading import Thread
import cv2
import numpy as np
from PIL import Image, ImageTk
import pyttsx3
import os
import time
import pygame
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
import random
from datetime import datetime
from queue import Queue

class CamaraApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Detector de imágenes - Demo")
        self.root.config(bg="#2e2e2e")

        pygame.mixer.init()

        self.camara_activa = False
        self.cap = None
        self.detectando_objetos = False
        self.engine = pyttsx3.init()
        self.historial_detecciones = []
        self.objetos_para_anunciar = set()
        self.predicciones = []
        self.last_alert_time = 0
        self.voz_activar = True
        self.alarma_activar = True
        self.camara_index = 0
        self.voice_queue = Queue()
        self.thread_voice = Thread(target=self.process_voice_queue)
        self.thread_voice.daemon = True
        self.thread_voice.start()

        self.modelo_entrenado = False
        self.entrenar_modelo_ejemplo()

        self.net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
        self.layer_names = self.net.getLayerNames()
        self.output_layers = [self.layer_names[i - 1] for i in self.net.getUnconnectedOutLayers()]
        self.classes = []
        with open("coco.names", "r") as f:
            self.classes = [line.strip() for line in f.readlines()]

        self.mostrar_tutorial()
        self.setup_ui()

    def mostrar_tutorial(self):
        tutorial_window = tk.Toplevel(self.root)
        tutorial_window.title("Tutorial")
        tutorial_window.geometry("500x400")
        tutorial_window.config(bg="#1a1a1a")

        titulo = tk.Label(tutorial_window, text="Bienvenido al Detector de Imágenes", bg="#1a1a1a", fg="white", font=("Arial", 16))
        titulo.pack(pady=10)

        tutorial_text = (
            "Esta herramienta permite detectar objetos peligrosos en tiempo real.\n\n"
            "1. Abre la cámara.\n"
            "2. Activa la detección de objetos.\n"
            "3. Recibe alertas por voz y visuales.\n"
            "4. Cierra la cámara cuando termines."
        )
        instrucciones = tk.Label(tutorial_window, text=tutorial_text, bg="#1a1a1a", fg="white", font=("Arial", 12))
        instrucciones.pack(pady=20)

        boton_omitir = tk.Button(tutorial_window, text="Cerrar Tutorial", command=tutorial_window.destroy, bg="#4CAF50", fg="white")
        boton_omitir.pack(pady=10)

        tutorial_window.protocol("WM_DELETE_WINDOW", tutorial_window.destroy)

    def setup_ui(self):
        self.frame_principal = tk.Frame(self.root, bg="#2e2e2e")
        self.frame_principal.pack(fill=tk.BOTH, expand=True)

        self.frame_video = tk.Frame(self.frame_principal, bg="#2e2e2e")
        self.frame_video.pack(side=tk.LEFT, padx=10, pady=10)

        self.cargar_logo()

        self.frame_historial = Frame(self.frame_principal)
        self.frame_historial.pack(side=tk.RIGHT, fill=tk.Y, padx=10, pady=10)

        self.scrollbar = Scrollbar(self.frame_historial)
        self.scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        self.label_historial_titulo = tk.Label(self.frame_historial, text="Historial", bg="#2e2e2e",
                                               fg="white", font=("Arial", 14, "bold"))
        self.label_historial_titulo.pack(pady=10)

        self.text_historial = tk.Text(self.frame_historial, bg="#2e2e2e", fg="white", font=("Arial", 12),
                                       wrap="word", height=20, width=40)
        self.text_historial.pack(fill=tk.BOTH, expand=True, padx=10)
        self.text_historial.config(yscrollcommand=self.scrollbar.set)
        self.scrollbar.config(command=self.text_historial.yview)

        self.canvas = tk.Canvas(self.frame_video, width=640, height=480)
        self.canvas.pack()

        self.frame_botones = tk.Frame(self.frame_video, bg="#3c3c3c")
        self.frame_botones.pack(pady=10, padx=10, fill=tk.X)

        self.boton_abrir = tk.Button(self.frame_botones, text="Abrir Cámara", command=self.abrir_camara, width=20,
                                     bg="#4CAF50", fg="white", font=("Arial", 12))
        self.boton_abrir.grid(row=0, column=0, padx=10, pady=10)

        self.boton_cerrar = tk.Button(self.frame_botones, text="Cerrar Cámara", command=self.cerrar_camara, width=20,
                                      state=tk.DISABLED, bg="#F44336", fg="white", font=("Arial", 12))
        self.boton_cerrar.grid(row=0, column=1, padx=10, pady=10)

        self.boton_deteccion = tk.Button(self.frame_botones, text="Detectar Objetos", command=self.toggle_deteccion,
                                         width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_deteccion.grid(row=1, column=0, padx=10, pady=10)

        self.boton_voz = tk.Button(self.frame_botones, text="Activar Voz", command=self.toggle_voz,
                                   width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_voz.grid(row=1, column=1, padx=10, pady=10)

        self.boton_alarma = tk.Button(self.frame_botones, text="Activar Alarma", command=self.toggle_alarma,
                                      width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_alarma.grid(row=2, column=0, padx=10, pady=10)

        self.boton_descargar = tk.Button(self.frame_botones, text="Guardar Historial", command=self.descargar_historial,
                                          width=20, state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_descargar.grid(row=2, column=1, padx=10, pady=10)

        self.boton_cambiar_camara = tk.Button(self.frame_botones, text="Cambiar Cámara", command=self.cambiar_camara, width=20,
                                              state=tk.DISABLED, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.boton_cambiar_camara.grid(row=3, column=0, padx=10, pady=10)

        self.boton_prediccion = tk.Button(self.frame_botones, text="Realizar Predicción", command=self.mostrar_prediccion,
                                           width=20, state=tk.DISABLED, bg="#009688", fg="white", font=("Arial", 12))
        self.boton_prediccion.grid(row=3, column=1, padx=10, pady=10)

        self.boton_salir = tk.Button(self.frame_botones, text="Salir", command=self.salir, width=20,
                                     bg="#9E9E9E", fg="white", font=("Arial", 12))
        self.boton_salir.grid(row=4, column=0, columnspan=2, padx=10, pady=10)

    def cargar_logo(self):
        try:
            logo_path = "Logo1.png"
            if os.path.exists(logo_path):
                self.logo_imagen = Image.open(logo_path).resize((300, 100), Image.Resampling.LANCZOS)
                self.logo_imagen_tk = ImageTk.PhotoImage(self.logo_imagen)
                self.label_logo = tk.Label(self.frame_video, image=self.logo_imagen_tk, bg="#2e2e2e")
                self.label_logo.pack(pady=20)
        except Exception as e:
            print(f"Error al cargar la imagen: {e}")

    def hablar(self, mensaje):
        if self.voz_activar:  # Solo agregar a la cola si la voz está activada
            self.voice_queue.put(mensaje)

    def process_voice_queue(self):
        while True:
            mensaje = self.voice_queue.get()
            if self.voz_activar:  # Verificar si la voz está activada
                self.engine.say(mensaje)
                self.engine.runAndWait()

    def entrenar_modelo_ejemplo(self):
        X = np.array([[0, 0, 1], [1, 0, 0], [0, 1, 0], [1, 1, 0], [0, 0, 0]])
        y = np.array(['bajo', 'alto', 'medio', 'alto', 'bajo'])
        
        self.le = LabelEncoder()
        y_encoded = self.le.fit_transform(y)
        
        self.modelo = MLPClassifier(hidden_layer_sizes=(10, 5), max_iter=5000, learning_rate_init=0.01)
        self.modelo.fit(X, y_encoded)
        self.modelo_entrenado = True

    def predecir_peligro(self, datos):
        if self.modelo_entrenado:
            prediccion_encoded = self.modelo.predict([datos])
            return self.le.inverse_transform(prediccion_encoded)[0]
        return "Error en el modelo"

    def generar_datos_prediccion(self):
        if len(self.predicciones) == 0:
            return [0, 0, 0]
        
        # Contar solo los objetos peligrosos detectados en la cámara
        peligrosos = sum(1 for obj in self.predicciones if obj['label'] in ["knife", "pistol", "scissors", "baseball bat", "bottle", "fire hydrant", "fork", "tennis racket", "umbrella"])
        no_peligrosos = len(self.predicciones) - peligrosos
        return [peligrosos, no_peligrosos, random.randint(0, 1)]

    def mostrar_prediccion(self):
        self.ventana_prediccion = tk.Toplevel(self.root)
        self.ventana_prediccion.title("Análisis de Riesgo")
        self.ventana_prediccion.geometry("400x300")
        self.ventana_prediccion.config(bg="#1a1a1a")

        titulo = tk.Label(self.ventana_prediccion, text="Análisis de Riesgo en Tiempo Real", bg="#1a1a1a", fg="white", font=("Arial", 16))
        titulo.pack(pady=10)

        self.label_resultado = tk.Label(self.ventana_prediccion, text="", bg="#1a1a1a", fg="white", font=("Arial", 14))
        self.label_resultado.pack(pady=10)

        self.label_peligrosos = tk.Label(self.ventana_prediccion, text="Objetos Peligrosos: 0", bg="#1a1a1a", fg="red", font=("Arial", 12))
        self.label_peligrosos.pack(pady=5)

        self.label_seguros = tk.Label(self.ventana_prediccion, text="Objetos Seguros: 0", bg="#1a1a1a", fg="green", font=("Arial", 12))
        self.label_seguros.pack(pady=5)

        self.label_nivel_riesgo = tk.Label(self.ventana_prediccion, text="Nivel de riesgo: BAJO", bg="#1a1a1a", fg="white", font=("Arial", 14))
        self.label_nivel_riesgo.pack(pady=20)

        nota = tk.Label(self.ventana_prediccion, text="* Basado en análisis de objetos detectados y patrones históricos", bg="#1a1a1a", fg="white", font=("Arial", 10))
        nota.pack(pady=5)

        self.thread_actualizar_prediccion = Thread(target=self.actualizar_prediccion_en_ventana)
        self.thread_actualizar_prediccion.daemon = True
        self.thread_actualizar_prediccion.start()

    def actualizar_prediccion_en_ventana(self):
        while self.camara_activa and self.detectando_objetos:
            datos_entrada = self.generar_datos_prediccion()
            resultado = self.predecir_peligro(datos_entrada)

            # Verificar si la ventana aún está abierta
            if not self.ventana_prediccion.winfo_exists():
                break

            # Contar objetos peligrosos y seguros
            peligrosos = datos_entrada[0]
            no_peligrosos = datos_entrada[1]

            self.label_resultado.config(text=f"Datos analizados: {resultado}")
            self.label_peligrosos.config(text=f"Objetos Peligrosos: {peligrosos}")
            self.label_seguros.config(text=f"Objetos Seguros: {no_peligrosos}")

            if resultado == "alto":
                self.label_nivel_riesgo.config(text="Nivel de riesgo: ALTO")
            else:
                self.label_nivel_riesgo.config(text="Nivel de riesgo: BAJO")

            time.sleep(1)

    def abrir_camara(self):
        if not self.camara_activa:
            self.cap = cv2.VideoCapture(self.camara_index)
            if not self.cap.isOpened():
                messagebox.showerror("Error", "No se pudo abrir la cámara.")
                return

            self.camara_activa = True
            self.boton_abrir.config(state=tk.DISABLED)
            self.boton_cerrar.config(state=tk.NORMAL)
            self.boton_deteccion.config(state=tk.NORMAL)
            self.boton_voz.config(state=tk.NORMAL)
            self.boton_alarma.config(state=tk.NORMAL)
            self.boton_descargar.config(state=tk.NORMAL)
            self.boton_cambiar_camara.config(state=tk.NORMAL)
            self.boton_prediccion.config(state=tk.NORMAL)

            self.agregar_historial("Cámara abierta")

            self.thread_video = Thread(target=self.actualizar_video)
            self.thread_video.daemon = True
            self.thread_video.start()

    def actualizar_video(self):
        while self.camara_activa:
            ret, frame = self.cap.read()
            if not ret:
                break

            if self.detectando_objetos:
                self.procesar_frame(frame)

            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_pil = Image.fromarray(frame_rgb)
            frame_tk = ImageTk.PhotoImage(frame_pil)
            self.canvas.create_image(0, 0, anchor=tk.NW, image=frame_tk)
            self.canvas.image = frame_tk

    def procesar_frame(self, frame):
        blob = cv2.dnn.blobFromImage(frame, 0.00392, (320, 320), (0, 0, 0), True, crop=False)
        self.net.setInput(blob)
        outs = self.net.forward(self.output_layers)

        class_ids = []
        confidences = []
        boxes = []

        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.6:
                    center_x = int(detection[0] * frame.shape[1])
                    center_y = int(detection[1] * frame.shape[0])
                    w = int(detection[2] * frame.shape[1])
                    h = int(detection[3] * frame.shape[0])
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)
                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.6, 0.4)

        objetos_detectados = []
        for i in indexes:
            x, y, w, h = boxes[i]
            label = str(self.classes[class_ids[i]])
            confidence = confidences[i] * 100
            objetos_detectados.append({'label': label, 'confidence': confidence})
            color = (0, 255, 0)
            if label in ["knife", "pistol", "scissors", "baseball bat", "bottle", "fire hydrant", "fork", "tennis racket", "umbrella"]:
                color = (255, 0, 0)
                self.reproducir_alerta()  # Llamar a la función de alerta

            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, f"{label} {confidence:.2f}%", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Anunciar el objeto detectado
            if self.voz_activar and label not in self.objetos_para_anunciar:
                self.hablar(f"Detectado: {label}")
                self.objetos_para_anunciar.add(label)

        self.predicciones = objetos_detectados
        self.agregar_historial(f"Objetos detectados: {', '.join([obj['label'] for obj in objetos_detectados])}")

    def reproducir_alerta(self):
        if self.alarma_activar:
            pygame.mixer.music.load("alerta.mp3")  # Asegúrate de tener un archivo de sonido llamado alerta.mp3
            pygame.mixer.music.play()

    def cerrar_camara(self):
        if self.camara_activa:
            self.camara_activa = False
            
            if self.cap is not None:
                self.cap.release()
            
            self.boton_abrir.config(state=tk.NORMAL)
            self.boton_cerrar.config(state=tk.DISABLED)
            self.boton_deteccion.config(state=tk.DISABLED)
            self.boton_voz.config(state=tk.DISABLED)
            self.boton_alarma.config(state=tk.DISABLED)
            self.boton_descargar.config(state=tk.DISABLED)
            self.boton_cambiar_camara.config(state=tk.DISABLED)

            self.agregar_historial("Cámara cerrada")

    def cambiar_camara(self):
        self.camara_index = 1 if self.camara_index == 0 else 0
        self.cerrar_camara()
        self.abrir_camara()
        self.agregar_historial(f"Cámara cambiada a {'Cámara web externa' if self.camara_index == 0 else 'Cámara de la laptop'}")

    def agregar_historial(self, mensaje):
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        self.historial_detecciones.append(f"{timestamp}: {mensaje}")
        self.text_historial.config(state=tk.NORMAL)
        self.text_historial.insert(tk.END, f"{timestamp}: {mensaje}\n")
        self.text_historial.config(state=tk.DISABLED)

    def toggle_deteccion(self):
        self.detectando_objetos = not self.detectando_objetos
        estado = "activada" if self.detectando_objetos else "desactivada"
        self.agregar_historial(f"Detección de objetos {estado}")

        # Detener la voz si se desactiva la detección
        if not self.detectando_objetos and self.voz_activar:
            self.engine.stop()
            self.objetos_para_anunciar.clear()

    def toggle_voz(self):
        self.voz_activar = not self.voz_activar
        estado = "activada" if self.voz_activar else "desactivada"
        self.agregar_historial(f"Voz {estado}")

        # Cambiar el texto del botón
        self.boton_voz.config(text="Desactivar Voz" if self.voz_activar else "Activar Voz")

    def toggle_alarma(self):
        self.alarma_activar = not self.alarma_activar
        estado = "activada" if self.alarma_activar else "desactivada"
        self.agregar_historial(f"Alarma {estado}")

    def descargar_historial(self):
        if self.historial_detecciones:
            archivo = filedialog.asksaveasfilename(defaultextension=".txt",
                                                    filetypes=[("Archivos de texto", "*.txt")])
            if archivo:
                with open(archivo, "w") as f:
                    for mensaje in self.historial_detecciones:
                        f.write(mensaje + "\n")
                messagebox.showinfo("Descargar Historial", "Historial guardado correctamente.")
        else:
            messagebox.showwarning("Descargar Historial", "No hay historial para guardar.")

    def salir(self):
        self.cerrar_camara()
        self.root.quit()

# Crear ventana principal
root = tk.Tk()
root.geometry("1500x1100")
root.resizable(False, False)
app = CamaraApp(root)
root.mainloop()